<a href="https://colab.research.google.com/github/eisbetterthanpi/ssm/blob/main/Mamba_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title hf byte dataset me
!pip install -qU datasets # restart?
import torch
from torch.utils.data import IterableDataset

class StreamDataset(IterableDataset):
    def __init__(self, dataset, seq_len=129, buffer_size=1024):
        self.vocab_size = 256 # utf-8 # self.enc.n_vocab # gpt2:50257
        self.data = iter(dataset)
        self.seq_len, self.buffer_size = seq_len, buffer_size  # must be ≥ seq_len
        self.buffer = []  # token buffer
        self.fill_buffer()

    def fill_buffer(self):
        while len(self.buffer) < self.buffer_size:
            x = next(self.data)
            # print(x)
            self.buffer.extend(x['text'].encode("utf-8"))

    def __iter__(self):
        while True:
            if len(self.buffer) < self.seq_len: self.fill_buffer()
            if len(self.buffer) < self.seq_len: return # raise StopIteration
            x, self.buffer = self.buffer[:self.seq_len], self.buffer[self.seq_len:]
            yield torch.tensor(x, dtype=torch.int32) # uint8 int32

from datasets import load_dataset
name = 'Skylion007/openwebtext' if torch.cuda.is_available() else 'stas/openwebtext-10k'
dataset = load_dataset(name, split="train", streaming=True, revision='refs/convert/parquet', cache_dir="/content/hf")

seq_len = 2**7+1 # 128
buffer_size = seq_len*1
train_data = StreamDataset(dataset, seq_len, buffer_size) # train_data = StreamDataset(dataset["train"], seq_len, buffer_size)
# del dataset

from torch.utils.data.dataloader import DataLoader
batch_size = 64 if torch.cuda.is_available() else 16 #64 512
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=0)
# del train_data
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# https://github.com/facebookresearch/blt/blob/main/bytelatent/tokenizers/blt_tokenizer.py#L137
# def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.uint8)#, device=device)#.unsqueeze(0)
def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.int32, device=device).unsqueeze(0)
def decode(x): return bytes(x.tolist()).decode("utf-8", errors='replace') # replace ignore

# for x in train_loader:
#     print(x.shape, x)
#     break
# print(decode(x[0][:64]))


In [ ]:
# @title ssd next
# https://goombalab.github.io/blog/2024/mamba2-part4-systems/
# https://tridao.me/blog/2024/mamba2-part3-algorithm/
# # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/ssd_combined.py
# y = mamba_chunk_scan_combined(x.unflatten(-1, (self.n_heads, self.d_head)), dt, A, # chunk_size=256,
#     B.unflatten(-1, (self.ngroups, self.d_state)), C.unflatten(-1, (self.ngroups, self.d_state)),
#     D=self.D, z=None).flatten(2) # norm before flatten? so norm is within group
import torch

def segsum(x): # [...,blk] # Naive segment sum calculation. exp(segsum(A)) produces a 1-SS matrix, which is equivalent to a scalar SSM.
    T = x.size(-1)
    x_cumsum = torch.cumsum(x, dim=-1)
    x_segsum = x_cumsum.unsqueeze(-1) - x_cumsum.unsqueeze(-2) # [...,blk,blk]
    mask = torch.triu(torch.ones(T, T, device=x.device, dtype=bool), diagonal=1)
    return x_segsum.masked_fill(mask, -torch.inf) # [...,blk,blk]

# c:t/blk / 1+t/blk
# l/s:blk
# n:s
# p:d
# z:1+t/blk
def ssd(X, A, B, C, block_len=64, h0=None): # X:[b,t,h,d], A:[b,t,h], B:[b,t,h,s], C:[b,t,h,s], h0:[b,h,d,s]
    assert X.dtype == A.dtype == B.dtype == C.dtype
    # assert X.shape[1] % block_len == 0
    if X.shape[1] % block_len != 0: X, A, B, C = [x.unsqueeze(1) for x in (X, A, B, C)]
    else: X, A, B, C = [x.unflatten(1, (-1,block_len)) for x in (X, A, B, C)]
    A = A.permute(0,3,1,2) # [b,t/blk,blk,h] -> [b,h,t/blk,blk]
    A_cumsum = torch.cumsum(A, dim=-1) # [b,h,t/blk,blk]

    # 1. Compute the output for each intra-chunk (diagonal blocks)
    L = torch.exp(segsum(A)) # [b,h,t/blk,blk,blk]
    Y_diag  = torch.einsum("bclhn,bcshn,bhcls,bcshp->bclhp", C, B, L, X) # [b,t/blk,blk,h,d]

    # 2. Compute the state for each intra-chunk # (right term of low-rank factorization of off-diagonal blocks; B terms)
    decay_states = torch.exp((A_cumsum[...,-1:] - A_cumsum)) # [b,h,t/blk,blk]
    states = torch.einsum("bclhn,bhcl,bclhp->bchpn", B, decay_states, X) # [b,t/blk,h,d,s]

    # 3. Compute the inter-chunk SSM recurrence; produces correct SSM states at chunk boundaries # (middle term of factorization of off-diag blocks; A terms)
    if h0==None: h0 = torch.zeros_like(states[:,0], device=device) # [b,h,d,s]
    states = torch.cat([h0.unsqueeze(1), states], dim=1) # [b,1+t/blk,h,d,s]
    decay_chunk = torch.exp(segsum(F.pad(A_cumsum[...,-1], (1,0)))) # [b,h,1+t/blk]-> # [b,h,1+t/blk,1+t/blk]
    new_states = torch.einsum("bhzc,bchpn->bzhpn", decay_chunk, states) # [b,1+t/blk,h,d,s]

    # 4. Compute state -> output conversion per chunk # (left term of low-rank factorization of off-diagonal blocks; C terms)
    Y_off = torch.einsum('bclhn,bchpn,bhcl->bclhp', C, new_states[:,:-1], torch.exp(A_cumsum)) # [b,t/blk,blk,h,d]

    # Add output of intra-chunk and inter-chunk terms (diagonal and off-diagonal blocks)
    Y = (Y_diag+Y_off).flatten(1,2)
    return Y, new_states[:,-1] # [b,t,h,d], [b,h,d,s]


In [ ]:
# @title appendix

T Time /target sequence axis
S Source sequence axis
D d_model Model dimension
N d_state State/feature dimension
P d_head
H n_head
G independent K and V heads, where 1 < G and G divides H

d_model, 2560
d_state=64,
d_conv=4,
expand=2,
headdim=128,
ngroups=1,


MQA < 1HA ~ MHA < MVA
X~V; B~K; C~Q

𝐴, 𝐵,𝐶 parameters have a state dimension N. ∈ R (T,T) .
operates over 𝑋 ∈ R (T,P) , independently over the P axis. -> is one head
H independent heads, for a total model dimension of D = d_model. The parameters may be tied across heads, leading to a head pattern.
state size N ~ 𝑄𝐾 head dimension ; head dimension P ~ 𝑉 head dimension
 when the model dimension D increases, we increase the number of heads while keeping the head dimensions N and P fixed

d_inner = expand * d_model = nheads * headdim


X/z: b,l,n_head,d_head    blhp
B/C: b,l,ngroups,d_state    blgn
dt: nheads
A: nheads
D: nheads


https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/selective_state_update.py#L104
state: (batch, dim, dstate) or (batch, nheads, dim, dstate)
x: (batch, dim) or (batch, nheads, dim)/
dt: (batch, dim) or (batch, nheads, dim) # nheads
A: (dim, dstate) or (nheads, dim, dstate) # nheads
B: (batch, dstate) or (batch, ngroups, dstate)/
C: (batch, dstate) or (batch, ngroups, dstate)/
D: (dim,) or (nheads, dim) # nheads
z: (batch, dim) or (batch, nheads, dim)/
dt_bias: (dim,) or (nheads, dim) # nheads


# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py
A = repeat(
    torch.arange(1, self.d_state + 1, dtype=torch.float32, device=device),
    "n -> d n",
    d=self.d_inner,
).contiguous()

ngroups1
expand



In [ ]:
# @title mamba_ssm/modules/mamba2_simple.py next
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Mamba2(nn.Module):
    # def __init__(self, d_model, expand=2, n_heads=8, n_groups=1, d_state=64, d_conv=4):
    # def __init__(self, d_model, expand=2, n_heads=8, n_groups=4, d_state=8, d_conv=4):
    def __init__(self, d_model, expand=2, n_heads=8, n_groups=8, d_state=8, d_conv=4):
        super().__init__()
        n_groups = min(n_heads, n_groups)
        assert n_heads % n_groups == 0, "nheads must be divisible by ngroups"
        self.d_model, self.n_groups, self.d_state, self.d_conv = d_model, n_groups, d_state, d_conv
        self.d_inner = expand * self.d_model
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.d_model, 2* self.d_inner + 2* self.n_groups*self.d_state + self.n_heads, bias=False) # z,x,B,C,dt
        conv_dim = self.d_inner + 2* self.n_groups*self.d_state # for x,B,C
        self.conv1d = nn.Conv1d(conv_dim, conv_dim, kernel_size=d_conv, groups=conv_dim, bias=True)
        # self.conv1d.weight._no_weight_decay = True
        self.act = nn.SiLU()

        # self.h0 = nn.Parameter(torch.zeros(self.n_heads, self.d_head, self.d_state))
        # self.h0._no_weight_decay = True
        dt_min, dt_max = .001, .1
        dt = torch.exp(torch.rand(self.n_heads) * (math.log(dt_max)-math.log(dt_min)) + math.log(dt_min)).clamp(min=1e-4)
        self.dt_bias = nn.Parameter(dt + torch.log(-torch.expm1(-dt))) # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        self.dt_bias._no_weight_decay = True
        A = torch.empty(self.n_heads, dtype=torch.float32).uniform_(1,16) # 1,16 ; .2,2
        self.A_log = nn.Parameter(torch.log(A))
        self.A_log._no_weight_decay = True
        self.D = nn.Parameter(torch.ones(self.n_heads))
        self.D._no_weight_decay = True
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    def forward(self, u, h=None): # [b,t,d], ([b,k-1,xbc], [b,h,d,s])
        # print('Mamba2Simple u', u.shape)
        A = -torch.exp(self.A_log) # [n_heads]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_groups*self.d_state, self.n_heads], dim=-1)
        dt = F.softplus(dt+self.dt_bias) # [b,t,h]

        if h==None:
            h_conv = F.pad(xBC, (0,0,self.d_conv-u.shape[1]-1,0)) # [b,k-1,xbc]
            h_ssm = torch.zeros(u.shape[0], self.n_heads, self.d_head, self.d_state, device=device) # [b,h,d,s]
            # h_ssm = self.h0.expand(u.size(0),-1,-1,-1) # [b,h,d,s]
            xBC = F.pad(xBC, (0,0,self.d_conv-1,0)) # [b,t+d_conv-1,xbc]
        else:
            h_conv, h_ssm = h
            xBC = torch.cat([h_conv, xBC], dim=1) # [b,t+d_conv-1,xbc]

        # print(h_conv.shape, xBC.shape)
        # xBC = self.act(xBC + self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1))
        xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)) # [b,t+d_conv-1,xbc]->[b,t,xbc]
        x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))
        h_g = self.n_heads//self.n_groups
        if h_g>1: B, C = B.repeat_interleave(h_g, dim=-2), C.repeat_interleave(h_g, dim=-2) # [b,g,s]->[b,h,s]

        # print('x dt a b', x.shape, dt.shape, A.shape, B.shape)
        y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h_ssm) # 256

        y = y + x * self.D.unsqueeze(-1)
        # y = self.norm(y.flatten(2) * self.act(z)) # [b,t,d_inner] # norm(x)*silu(z) if norm_before_gate, else norm(x*silu(z)) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/layernorm_gated.py#L18
        y = self.norm(y.flatten(2)) * self.act(z)
        out = self.out_proj(y)
        return out, (h_conv, h_ssm) # [b,t,in], ([b,k-1,xbc], [b,h,d,s])


# b,t,d_model=5,256,32
b,t,d_model=5,7,32
d_model = 64
u = torch.randn(b,t,d_model, device=device)
model = Mamba2(d_model).to(device)
out, h = model(u)
# h0 = torch.randn(b, model.n_heads, model.d_head, model.d_state)
# print(out.shape)
# print(out.shape, h.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])
out, h = model(u, h)
h_conv, h_ssm = h
print(h_conv.shape, h_ssm.shape)

# u = torch.randn(b,7,d_model, device=device)
# # out, h = model(u[:,-1:], h)
# print(out.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])


# cpu seq ~40-41s
# this is what sherti, epow sour wocks I rmtre herring' Coer alaid it the adoo
# 700 time: 43.38966917991638 0.3799727667075251
# strain 2.045107126235962
# this is whate lilln the Tont to pleesyecise merefes asse sperced is a a cagk

# gpu ssd seqlen 2^11 29.5s

# n_groups, vs d_state



In [ ]:
# print(model.blk[0].rnn[0].A_log.exp())
# print(model.blk.rnn[1].A_log.exp())
print(model.blk[0].rnn.A_log.exp())
print(model.blk[1].rnn.A_log.exp())
# [0.7099, 0.6012, 0.6362, 0.6542, 0.6469, 0.5790, 0.5655, 0.8270],
# [1.1237, 0.4842, 0.8551, 1.5170, 0.5162, 0.9203, 0.7363, 0.9541]
# [1.2897, 0.8572, 0.6168, 1.1675, 1.4585, 0.8542, 1.2247, 1.2768],
# [1.1875, 0.9908, 1.3625, 1.2116, 1.2327, 0.8914, 1.1673, 1.1140],


In [ ]:
# @title Block
# import math
import torch
import torch.nn as nn
# import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# def zero_module(module):
#     for p in module.parameters():
#         p.detach().zero_()
#     return module

# class SwiGLU(nn.Module): # https://arxiv.org/pdf/2002.05202
#     def __init__(self, d_model, ff_dim): # d_model * 3*ff_dim params
#         super().__init__()
#         self.lin0 = nn.Linear(d_model, 2*ff_dim, bias=False)
#         self.lin1 = zero_module(nn.Linear(ff_dim, d_model, bias=False))
#         # torch.nn.init.normal_(self.lin0.weight, std=.02)
#         torch.nn.init.normal_(self.lin0.weight, std=1/(math.sqrt(d_model)+math.sqrt(ff_dim)))

#     def forward(self, x): # [b,t,d]
#         x0, x1 = self.lin0(x).chunk(2, dim=-1)
#         return self.lin1(x0*F.silu(x1))

class Block(nn.Module):
    def __init__(self, d_model, n_heads, ff_mult=4, drop=0):
        super().__init__()
        self.d_model = d_model
        self.norm1 = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        # self.norm2 = nn.RMSNorm(d_model)
        # self.drop = nn.Dropout(drop)
        self.rnn = Mamba2(d_model, n_heads=n_heads)
        # act = nn.SiLU() # GELU SiLU ReLU
        # ff_dim=d_model*ff_mult#4
        # # if ff_dim==None: ff_dim=int(d_model*3/2)#4
        # self.ff = nn.Sequential(
        #     nn.RMSNorm(d_model), nn.Linear(d_model, ff_dim), act,
        #     nn.RMSNorm(ff_dim), nn.Dropout(drop), zero_module(nn.Linear(ff_dim, d_model))
        # )
        # torch.nn.init.normal_(self.ff[1].weight, std=.02)
        # # torch.nn.init.normal_(self.ff[1].weight, std=1/(math.sqrt(d_model)+math.sqrt(ff_dim)))
        # self.ff = SwiGLU(d_model, ff_dim)

    def forward(self, x, h=None): # [b,t,d], (,)
        # print('blk fwd', x.shape, h.shape)
        x_, h = self.rnn(self.norm1(x), h) #
        x = x + x_
        # x = x + self.ff(x)
        return x, h

b,t,d = 2,256,16
# b,t,d = 2,5,256
x = torch.rand(b,t,d, device=device)
model = Block(d_model=d, n_heads=4).to(device)
# model = Seq(*[Block(d_model=d, n_heads=4) for _ in range(2)])
out, h = model(x)
print(out.shape)

# import time
# start = time.time()
# # out = model(x)
# print(out.shape)
# print(time.time()-start)


In [ ]:
# @title GPT
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
    def forward(self, x, hid=None):
        hidden = []
        for i, layer in enumerate(self):
            x, h = layer(x, hid[i] if hid!=None else None)
            hidden.append(h)
        return x, hidden

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=256, dropout=0):
        super().__init__()
        self.d_model, self.n_layers = d_model, n_layers
        self.tok_emb = nn.Embedding(in_dim+1, d_model)
        # self.blk = Seq(*[Mamba2(d_model, n_heads=n_heads) for _ in range(n_layers)])
        self.blk = Seq(*[Block(d_model, n_heads=n_heads) for _ in range(n_layers)])
        # self.out = nn.Linear(d_model, out_dim or in_dim)
        # self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying
        self.out = lambda x: x @ self.tok_emb.weight[:-1].T # [vocab+1,d]->[d,vocab] # weight tying for nn.Embedding
        emb_std = d_model**-.5
        nn.init.trunc_normal_(self.tok_emb.weight, mean=0, std=emb_std, a=-3*emb_std, b=3*emb_std) # https://github.com/facebookresearch/blt/blob/main/bytelatent/model/local_models.py#L136

    def forward(self, x, hid=None): # [b,t], [n_lyr,t,b,d]
        x = self.tok_emb(x) # [b,t,d]
        x, hid = self.blk(x, hid) # [b,t,d], list[b,t,t]
        x = self.out(x)
        return x, hid # [b,t,d], [n_lyr,t,b,d]

# gpt 2
# Parameters Layers dmodel
# 117M 12 768 gpt1
# 345M 24 1024
# 762M 36 1280
# 1542M 48 1600

try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50
# d1024 12lyr h16, 768 8 12, 512 6 8, 256 4 8, , 64 1 8
# 768 16 12, 768 12 18
# model = GPT(vocab_size, d_model=512, out_dim=vocab_size, n_layers=3, n_heads=16).to(device)
model = GPT(vocab_size, d_model=64, out_dim=vocab_size, n_layers=2, n_heads=8).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
# optim = torch.optim.AdamW(model.parameters(), 1e-3) # og betas=(0.9, 0.999), wd.01
optim = torch.optim.AdamW(model.parameters(), 1e-3, betas=(0.9, 0.95), weight_decay=1e-1) # og wd.01
# optim = torch.optim.AdamW(model.parameters(), 1e-3, weight_decay=1e-1) # og wd.01

# gradclip important
# betas, scheduler helps
# diff num_heads not much diff, ~h_dim32 is good
# tokemb init helps significantly
# noinit init.02 < stableinit?


x = torch.randint(0, vocab_size, (4, 128), device=device)
out, hid = model(x)
print(out.shape)
print(type(hid), len(hid))
print(type(hid[0]), len(hid[0]))
# print(hid[0].shape)
h_conv, h_ssm = hid[0]
print(h_conv.shape, h_ssm.shape)

# out, hid = model(x, hid)
# print(out.shape)
# # print(out.shape, hid.shape)
# # print(out)



# gpt d64 2lyr, 8,8head swiglu2 98624p
# this is whatik Acounteats'nltenk IsDplardiopeng-601teack anat A oned alrenow
# 700 time: 36.86555862426758 0.37161261685054414
# strain 2.3351149559020996
# this is whatt sas beme llot the the palerorste the theuf rald pecene has sed
# this is what incluer is powerly idaismed said.
# We why won woulded, unsers w
# 6400 time: 2.9560906887054443 0.029622032612194364
# strain 2.0638203620910645
# this is what I gooding conreselfle landing of the comberold. Then is in the




# b64,l128 8.0ram,10.0gpu l128*2:oom

# b64,l128 w64 8.2ram,6.8gpu
# b64,l128*2 w64 8.1ram,14.0gpu

# swa64 128*2 7.4,12.0

# d256 6.9,6.7

# dim128 3lyr 3.0,.7
# this is whats more of ietre'soni, they asari, your very
# they vate. This iat
# 9600 time: 5.7726781368255615 0.0006012580827081866
# strain 1.481871247291565
# this is what line wasn't good to both feel
# like a delieve able or so foll,

# mha
# seq128*4+1 3.0,4.3
# seq128*8+1 oom


# seq128*8+1 w64:oom w32:2.4,7.8

# lru 31s, strain 2


# mamba2 no n_groups; dmodel512? expand1
# this is what the nneed the asregative seem excibitions, politicals have not
# 600 time: 103.91108822822571 1.0513688605557663
# strain 2.6979057788848877
# this is what involved programmess about disclarating diologios indeedoon Att
# 700 time: 103.86492347717285 1.051418395933513
# strain 1.6710600852966309
# this is whating from the boyfrigpters of mind. It’s Calday Said this oil N
# this is whati
# Some Inthiamingly Up winder Center Is Total, who zave Lee Ley
# 12700 time: 104.1569652557373 1.0526777770648592
# strain 1.3009895086288452
# this is whather one of the dispatch just requests for right. It also loses t
# 12800 time: 104.19663453102112 1.052689873383442
# strain 1.530670404434204
# this is whathesis, as a shellissfringed, occurs with a higher environment, m
# 12900 time: 104.17223763465881 1.052699970502316
# strain 1.3385021686553955
# this is whative tour. “I have here around the peace picking up. Sketch. F


# d_model64, expand=2, n_heads=8, n_groups=1, d_state=8
# this is what Aolthrest toit trow Wex. Bargreat Ghive vearred ("But colled to
# 700 time: 15.093467950820923 0.14656143011618272
# strain 1.8304060697555542
# this is whate stdatfor, gasted. Narts disauser atcordivicters meopestes Duc
# this is whatop. The proselfense than-he’s consepportants better whiched f
# 6400 time: 14.46086049079895 0.15278742198291523
# strain 1.930558681488037
# this is whate of you’t reen’th his at at the bost effect" in going disca


# d_model64, expand=2, n_heads=8, n_groups=1, d_state=64 85872p
# this is whathounce in Goment Kimb Malrangeeds fe) acted tecloods his about f
# 700 time: 17.682673692703247 0.1755162530210661
# strain 1.9856963157653809
# this is whating Evanory doble even seextres sculgy Necres. Each Weatchile o
# this is whature li late becaused kate: Hode washing ’ ducinisating for the
# 6400 time: 17.91191840171814 0.18049865636094028
# strain 1.9032965898513794
# this is what her Ferruminationed Horstaring In’t Beacthrough the lot dives

# this is what a only to two iringly provides to enates and has man informatic
# 6400 time: 2.138993501663208 0.023055557905929273
# strain 1.705070972442627
# this is whats besically governmentalone then a riting on pitch against has a
# this is whate pulitary fquality and approtien and judging begged filed at cu
# 11400 time: 2.413839817047119 0.02302427980295661
# strain 2.1769118309020996
# this is whative. Medies realists of brapk, more block object that hut that



# d_model64, expand=2, n_heads=8, n_groups=8, d_state=8 85872p
# this is whation. Fodicuractiong ligher higheseberept’s Loves,Femmip or are
# 700 time: 16.98404598236084 0.16986171767306907
# strain 2.0250403881073
# this is what buy ovethina’s not and Os, surves). We pronen: CI seam for fo
# this is whating time a yyman stundanchy intelligende: “Buck: he, they ogr
# 6400 time: 17.89343810081482 0.17402444938405792
# strain 2.2177774906158447
# this is whation, informer", fhick during cources, 581 benefirention that sin
# this is what poizeers colode. You cliping. '( Thouks 5. Saladmy, as any rec
# 133700 time: 2.0944693088531494 0.023651174364440832
# strain 1.7085469961166382
# this is what I’m your know (IB2, it is including leaners, although office
# 133800 time: 2.1193015575408936 0.02365037872852213
# strain 1.8316044807434082
# this is what drain the cutool us was cloud in dock Tauk--aQport to fibe othe
# 133900 time: 2.1037728786468506 0.023649454599352675
# strain 1.5083467960357666
# this is what with the aware nashas made anTools-addropped or the covering pr


# act(xbc+conv(xbc))
# this is whats doliflalows Q nulalin Thare fompale hax re'p to 800ar corss.0
# 700 time: 18.463230848312378 0.18301630190197649
# strain 2.0806822776794434
# this is what nit's as.; fomup Sare mised mally nears port of the pot to he
# this is what in chis wurgest about Olrow, you wron ilanit it my for apcaid o
# 2000 time: 17.46792197227478 0.18287333448430051
# strain 2.216662645339966
# this is what hetwar, and cyaker, “-1V� N!/ Rovego Pach to cickrep. — 2-t




# ngroups1/8/64 dstate 64/8/1
# _no_weight_decay conv
# _no_weight_decay dt A D
# A init unif1,16
# expand 1/2
# pre/post norm
# rms/layer norm



In [ ]:
# @title wandb
!pip install -q wandb
import wandb # https://docs.wandb.ai/quickstart
wandb.login(key='487a2109e55dce4e13fc70681781de9f50f27be7')
try: run.finish()
except NameError: pass
run = wandb.init(project="mamba", config={"model": "mamba",}) #

In [ ]:
# @title train generate
import torch
from torch.nn import functional as F
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# scaler = torch.GradScaler()

# https://www.comet.com/site/blog/perplexity-for-llm-evaluation/
def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
    log_probs = F.log_softmax(logits, dim=-1)
    nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
    perplexity = nll.mean().exp()
    return perplexity

import time
def strain(model, dataloader, optim, scheduler=None): # train function with automatic mixed precision
    start = begin = time.time()
    model.train()
    for i, x in enumerate(dataloader):
        x, y = x[:,:-1].to(device), x[:,1:].to(device)
        # with torch.autocast(device_type=device, dtype=torch.bfloat16):
        logits, _ = model(x) #output = [batch size, trg len - 1, output dim]
        loss = F.cross_entropy(logits.flatten(0,1), y.flatten().to(int)) # [b*t,d], [b*t]
        optim.zero_grad()
        loss.backward()
        optim.step()
        # scaler.scale(loss).backward()
        # # scaler.unscale_(optimizer)
        # # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        # scaler.step(optimizer)
        # scaler.update()
        if scheduler is not None: scheduler.step()
        if i % 100 == 0:
            print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
            print("strain",loss.item())
            print(generate(model, "this is what"))
            model.train()
            # perplexity = Perplexity(logits.detach(), y).item()
            start = time.time()
        try: wandb.log({"train loss": loss.item()})
        except NameError: pass

def generate(model, context, max_steps=64, temperature=1):
    x = encode(context)#.to(device)
    model.eval()
    hid=None
    for n in range(max_steps):
        with torch.no_grad(): out, hid = model(x if hid==None else x[:,:-1], hid)
        # with torch.no_grad(): out, hid = model(x, hid)
        # print('generate', out.shape)
        out = out[:,-1] # get logit for last character
        out = F.softmax(out/temperature, dim=-1) # vocab_size to char
        ix = torch.multinomial(out, num_samples=1) # rand sample by output distribution
        x = torch.cat([x, ix], dim=1)
    completion = decode(x.squeeze(0))
    return completion

# scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=400, num_training_steps=40000) # https://docs.pytorch.org/torchtune/0.2/generated/torchtune.modules.get_cosine_schedule_with_warmup.html
# import time
# start = begin = time.time()
for i in range(1):
    # train_loss = strain(model, train_loader, optim, scheduler=None)
    strain(model, train_loader, optim, scheduler=None)
    # strain(model, train_loader, optim, scheduler=scheduler)
    # print(generate(model, "this is what"))
    # print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    # start = time.time()


In [ ]:
print(generate(model, "this is what"))

### hydra

In [ ]:
# @title hydra me
# https://github.com/goombalab/hydra/blob/main/hydra/modules/hydra.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Hydra(nn.Module):
    # def __init__(self, d_model, expand=2, n_heads=8, n_groups=1, d_state=64, d_conv=4):
    def __init__(self, d_model, expand=2, n_heads=8, n_groups=8, d_state=8, d_conv=7):
        super().__init__()
        self.d_model, self.n_groups, self.d_state, self.d_conv = d_model, n_groups, d_state, d_conv
        self.d_inner = expand * self.d_model
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.d_model, 2* self.d_inner + 2* (2*self.n_groups*self.d_state) + 2* self.n_heads, bias=False) # z,x,B,C,dt
        conv_dim = self.d_inner + 2* (2*self.n_groups*self.d_state) # for x,B,C
        # print(d_conv, conv_dim)
        self.conv1d = nn.Conv1d(conv_dim, conv_dim, bias=True, kernel_size=d_conv, groups=conv_dim, padding=d_conv//2)
        # self.conv1d.weight._no_weight_decay = True
        self.act = nn.SiLU()

        # self.h0 = nn.Parameter(torch.zeros(self.n_heads, self.d_head, self.d_state))
        # self.h0._no_weight_decay = True
        dt_min, dt_max = .001, .1
        dt = torch.exp(torch.rand(self.n_heads) * (math.log(dt_max)-math.log(dt_min)) + math.log(dt_min)).clamp(min=1e-4)
        self.dt_bias = nn.Parameter(dt + torch.log(-torch.expm1(-dt))) # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        self.dt_bias._no_weight_decay = True
        # A = torch.empty(self.n_heads, dtype=torch.float32).uniform_(1,16)
        A = torch.ones(self.n_heads, dtype=torch.float32)
        A_log = torch.log(A)
        self.A_log = nn.Parameter(A_log)
        self.A_log._no_weight_decay = True
        self.D = nn.Parameter(torch.ones(self.n_heads))
        self.D._no_weight_decay = True
        self.fc_D = nn.Linear(self.d_inner, self.n_heads, bias=False)
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    def forward(self, u): # [b,t,d]
        b = u.shape[0]
        # print('Hydra u', u.shape)
        A = -torch.exp(self.A_log) # [n_heads]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* (2*self.n_groups*self.d_state), 2* self.n_heads], dim=-1)

        dt = torch.cat((dt[:, :, :self.n_heads], torch.flip(dt[:, :, self.n_heads:], (1,))), dim=0)
        dt = F.softplus(dt+self.dt_bias) # [b,t,h]

        # xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1),:])  # [b,t, d_inner + 2* n_groups*d_state]
        xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1))  # [b,t, d_inner + 2* n_groups*d_state]
        # x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        x, BC = xBC.split([self.d_inner, 2* (2*self.n_groups*self.d_state)], dim=-1)
        x_og = x
        x = torch.cat((x, torch.flip(x,(1,))), dim=0)
        BC = torch.cat((BC[:,:,:2*self.n_groups*self.d_state], torch.flip(BC[:,:,2*self.n_groups*self.d_state:], (1,))), dim=0)
        B, C = BC.split([self.n_groups * self.d_state, self.n_groups * self.d_state], dim=-1)
        x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))


        # h0 = self.h0.expand(u.size(0),-1,-1,-1) # [b,n,d,s]
        # print('x dt a b', x.shape, dt.shape, A.shape, B.shape)

        # y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h0) # 256
        y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64) # 256

        y = y + x * self.D.unsqueeze(-1)
        # y = self.norm(y.flatten(2) * self.act(z)) # [b,t,d_inner] # norm(x)*silu(z) if norm_before_gate, else norm(x*silu(z)) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/layernorm_gated.py#L18

        y = y.flatten(2) # [b,l,inner]
        y = torch.roll(y, shifts=1, dims=1) # 123...l -> l12...l-1
        y[:,0,:] = 0. # 012...l-1
        y_fw, y_bw = y[:b], torch.flip(y[b:], (1,))
        y = y_fw + y_bw + x_og * F.linear(x_og, self.fc_D.weight, bias=self.D).repeat(1,1,self.d_head)


        y = self.norm(y.flatten(2)) * self.act(z)
        out = self.out_proj(y)
        return out # [b,t,in]



b,t,d_model=5,256,32
# b,t,d_model=5,7,32
d_model = 64
u = torch.randn(b,t,d_model, device=device)
model = Hydra(d_model).to(device)
out, h = model(u)
# h0 = torch.randn(b, model.n_heads, model.d_head, model.d_state)
# print(out.shape)
# print(out.shape, h.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])
# out, h = model(u, h)

# u = torch.randn(b,7,d_model, device=device)
# # out, h = model(u[:,-1:], h)
# out, h = model.step(u, h)
# out, h = model.step(u)
print(out.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])



In [ ]:
# @title goombalab hydra.py
# https://github.com/goombalab/hydra/blob/main/hydra/modules/hydra.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange, repeat
try:
    from mamba_ssm.ops.triton.layernorm_gated import RMSNorm as RMSNormGated
except ImportError:
    RMSNormGated = None
from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined
from hydra.modules.ops import hydra_split_conv1d_scan_combined


class Hydra(nn.Module):
    def __init__(
        self,
        d_model,
        d_state=64,
        d_conv=7,
        conv_init=None,
        expand=2,
        headdim=64,
        ngroups=1,
        dt_min=0.001,
        dt_max=0.1,
        dt_init_floor=1e-4,
        dt_limit=(0.0, float("inf")),
        learnable_init_states=False,
        activation="swish",
        bias=False,
        conv_bias=True,
        # Fused kernel and sharding options
        chunk_size=256,
        layer_idx=None,  # Absorb kwarg for general module
        device=None,
        dtype=None,
    ):
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.conv_init = conv_init
        self.expand = expand
        self.d_inner = self.expand * self.d_model
        self.headdim = headdim
        self.ngroups = ngroups
        assert self.d_inner % self.headdim == 0
        self.nheads = self.d_inner // self.headdim
        self.dt_limit = dt_limit
        self.learnable_init_states = learnable_init_states
        self.activation = activation
        self.chunk_size = chunk_size
        self.layer_idx = layer_idx

        # Order: [z, x, B, C, dt]
        d_in_proj = 2 * self.d_inner + 2 * (2 * self.ngroups * self.d_state) + 2 * self.nheads
        self.in_proj = nn.Linear(self.d_model, d_in_proj, bias=bias, **factory_kwargs)

        conv_dim = self.d_inner + 2 * (2 * self.ngroups * self.d_state)
        self.conv1d = nn.Conv1d(
            in_channels=conv_dim,
            out_channels=conv_dim,
            bias=conv_bias,
            kernel_size=d_conv,
            groups=conv_dim,
            padding=d_conv // 2,
            **factory_kwargs,
        )
        if self.conv_init is not None:
            nn.init.uniform_(self.conv1d.weight, -self.conv_init, self.conv_init)
        # self.conv1d.weight._no_weight_decay = True

        if self.learnable_init_states:
            self.init_states = nn.Parameter(torch.zeros(self.nheads, self.headdim, self.d_state, **factory_kwargs))
            self.init_states._no_weight_decay = True

        self.act = nn.SiLU()

        # Initialize log dt bias
        dt = torch.exp(torch.rand(self.nheads, **factory_kwargs) * (math.log(dt_max) - math.log(dt_min)) + math.log(dt_min))
        dt = torch.clamp(dt, min=dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + torch.log(-torch.expm1(-dt))
        self.dt_bias = nn.Parameter(inv_dt)
        # Just to be explicit. Without this we already don't put wd on dt_bias because of the check
        # name.endswith("bias") in param_grouping.py
        self.dt_bias._no_weight_decay = True

        # A parameter
        A = torch.ones(self.nheads, dtype=torch.float32, device=device)
        A_log = torch.log(A).to(dtype=dtype)
        self.A_log = nn.Parameter(A_log)
        # self.register_buffer("A_log", torch.zeros(self.nheads, dtype=torch.float32, device=device), persistent=True)
        self.A_log._no_weight_decay = True

        # D "skip" parameter
        self.D = nn.Parameter(torch.ones(self.nheads, device=device))
        self.D._no_weight_decay = True
        self.fc_D = nn.Linear(self.d_inner, self.nheads, bias=False, **factory_kwargs)

        # Extra normalization layer right before output projection
        assert RMSNormGated is not None
        self.norm = RMSNormGated(self.d_inner, eps=1e-5, norm_before_gate=True, **factory_kwargs)

        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias, **factory_kwargs)

    def forward(self, u, seq_idx=None): # [b,l,d]
        batch, seqlen, dim = u.shape
        zxbcdt = self.in_proj(u)  # (B, L, d_in_proj)
        A = -torch.exp(self.A_log.float())  # (nheads) or (d_inner, d_state)
        initial_states = repeat(self.init_states, "... -> b ...", b=2*batch) if self.learnable_init_states else None
        dt_limit_kwargs = {} if self.dt_limit == (0.0, float("inf")) else dict(dt_limit=self.dt_limit)
        z, xBC, dt = torch.split(zxbcdt, [self.d_inner, self.d_inner + 2 * (2 * self.ngroups * self.d_state), 2 * self.nheads], dim=-1)

        dt = torch.cat((dt[:, :, :self.nheads], torch.flip(dt[:, :, self.nheads:], (1,))), dim=0)
        dt = F.softplus(dt + self.dt_bias)  # (2 * B, L, nheads)
        assert self.activation in ["silu", "swish"]

        # 1D Convolution
        xBC = self.act(
            self.conv1d(xBC.transpose(1, 2)).transpose(1, 2)
        )  # (B, L, self.d_inner + 2 * (2 * ngroups * d_state))

        # Split into 3 main branches: X, B, C
        # These correspond to V, K, Q respectively in the SSM/attention duality
        x, BC = torch.split(xBC, [self.d_inner, 2 * (2 * self.ngroups * self.d_state)], dim=-1)
        x_og = x
        x = torch.cat((x, torch.flip(x, (1,))), dim=0)
        BC = torch.cat(
            (BC[:, :, :2 * self.ngroups * self.d_state],
             torch.flip(BC[:, :, 2 * self.ngroups * self.d_state:], (1,))),
            dim=0
        )
        B, C = torch.split(BC, [self.ngroups * self.d_state, self.ngroups * self.d_state], dim=-1)

        y = mamba_chunk_scan_combined(
            rearrange(x, "b l (h p) -> b l h p", p=self.headdim),
            dt,
            A,
            rearrange(B, "b l (g n) -> b l g n", g=self.ngroups),
            rearrange(C, "b l (g n) -> b l g n", g=self.ngroups),
            chunk_size=self.chunk_size,
            D=None,
            z=None,
            seq_idx=seq_idx,
            initial_states=initial_states,
            **dt_limit_kwargs,
        )
        y = rearrange(y, "b l h p -> b l (h p)")
        y = torch.roll(y, shifts=1, dims=1)
        y[:, 0, :] = 0.0
        y_fw, y_bw = y[:batch], torch.flip(y[batch:], (1,))
        y = y_fw + y_bw + x_og * repeat(
            F.linear(x_og, self.fc_D.weight, bias=self.D), "b l h -> b l (h p)", p=self.headdim
        )

        # Multiply "gate" branch and apply extra normalization layer
        y = self.norm(y, z)
        out = self.out_proj(y)
        return out


## mamba implementations

In [ ]:
# @title ssd
# https://goombalab.github.io/blog/2024/mamba2-part4-systems/
# https://tridao.me/blog/2024/mamba2-part3-algorithm/
# # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/ssd_combined.py
# y = mamba_chunk_scan_combined(x.unflatten(-1, (self.n_heads, self.d_head)), dt, A, # chunk_size=256,
#     B.unflatten(-1, (self.ngroups, self.d_state)), C.unflatten(-1, (self.ngroups, self.d_state)),
#     D=self.D, z=None).flatten(2) # norm before flatten? so norm is within group

def segsum(x):
    """Naive segment sum calculation. exp(segsum(A)) produces a 1-SS matrix,
       which is equivalent to a scalar SSM."""
    T = x.size(-1)
    x_cumsum = torch.cumsum(x, dim=-1)
    x_segsum = x_cumsum[...,:,None] - x_cumsum[...,None,:]
    mask = torch.tril(torch.ones(T, T, device=x.device, dtype=bool), diagonal=0)
    x_segsum = x_segsum.masked_fill(~mask, -torch.inf)
    # mask = torch.triu(torch.ones(T, T, device=x.device, dtype=bool))
    # x_segsum = x_segsum.masked_fill(mask, -torch.inf)
    return x_segsum

def ssd(X, A, B, C, block_len=64, h0=None): # X:[b,t,h,d], A:[b,t,h], B:[b,t,g,s], C:[b,t,g,s]
    assert X.dtype == A.dtype == B.dtype == C.dtype
    assert X.shape[1] % block_len == 0

    X, A, B, C = [x.unflatten(1, (-1,block_len)) for x in (X, A, B, C)]
    A = A.permute(0,3,1,2) # [b,t/block_len,block_len,h] -> [b,h,t/block_len,block_len]
    A_cumsum = torch.cumsum(A, dim=-1)
    # 1. Compute the output for each intra-chunk (diagonal blocks)
    L = torch.exp(segsum(A))
    Y_diag  = torch.einsum("bclhn,bcshn,bhcls,bcshp->bclhp", C, B, L, X)

    # 2. Compute the state for each intra-chunk
    # (right term of low-rank factorization of off-diagonal blocks; B terms)
    decay_states = torch.exp((A_cumsum[:,:,:,-1:] - A_cumsum))
    states = torch.einsum("bclhn,bhcl,bclhp->bchpn", B, decay_states, X)

    # 3. Compute the inter-chunk SSM recurrence; produces correct SSM states at chunk boundaries
    # (middle term of factorization of off-diag blocks; A terms)
    # if h0==None: h0 = torch.zeros_like(states[:,:1])
    if h0==None: h0 = torch.zeros(states.size(0),*states.shape[2:], device=device) # [b,h]
    # print('ssd', h0.shape, states.shape, [states.size(0),1,*states.shape[2:]])
    states = torch.cat([h0.unsqueeze(1), states], dim=1)
    decay_chunk = torch.exp(segsum(F.pad(A_cumsum[:,:,:,-1], (1,0))))
    new_states = torch.einsum("bhzc,bchpn->bzhpn", decay_chunk, states)

    # 4. Compute state -> output conversion per chunk
    # (left term of low-rank factorization of off-diagonal blocks; C terms)
    Y_off = torch.einsum('bclhn,bchpn,bhcl->bclhp', C, new_states[:,:-1], torch.exp(A_cumsum))

    # Add output of intra-chunk and inter-chunk terms (diagonal and off-diagonal blocks)
    Y = (Y_diag+Y_off).flatten(1,2)
    return Y, new_states[:,-1] # [b,t,n_heads,d_head],



In [ ]:
# @title mamba_ssm/modules/mamba2_simple.py next
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Mamba2(nn.Module):
    # def __init__(self, d_model, expand=2, n_heads=8, n_groups=1, d_state=64, d_conv=4):
    def __init__(self, d_model, expand=2, n_heads=8, n_groups=8, d_state=8, d_conv=4):
        super().__init__()
        n_groups = min(n_heads, n_groups)
        assert n_heads % n_groups == 0, "nheads must be divisible by ngroups"
        self.d_model, self.n_groups, self.d_state, self.d_conv = d_model, n_groups, d_state, d_conv
        self.d_inner = expand * self.d_model
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.d_model, 2* self.d_inner + 2* self.n_groups*self.d_state + self.n_heads, bias=False) # z,x,B,C,dt
        conv_dim = self.d_inner + 2* self.n_groups*self.d_state # for x,B,C
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, bias=True, kernel_size=d_conv, groups=conv_dim, padding=d_conv-1)
        # self.conv1d.weight._no_weight_decay = True
        self.act = nn.SiLU()

        # self.h0 = nn.Parameter(torch.zeros(self.n_heads, self.d_head, self.d_state))
        # self.h0._no_weight_decay = True
        dt_min, dt_max = .001, .1
        dt = torch.exp(torch.rand(self.n_heads) * (math.log(dt_max)-math.log(dt_min)) + math.log(dt_min)).clamp(min=1e-4)
        self.dt_bias = nn.Parameter(dt + torch.log(-torch.expm1(-dt))) # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        self.dt_bias._no_weight_decay = True
        A = torch.empty(self.n_heads, dtype=torch.float32).uniform_(1,16) # 1,16 ; .2,2
        self.A_log = nn.Parameter(torch.log(A))
        self.A_log._no_weight_decay = True
        self.D = nn.Parameter(torch.ones(self.n_heads))
        self.D._no_weight_decay = True
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    def forward(self, u, h0=None): # [b,t,d]
    # def forward(self, u): # [b,t,d]
        # if h0!=None: return self.step(u, h0)
        # print('Mamba2Simple u', u.shape)
        A = -torch.exp(self.A_log) # [n_heads]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_groups*self.d_state, self.n_heads], dim=-1)
        dt = F.softplus(dt+self.dt_bias) # [b,t,h]

        h_conv = F.pad(xBC, (0,0,self.d_conv-u.shape[1],0)) # [b,k,xbc]
        # xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1),:])  # [b,t, d_inner + 2* ngroups*d_state]
        # print(xBC[:,:-1].shape, self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1)]).shape)
        # xBC = xBC[:,:-1] + self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1)])  # [b,t, d_inner + 2* ngroups*d_state]
        x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))
        # h0 = self.h0.expand(u.size(0),-1,-1,-1) # [b,n,d,s]

        # dt = F.softplus(dt + dt_bias) # [b,h,d]
        # dA = torch.exp(dt.unsqueeze(-1) * A)  # [b,h,d,1]*[h,d,s]->[b,h,d,s]
        # # B = repeat(B, "b g n -> b (g h) n", h=nheads // ngroups)  # (batch, nheads, dstate)
        # B = B.repeat(1,nheads//ngroups,1) # [b,g,s]->[b,h,s]
        # print(self.n_heads//self.n_groups)
        h_g = max(1, self.n_heads//self.n_groups)
        B, C = B.repeat_interleave(h_g, dim=-2), C.repeat_interleave(h_g, dim=-2) # [b,g,s]->[b,h,s]
        # dB = dt.unsqueeze(-1) * B.unsqueeze(-2) # [b,h,d,1]*[b,h,1,s]->[b,h,d,s]
        # state.copy_(state * dA + dB * x.unsqueeze(-1)) # [b,h,d,s]
        # out = torch.einsum("bhdn,bhn->bhd", state.to(C.dtype), C) # [b,h,d]
        # out += (x * D).to(out.dtype)
        # out = out * F.silu(z).to(x.dtype)

        # print(h_g)
        # print('x dt a b', x.shape, dt.shape, A.shape, B.shape)

        # # y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h0) # 256
        y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64) # 256


        # dA = torch.exp(dt*A)[...,None,None] # [b,t,h,1,1]
        # dBx = torch.einsum("bth,btgs,bthd->bthds", dt, B, x)
        # # h_ssm = seq(dA, dBx, h0=None) # [b,t,d], [b,t,d], [b,d]
        # h_ssm = lcse(dA+0j, dBx+0j, h0=None).real # [b,t,d], [b,t,d], [b,d]
        # y = torch.einsum("bthds,btgs->bthd", h_ssm, C) # [b,h,d,s]*[b,h,s]


        y = y + x * self.D.unsqueeze(-1)
        # y = self.norm(y.flatten(2) * self.act(z)) # [b,t,d_inner] # norm(x)*silu(z) if norm_before_gate, else norm(x*silu(z)) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/layernorm_gated.py#L18
        y = self.norm(y.flatten(2)) * self.act(z)
        out = self.out_proj(y)
        return out, (h_conv, h_ssm) # [b,t,in], [b,h,d,s]
        # return out, (h_conv, h_ssm[:,-1]) # [b,t,in], [b,h,d,s]

    # def step(self, u, h=None): # [b,t,in], (conv[b,k,xbc], ssm[b,h,d,s])
    #     u = u[:,-self.d_conv:] # [b,min(k,t),in]
    #     A = -torch.exp(self.A_log)  # [n_heads]
    #     z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_groups*self.d_state, self.n_heads], dim=-1) # [b,min(k,t),_]
    #     dt = F.softplus(dt[:,-1]+self.dt_bias) # [b,n_heads]

    #     if h==None: h_conv = F.pad(xBC, (0,0,max(0,self.d_conv-u.shape[1]),0))
    #     else: h_conv = torch.cat([h[0], xBC], dim=1)[:,-self.d_conv:] # [b,k,xbc]
    #     # print(h_conv.shape, self.conv1d.weight.shape, self.conv1d.bias.shape)

    #     # xBC = F.silu(torch.einsum("bkc,cik->bc", h_conv, self.conv1d.weight) + self.conv1d.bias) # [b,kernel,xbc], [xbc,1,kernel], +[xbc] -> [b,xbc]
    #     # xBC = xBC + F.silu(torch.einsum("bkc,cik->bc", h_conv, self.conv1d.weight) + self.conv1d.bias) # [b,kernel,xbc], [xbc,1,kernel], +[xbc] -> [b,xbc]
    #     xBC = h_conv[:,-1]
    #     x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
    #     x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))
    #     h_g = max(1, self.n_heads//self.n_groups)
    #     B, C = B.repeat_interleave(h_g, dim=-2), C.repeat_interleave(h_g, dim=-2) # [b,g,s]->[b,h,s]

    #     # SSM step
    #     dA = torch.exp(dt*A) # [b,n_heads] # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/selective_state_update.py#L104
    #     # print('dt, B, x', dt.shape, B.shape, x.shape)
    #     dBx = torch.einsum("bh,bhs,bhd->bhds", dt, B, x)

    #     if h==None: h_ssm = dBx
    #     else: h_ssm = h[1] * dA[...,None,None] + dBx # [b,h,d,s]*[b,h,1,1]+[b,h,d,s]
    #     # print('h.ssm_state, C', h.ssm_state.shape, C.shape)
    #     y = torch.einsum("bhds,bhs->bhd", h_ssm, C) # [b,h,d,s]*[b,h,s]

    #     y = y + self.D.unsqueeze(-1) * x
    #     # print('step', y.shape,z.shape) # [5, 8, 16]) torch.Size([5, 4, 128]
    #     # y = self.norm(y.flatten(1) * F.silu(z[:,-1]))
    #     y = self.norm(y.flatten(1)) * F.silu(z[:,-1])
    #     y = self.out_proj(y)
    #     return y.unsqueeze(1), (h_conv, h_ssm)



# b,t,d_model=5,256,32
b,t,d_model=5,7,32
d_model = 64
u = torch.randn(b,t,d_model, device=device)
model = Mamba2(d_model).to(device)
out, h = model(u)
# h0 = torch.randn(b, model.n_heads, model.d_head, model.d_state)
# print(out.shape)
# print(out.shape, h.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])
# out, h = model(u, h)

# u = torch.randn(b,7,d_model, device=device)
# # out, h = model(u[:,-1:], h)
# out, h = model.step(u, h)
# # out, h = model.step(u)
# print(out.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])


# cpu seq ~40-41s
# this is what sherti, epow sour wocks I rmtre herring' Coer alaid it the adoo
# 700 time: 43.38966917991638 0.3799727667075251
# strain 2.045107126235962
# this is whate lilln the Tont to pleesyecise merefes asse sperced is a a cagk

# gpu ssd seqlen 2^11 29.5s

# lcsm



In [ ]:
# @title mamba_ssm/modules/mamba2_simple.py works
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Mamba2Simple(nn.Module):
    # def __init__(self, d_model, expand=2, n_heads=8, n_groups=1, d_state=64, d_conv=4):
    def __init__(self, d_model, expand=2, n_heads=8, n_groups=8, d_state=8, d_conv=4):
        super().__init__()
        self.d_model, self.n_groups, self.d_state, self.d_conv = d_model, n_groups, d_state, d_conv
        self.d_inner = expand * self.d_model
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.d_model, 2* self.d_inner + 2* self.n_groups*self.d_state + self.n_heads, bias=False) # z,x,B,C,dt
        conv_dim = self.d_inner + 2* self.n_groups*self.d_state # for x,B,C
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, bias=True, kernel_size=d_conv, groups=conv_dim, padding=d_conv-1)
        # self.conv1d.weight._no_weight_decay = True
        self.act = nn.SiLU()

        # self.h0 = nn.Parameter(torch.zeros(self.n_heads, self.d_head, self.d_state))
        # self.h0._no_weight_decay = True
        dt_min, dt_max = .001, .1
        dt = torch.exp(torch.rand(self.n_heads) * (math.log(dt_max)-math.log(dt_min)) + math.log(dt_min)).clamp(min=1e-4)
        self.dt_bias = nn.Parameter(dt + torch.log(-torch.expm1(-dt))) # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        self.dt_bias._no_weight_decay = True
        A = torch.empty(self.n_heads, dtype=torch.float32).uniform_(1,16)
        A_log = torch.log(A)
        self.A_log = nn.Parameter(A_log)
        self.A_log._no_weight_decay = True
        self.D = nn.Parameter(torch.ones(self.n_heads))
        self.D._no_weight_decay = True
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    # def forward(self, u, h0=None): # [b,t,d]
    def forward(self, u): # [b,t,d]
        # if h0!=None: return self.step(u, h0)
        # print('Mamba2Simple u', u.shape)
        A = -torch.exp(self.A_log) # [n_heads]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_groups*self.d_state, self.n_heads], dim=-1)
        dt = F.softplus(dt+self.dt_bias) # [b,t,h]

        h_conv = F.pad(xBC, (0,0,self.d_conv-u.shape[1],0)) # [b,k,xbc]
        xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1),:])  # [b,t, d_inner + 2* ngroups*d_state]
        x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))
        # h0 = self.h0.expand(u.size(0),-1,-1,-1) # [b,n,d,s]
        # print('x dt a b', x.shape, dt.shape, A.shape, B.shape)

        # y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h0) # 256
        y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64) # 256

        y = y + x * self.D.unsqueeze(-1)
        y = self.norm(y.flatten(2) * self.act(z)) # [b,t,d_inner] # norm(x)*silu(z) if norm_before_gate, else norm(x*silu(z)) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/layernorm_gated.py#L18
        out = self.out_proj(y)
        return out, (h_conv, h_ssm) # [b,t,in], [b,h,d,s]

    def step(self, u, h=None): # [b,t,in], (conv[b,k,xbc], ssm[b,h,d,s])
        u = u[:,-self.d_conv:] # [b,min(k,t),in]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_groups*self.d_state, self.n_heads], dim=-1) # [b,min(k,t),_]

        if h==None: h_conv = F.pad(xBC, (0,0,max(0,self.d_conv-u.shape[1]),0))
        else: h_conv = torch.cat([h[0], xBC], dim=1)[:,-self.d_conv:] # [b,k,xbc]
        # print(h_conv.shape, self.conv1d.weight.shape, self.conv1d.bias.shape)

        xBC = F.silu(torch.einsum("bkc,cik->bc", h_conv, self.conv1d.weight) + self.conv1d.bias) # [b,kernel,xbc], [xbc,1,kernel], +[xbc] -> [b,xbc]


        x, B, C = xBC.split([self.n_heads*self.d_head, self.n_groups*self.d_state, self.n_groups*self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,g,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        x, B, C = x.unflatten(-1, (self.n_heads, self.d_head)), B.unflatten(-1, (self.n_groups, self.d_state)), C.unflatten(-1, (self.n_groups, self.d_state))

        A = -torch.exp(self.A_log)  # [n_heads]

        # SSM step
        dt = F.softplus(dt[:,-1]+self.dt_bias) # [b,n_heads]
        dA = torch.exp(dt*A) # [b,n_heads] # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/selective_state_update.py#L104
        # print('dt, B, x', dt.shape, B.shape, x.shape)
        dBx = torch.einsum("bh,bhs,bhd->bhds", dt, B, x)
        if h==None: h_ssm = dBx
        else: h_ssm = h[1] * dA[...,None,None] + dBx # [b,h,d,s]*[b,h,1,1]+[b,h,d,s]
        # print('h.ssm_state, C', h.ssm_state.shape, C.shape)
        y = torch.einsum("bhds,bhs->bhd", h_ssm, C) # [b,h,d,s]*[b,h,s]
        y = y + self.D.unsqueeze(-1) * x
        y = self.norm(y.flatten(1) * F.silu(z[:,-1]))
        y = self.out_proj(y)
        return y.unsqueeze(1), (h_conv, h_ssm)



b,t,d_model=5,256,32
# b,t,d_model=5,7,32
d_model = 64
u = torch.randn(b,t,d_model, device=device)
model = Mamba2Simple(d_model).to(device)
out, h = model(u)
# h0 = torch.randn(b, model.n_heads, model.d_head, model.d_state)
# print(out.shape)
# print(out.shape, h.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])
# out, h = model(u, h)

# u = torch.randn(b,7,d_model, device=device)
# # out, h = model(u[:,-1:], h)
out, h = model.step(u, h)
# out, h = model.step(u)
print(out.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])



In [ ]:
# @title mamba_ssm/modules/mamba2_simple.py next
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Mamba2Simple(nn.Module):
    def __init__(self, in_dim, d_state=64, expand=1, n_heads=8, d_conv=4):
        super().__init__()
        self.in_dim, self.d_state, self.d_conv = in_dim, d_state, d_conv
        self.d_inner = expand * self.in_dim
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.in_dim, 2* self.d_inner + 2* self.n_heads*self.d_state + self.n_heads, bias=False) # z,x,B,C,dt
        conv_dim = self.d_inner + 2* self.n_heads*self.d_state # for x,B,C
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, bias=True, kernel_size=d_conv, groups=conv_dim, padding=d_conv-1)
        # self.conv1d.weight._no_weight_decay = True
        self.act = nn.SiLU()

        # self.h0 = nn.Parameter(torch.zeros(self.n_heads, self.d_head, self.d_state))
        # self.h0._no_weight_decay = True
        dt_min, dt_max = .001, .1
        dt = torch.exp(torch.rand(self.n_heads) * (math.log(dt_max)-math.log(dt_min)) + math.log(dt_min)).clamp(min=1e-4)
        self.dt_bias = nn.Parameter(dt + torch.log(-torch.expm1(-dt))) # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        self.dt_bias._no_weight_decay = True
        A = torch.empty(self.n_heads, dtype=torch.float32).uniform_(1,16)
        A_log = torch.log(A)
        self.A_log = nn.Parameter(A_log)
        self.A_log._no_weight_decay = True
        self.D = nn.Parameter(torch.ones(self.n_heads))
        self.D._no_weight_decay = True
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.in_dim, bias=False)

    # def forward(self, u, h0=None): # [b,t,d]
    def forward(self, u): # [b,t,d]
        # if h0!=None: return self.step(u, h0)
        # print('Mamba2Simple u', u.shape)
        A = -torch.exp(self.A_log) # [n_heads]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_heads*self.d_state, self.n_heads], dim=-1)
        dt = F.softplus(dt+self.dt_bias) # [b,t,h]

        h_conv = F.pad(xBC, (0,0,self.d_conv-u.shape[1],0)) # [b,k,xbc]
        xBC = self.act(self.conv1d(xBC.transpose(-2,-1)).transpose(-2,-1)[:,:u.size(1),:])  # [b,t, d_inner + 2* ngroups*d_state]
        x, B, C = xBC.unflatten(-1, (self.n_heads,-1)).split([self.d_head, self.d_state, self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,h,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        # h0 = self.h0.expand(u.size(0),-1,-1,-1) # [b,n,d,s]
        # print('x dt a b', x.shape, dt.shape, A.shape, B.shape)

        # y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h0) # 256
        y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64) # 256

        y = y + x * self.D.unsqueeze(-1)
        y = self.norm(y.flatten(2) * self.act(z)) # [b,t,d_inner] # norm(x)*silu(z) if norm_before_gate, else norm(x*silu(z)) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/layernorm_gated.py#L18
        out = self.out_proj(y)
        return out, (h_conv, h_ssm) # [b,t,in], [b,h,d,s]

    def step(self, u, h=None): # [b,t,in], (conv[b,k,xbc], ssm[b,h,d,s])
        u = u[:,-self.d_conv:] # [b,min(k,t),in]
        z, xBC, dt = self.in_proj(u).split([self.d_inner, self.d_inner + 2* self.n_heads*self.d_state, self.n_heads], dim=-1) # [b,min(k,t),_]

        if h==None: h_conv = F.pad(xBC, (0,0,max(0,self.d_conv-u.shape[1]),0))
        else: h_conv = torch.cat([h[0], xBC], dim=1)[:,-self.d_conv:] # [b,k,xbc]
        # print(h_conv.shape, self.conv1d.weight.shape, self.conv1d.bias.shape)

        xBC = F.silu(torch.einsum("bkc,cik->bc", h_conv, self.conv1d.weight) + self.conv1d.bias) # [b,kernel,xbc], [xbc,1,kernel], +[xbc] -> [b,xbc]
        x, B, C = xBC.unflatten(-1, (self.n_heads,-1)).split([self.d_head, self.d_state, self.d_state], dim=-1) # x:[b,t,h,d], B/C:[b,t,h,s] # X, B, C correspond to V, K, Q respectively in the SSM/attention duality
        A = -torch.exp(self.A_log)  # [n_heads]

        # SSM step
        dt = F.softplus(dt[:,-1]+self.dt_bias) # [b,n_heads]
        dA = torch.exp(dt*A) # [b,n_heads]
        # print('dt, B, x', dt.shape, B.shape, x.shape)
        dBx = torch.einsum("bh,bhs,bhd->bhds", dt, B, x)
        if h==None: h_ssm = dBx
        else: h_ssm = h[1] * dA[...,None,None] + dBx # [b,h,d,s]*[b,h,1,1]+[b,h,d,s]
        # print('h.ssm_state, C', h.ssm_state.shape, C.shape)
        y = torch.einsum("bhds,bhs->bhd", h_ssm, C) # [b,h,d,s]*[b,h,s]
        y = y + self.D.unsqueeze(-1) * x
        y = self.norm(y.flatten(1) * F.silu(z[:,-1]))
        y = self.out_proj(y)
        return y.unsqueeze(1), (h_conv, h_ssm)



b,t,in_dim=5,256,32
# b,t,in_dim=5,7,32
d_model = 64
u = torch.randn(b,t,in_dim, device=device)
model = Mamba2Simple(in_dim, d_model).to(device)
out, h = model(u)
# h0 = torch.randn(b, model.n_heads, model.d_head, model.d_state)
# print(out.shape)
# print(out.shape, h.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])
# out, h = model(u, h)

# u = torch.randn(b,7,in_dim, device=device)
# # out, h = model(u[:,-1:], h)
out, h = model.step(u, h)
# out, h = model.step(u)
print(out.shape)
# print(out[0,-3:,:5], h[0,:2,:5,:5])



In [ ]:
# @title from johnma2006/mamba-minimal
# https://github.com/johnma2006/mamba-minimal/blob/master/model.py
"""[1] Mamba: Linear-Time Sequence Modeling with Selective State Spaces (Albert Gu and Tri Dao) https://arxiv.org/abs/2312.00752
    [2] The Annotated S4 (Sasha Rush and Sidd Karamcheti) https://srush.github.io/annotated-s4
    n or d_state: latent state dim      (`N` in [1] Algorithm 2)
    d_in or d_inner: d * expansion factor         (`D` in [1] Algorithm 2) ; expansion factor (`E` in [1] Section 3.4)
    A, B, C, D: state space parameters  (See any state space representation formula)
                                        (B, C are input-dependent (aka selective, a key innovation in Mamba); A, D are not)
    Δ or delta: input-dependent step size
    dt_rank: rank of Δ                  (See [1] Section 3.6 "Parameterization of ∆")
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Mamba(nn.Module):
    def __init__(self, d_model=256, d_state=16, d_inner=512, n_layer=3):
        super().__init__()
        self.layers = nn.ModuleList([ResidualBlock(d_model, d_state, d_inner) for _ in range(n_layer)])
        self.norm_f = nn.RMSNorm(d_model)

    def forward(self, x): # (b, l, d)
        for layer in self.layers:
            x = layer(x)
        x = self.norm_f(x)
        return x

class ResidualBlock(nn.Module):
    def __init__(self, d_model, d_state, d_inner):
        super().__init__()
        self.mixer = MambaBlock(d_model, d_state, d_inner)
        self.norm = nn.RMSNorm(d_model)

    def forward(self, x): # (b, l, d) # Block.forward(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L297
        output = x + self.mixer(self.norm(x))
        return output # (b, l, d)


class MambaBlock(nn.Module):
    def __init__(self, d_model, d_state, d_inner, dt_rank=16):
        super().__init__()
        self.dt_rank = dt_rank
        self.in_proj = nn.Linear(d_model, d_inner * 2, bias=False)
        d_conv = 4
        self.conv1d = nn.Conv1d(d_inner, d_inner, d_conv, padding=d_conv-1, groups=d_inner)
        self.x_proj = nn.Linear(d_inner, dt_rank + 2*d_state, bias=False) # takes in `x` and outputs the input-specific Δ, B, C
        self.dt_proj = nn.Linear(dt_rank, d_inner, bias=True) # project Δ from dt_rank to d_in
        self.A_log = nn.Parameter(torch.log(torch.arange(1, d_state + 1).repeat(d_inner, 1)))
        self.D = nn.Parameter(torch.ones(d_inner))
        self.out_proj = nn.Linear(d_inner, d_model, bias=False)

    def forward(self, x): # (b, l, d) # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L119
        b, l, d = x.shape
        x_res = self.in_proj(x)  # (b, l, 2 * d_in)
        x, res = torch.chunk(x_res, 2, dim=-1) # [b, l, d_in]
        x = self.conv1d(x.permute(0, 2, 1))[:, :, :l].permute(0, 2, 1) # (b, l, d_in)
        y = self.ssm(F.silu(x)) * F.silu(res)
        output = self.out_proj(y)
        return output # (b, l, d)

    def ssm(self, x): # (b, l, d_in)
        """- Algorithm 2 in Section 3.2 in the Mamba paper [1] - run_SSM(A, B, C, u) in The Annotated S4 [2]
        Official Implementation: mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311"""
        d_in, n = self.A_log.shape
        A = -torch.exp(self.A_log.float())  # shape (d_in, n)
        # D = self.D.float()

        # 2: B : (B, L, N) ← sB (x)
        # 3: C : (B, L, N) ← sC (x)
        # 4: Δ : (B, L, D) ← 𝜏Δ(Parameter+𝑠Δ(x))
        # 5: A, B : (B, L, D, N) ← discretize(Δ, A, B)
        # 6: y ← SSM(A, B, C) (x)

        delta, B, C = self.x_proj(x).split([self.dt_rank, n, n], dim=-1) # [b,l,dt_rank] [b,l,n], [b,l,n]
        delta = F.softplus(self.dt_proj(delta)) # (b, l, d_in)
        y = selective_scan(x, delta, A, B, C, self.D)  # This is similar to run_SSM(A, B, C, u) in The Annotated S4 [2]
        return y # (b, l, d_in)

def selective_scan(u, delta, A, B, C, D): # u: (b, l, d_in); delta: (b, l, d_in); A: (d_in, n); B: (b, l, n); C: (b, l, n); D: (d_in)
    """- Section 2 State Space Models in the Mamba paper [1] - Algorithm 2 in Section 3.2 in the Mamba paper [1] - run_SSM(A, B, C, u) in The Annotated S4 [2]
    x(t+1) = Ax(t) + Bu(t)
    y(t)   = Cx(t) + Du(t)
    except B and C (and the step size delta, which is used for discretization) are dependent on the input x(t).

    Official Implementation: selective_scan_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L86
    Note: I refactored some parts out of `selective_scan_ref` out, so the functionality doesn't match exactly."""
    b, l, d_in = u.shape
    n = A.shape[1]

    # Discretize continuous parameters (A, B)
    deltaA = torch.exp(torch.einsum('bld,dn->bldn', delta, A)) # [b l d_in n] # - A is discretized using zero-order hold (ZOH) discretization (see Section 2 Equation 4 in the Mamba paper [1])
    deltaB_u = torch.einsum('bld,bln,bld->bldn', delta, B, u) # [b l d_in n] # - B is discretized using a simplified Euler discretization instead of ZOH. From a discussion with authors: "A is the more important term and the performance doesn't change much with the simplification on B"

    # selective scan (scan_SSM() in The Annotated S4 [2])
    # Note that the below is sequential, while the official implementation does a much faster parallel scan that is additionally hardware-aware (like FlashAttention).
    x = torch.zeros((b, d_in, n), device=device)
    ys = []
    for i in range(l):
        x = deltaA[:, i] * x + deltaB_u[:, i]
        y = torch.einsum('bdn,bn->bd', x, C[:, i, :]) # b d_in
        ys.append(y)
    y = torch.stack(ys, dim=1)  # shape (b, l, d_in)
    y = y + u * D
    return y # (b, l, d_in)


pred = Mamba()

batch=4
seq_len=500
d_model=256
x = torch.randn(batch, seq_len, d_model)
y = pred(x)
print(x.shape, y.shape)
# print(x)
# print(y)


In [ ]:
# @title tommyip/mamba2-minimal+
# https://github.com/tommyip/mamba2-minimal/blob/main/mamba2.py
import torch
from torch import nn
import torch.nn.functional as F
from einops import rearrange, repeat


from torch import Tensor
from typing import NamedTuple, cast
class InferenceCache(NamedTuple):
    conv_state: Tensor  # (batch, d_inner + 2* d_state, d_conv)
    ssm_state: Tensor  # (batch, nheads, d_head, d_state)
    @staticmethod
    def alloc(batch_size, d_inner, n_heads, d_head, d_state, d_conv, device=None):
        return InferenceCache(torch.zeros(batch_size, d_inner + 2* d_state, d_conv, device=device),
            torch.zeros(batch_size, n_heads, d_head, d_state, device=device))


class Mamba2(nn.Module):
    # def __init__(self, args: Mamba2Config, device = None):
    def __init__(self, d_model=256, d_state=128, n_heads=4, expand=2, d_conv=4, n_layer=2):
        super().__init__()
        self.d_model, self.d_state, self.d_conv = d_model, d_state, d_conv
        self.d_inner = expand * d_model
        self.n_heads, self.d_head = n_heads, self.d_inner//n_heads

        self.in_proj = nn.Linear(self.d_model, 2* self.d_inner + 2* self.d_state + self.n_heads, bias=False) # z, x, B, C, dt
        conv_dim = self.d_inner + 2* self.d_state
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, kernel_size=self.d_conv, groups=conv_dim, padding=self.d_conv-1)

        self.dt_bias = nn.Parameter(torch.empty(self.n_heads))
        self.A_log = nn.Parameter(torch.empty(self.n_heads))
        self.D = nn.Parameter(torch.empty(self.n_heads))
        self.norm = nn.RMSNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=False)

    def forward(self, u, h=None): # (batch, seqlen, d_model)
        # print('fwd', u.shape, h.shape if h!=None else None)
        # print('fwd', u.shape)
        if h: return self.step(u, h)
        A = -torch.exp(self.A_log)  # (n_heads,)
        zxbcdt = self.in_proj(u)  # (batch, seqlen, d_in_proj)
        z, xBC, dt = torch.split(zxbcdt, [self.d_inner, self.d_inner + 2* self.d_state, self.n_heads], dim=-1,)
        dt = F.softplus(dt + self.dt_bias)  # (batch, seqlen, n_heads)

        # Pad or truncate xBC seqlen to d_conv
        conv_state = F.pad(rearrange(xBC, "b l d -> b d l"), (self.d_conv - u.shape[1], 0))
        xBC = F.silu(self.conv1d(xBC.transpose(1, 2)).transpose(1, 2)[:, : u.shape[1], :])  # (batch, seqlen, d_inner + 2 * d_state))
        x, B, C = torch.split(xBC, [self.d_inner, self.d_state, self.d_state], dim=-1)

        x = rearrange(x, "b l (h p) -> b l h p", p=self.d_head)
        y, ssm_state = ssd(x * dt.unsqueeze(-1), A * dt,
            rearrange(B, "b l n -> b l 1 n"), rearrange(C, "b l n -> b l 1 n"), 8)

        y = y + x * self.D.unsqueeze(-1)
        y = rearrange(y, "b l h p -> b l (h p)")
        y = self.norm(y * F.silu(z))
        y = self.out_proj(y)
        h = InferenceCache(conv_state, ssm_state)
        return y, h # (batch, seqlen, d_model)

    def step(self, u, h): # (batch, 1, d_model)
        assert u.shape[1] == 1, "Only one token can be decoded per inference step"
        A = -torch.exp(self.A_log)  # [n_heads]
        zxbcdt = self.in_proj(u.squeeze(1))  # (batch, d_in_proj)
        z, xBC, dt = torch.split(zxbcdt, [self.d_inner, self.d_inner + 2* self.d_state, self.n_heads], dim=-1,)
        dt = F.softplus(dt+self.dt_bias)  # (batch, n_heads)

        # Advance convolution input
        h.conv_state.copy_(torch.roll(h.conv_state, shifts=-1, dims=-1))
        h.conv_state[:,:,-1] = xBC
        xBC = F.silu(torch.sum(h.conv_state * self.conv1d.weight.squeeze(1), dim=-1) + self.conv1d.bias)
        x, B, C = xBC.split([self.d_inner, self.d_state, self.d_state], dim=-1)

        # SSM step
        dA = torch.exp(dt*A)  # (batch, n_heads)
        x = x.unflatten(-1, (-1,self.d_head))
        dBx = torch.einsum("bh,bn,bhp->bhpn", dt, B, x)
        h.ssm_state.copy_(h.ssm_state * dA[...,None,None] + dBx)
        y = torch.einsum("bhpn,bn->bhp", h.ssm_state, C)

        y = y + self.D.unsqueeze(-1) * x
        y = self.norm(y.flatten(1) * F.silu(z))
        y = self.out_proj(y)
        return y.unsqueeze(1), h # (batch, 1, d_model)


# def segsum(x, device = None):
#     """Stable segment sum calculation.
#     `exp(segsum(A))` produces a 1-semiseparable matrix, which is equivalent to a scalar SSM.
#     Source: https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/ssd_minimal.py#L23-L32"""
#     T = x.size(-1)
#     x = repeat(x, "... d -> ... d e", e=T)
#     mask = torch.tril(torch.ones(T, T, dtype=torch.bool, device=device), diagonal=-1)
#     x = x.masked_fill(~mask, 0)
#     x_segsum = torch.cumsum(x, dim=-2)
#     mask = torch.tril(torch.ones(T, T, dtype=torch.bool, device=device), diagonal=0)
#     x_segsum = x_segsum.masked_fill(~mask, -torch.inf)
#     return x_segsum


# def ssd(x, A, B, C, chunk_size, initial_states=None, device = None):
#     """Structed State Space Duality (SSD) - the core of Mamba-2
#     This is almost the exact same minimal SSD code from the blog post.
#     Arguments
#         x: (batch, seqlen, n_heads, d_head)
#         A: (batch, seqlen, n_heads)
#         B: (batch, seqlen, n_heads, d_state)
#         C: (batch, seqlen, n_heads, d_state)
#     Return y: (batch, seqlen, n_heads, d_head)

#     Source
#      1. https://tridao.me/blog/2024/mamba2-part3-algorithm/
#      2. https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/ssd_minimal.py#L34-L78
#     """
#     assert x.shape[1] % chunk_size == 0

#     # Rearrange into chunks
#     # Step 1, 2 and 4 of SSD can be computed in parallel for each chunk across devices (sequence parallel)
#     # This is not implemented and left as an exercise for the reader 😜
#     x, A, B, C = [rearrange(m, "b (c l) ... -> b c l ...", l=chunk_size) for m in (x, A, B, C)]

#     A = rearrange(A, "b c l h -> b h c l")
#     A_cumsum = torch.cumsum(A, dim=-1)

#     # 1. Compute the output for each intra-chunk (diagonal blocks)
#     L = torch.exp(segsum(A, device=device))
#     Y_diag = torch.einsum("bclhn, bcshn, bhcls, bcshp -> bclhp", C, B, L, x)

#     # 2. Compute the state for each intra-chunk
#     # (right term of low-rank factorization of off-diagonal blocks; B terms)
#     decay_states = torch.exp(A_cumsum[:, :, :, -1:] - A_cumsum)
#     states = torch.einsum("bclhn, bhcl, bclhp -> bchpn", B, decay_states, x)

#     # 3. Compute the inter-chunk SSM recurrence; produces correct SSM states at chunk boundaries
#     # (middle term of factorization of off-diag blocks; A terms)
#     if initial_states is None:
#         initial_states = torch.zeros_like(states[:, :1])
#     states = torch.cat([initial_states, states], dim=1)
#     decay_chunk = torch.exp(segsum(F.pad(A_cumsum[:, :, :, -1], (1, 0)), device=device))
#     new_states = torch.einsum("bhzc, bchpn -> bzhpn", decay_chunk, states)
#     states, final_state = new_states[:, :-1], new_states[:, -1]

#     # 4. Compute state -> output conversion per chunk
#     # (left term of low-rank factorization of off-diagonal blocks; C terms)
#     state_decay_out = torch.exp(A_cumsum)
#     Y_off = torch.einsum("bclhn, bchpn, bhcl -> bclhp", C, states, state_decay_out)

#     # Add output of intra-chunk and inter-chunk terms (diagonal and off-diagonal blocks)
#     Y = rearrange(Y_diag + Y_off, "b c l h p -> b (c l) h p")
#     return Y, final_state


batch=4
seq_len=16
d_model=256

pred = Mamba2(d_model)

x = torch.randn(batch, seq_len, d_model)
y, h = pred(x)
# print(x)
# print(y)
print(x.shape)
print(y.shape)

x = torch.randn(batch, 1, d_model)
y, h = pred(x,h)

print(y.shape)
# print(y)



In [ ]:
# @title tommyip/mamba2-minimal
# https://github.com/tommyip/mamba2-minimal/blob/main/mamba2.py
import torch
from torch import nn
import torch.nn.functional as F
from einops import rearrange, repeat

from dataclasses import dataclass
@dataclass
class Mamba2Config:
    d_model =256 # model dimension (D)
    n_layer = 2  # number of Mamba-2 layers in the language model
    d_state = 256  # state dimension (N)
    d_conv = 4  # convolution kernel size
    headdim = 64  # head dimension (P)
    chunk_size = 8  # matrix partition size (Q)
    d_inner = 2 * d_model # expansion factor (E) * d_model
    nheads = d_inner // headdim

from torch import Tensor
from typing import NamedTuple, cast
class InferenceCache(NamedTuple):
    conv_state: Tensor  # (batch, d_inner + 2 * d_state, d_conv)
    ssm_state: Tensor  # (batch, nheads, headdim, d_state)

    @staticmethod
    def alloc(batch_size, args: Mamba2Config, device = None):
        return InferenceCache(torch.zeros(batch_size, args.d_inner + 2 * args.d_state, args.d_conv, device=device),
            torch.zeros(batch_size, args.nheads, args.headdim, args.d_state, device=device),)


class Mamba2LMHeadModel(nn.Module):
    def __init__(self, args: Mamba2Config, device = None):
        super().__init__()
        self.args = args
        self.device = device
        self.backbone = nn.ModuleDict(dict(
                layers=nn.ModuleList(
                    [nn.ModuleDict(dict(mixer=Mamba2(args, device=device), norm=nn.RMSNorm(args.d_model, device=device),))
                        for _ in range(args.n_layer)]),
                norm_f=nn.RMSNorm(args.d_model, device=device),
            )
        )

    def forward(self, x, h: list[InferenceCache] | list[None] | None = None): # (batch, seqlen, d_model)
        """h: hidden states for inference step. If present the constant-time
               (wrt sequence length) inference path will be taken, input_ids
               should have shape (batch, 1) containing the next batch of prompt token."""
        seqlen = x.shape[1]
        if h is None: h = [None for _ in range(self.args.n_layer)]
        for i, layer in enumerate(self.backbone.layers):
            y, h[i] = layer.mixer(layer.norm(x), h[i])
            x = y + x
        x = self.backbone.norm_f(x)
        return x[:, :seqlen], cast(list[InferenceCache], h) # (batch, seqlen, d_model)


class Mamba2(nn.Module):
    def __init__(self, args: Mamba2Config, device = None):
        super().__init__()
        self.args = args
        self.device = device
        # Order: (z, x, B, C, dt)
        d_in_proj = 2 * args.d_inner + 2 * args.d_state + args.nheads
        self.in_proj = nn.Linear(args.d_model, d_in_proj, bias=False, device=device)

        conv_dim = args.d_inner + 2 * args.d_state
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, kernel_size=args.d_conv,
            groups=conv_dim, padding=args.d_conv-1, device=device,)

        self.dt_bias = nn.Parameter(torch.empty(args.nheads, device=device))
        self.A_log = nn.Parameter(torch.empty(args.nheads, device=device))
        self.D = nn.Parameter(torch.empty(args.nheads, device=device))
        self.norm = nn.RMSNorm(args.d_inner, device=device)
        self.out_proj = nn.Linear(args.d_inner, args.d_model, bias=False, device=device)

    def forward(self, u, h=None): # (batch, seqlen, d_model)
        if h: return self.step(u, h)

        A = -torch.exp(self.A_log)  # (nheads,)
        zxbcdt = self.in_proj(u)  # (batch, seqlen, d_in_proj)
        z, xBC, dt = torch.split(zxbcdt, [self.args.d_inner, self.args.d_inner + 2 * self.args.d_state, self.args.nheads,], dim=-1,)
        dt = F.softplus(dt + self.dt_bias)  # (batch, seqlen, nheads)

        # Pad or truncate xBC seqlen to d_conv
        conv_state = F.pad(rearrange(xBC, "b l d -> b d l"), (self.args.d_conv - u.shape[1], 0))

        xBC = F.silu(self.conv1d(xBC.transpose(1, 2)).transpose(1, 2)[:, : u.shape[1], :])  # (batch, seqlen, d_inner + 2 * d_state))
        x, B, C = torch.split(xBC, [self.args.d_inner, self.args.d_state, self.args.d_state], dim=-1)
        x = rearrange(x, "b l (h p) -> b l h p", p=self.args.headdim)
        y, ssm_state = ssd(x * dt.unsqueeze(-1), A * dt,
            rearrange(B, "b l n -> b l 1 n"), rearrange(C, "b l n -> b l 1 n"),
            self.args.chunk_size, device=self.device,)
        y = y + x * self.D.unsqueeze(-1)
        y = rearrange(y, "b l h p -> b l (h p)")
        # y = self.norm(y, z)
        y = self.norm(y * F.silu(z))
        y = self.out_proj(y)
        h = InferenceCache(conv_state, ssm_state)
        return y, h # (batch, seqlen, d_model)

    def step(self, u, h): # (batch, 1, d_model)
        """Take a single inference step for the current input and hidden state

        Unlike attention-based models, RNN-based models (eg Mamba) does not need
        to look back at all the past tokens to generate a new token. Instead a
        hidden state (initialized to 0s initially) is updated for each input and
        passed to the next inference step. This means that the total inference
        time is linear with respect to the sequence length instead of quadratic
        in attention's case."""
        assert u.shape[1] == 1, "Only one token can be decoded per inference step"

        zxbcdt = self.in_proj(u.squeeze(1))  # (batch, d_in_proj)
        z, xBC, dt = torch.split(zxbcdt, [self.args.d_inner, self.args.d_inner + 2 * self.args.d_state, self.args.nheads,], dim=-1,)

        # Advance convolution input
        h.conv_state.copy_(torch.roll(h.conv_state, shifts=-1, dims=-1))
        h.conv_state[:, :, -1] = xBC
        # Convolution step
        xBC = torch.sum(h.conv_state * rearrange(self.conv1d.weight, "d 1 w -> d w"), dim=-1)
        xBC += self.conv1d.bias
        xBC = F.silu(xBC)

        x, B, C = torch.split(xBC, [self.args.d_inner, self.args.d_state, self.args.d_state], dim=-1)
        A = -torch.exp(self.A_log)  # (nheads,)

        # SSM step
        dt = F.softplus(dt + self.dt_bias)  # (batch, nheads)
        dA = torch.exp(dt * A)  # (batch, nheads)
        x = rearrange(x, "b (h p) -> b h p", p=self.args.headdim)
        dBx = torch.einsum("bh, bn, bhp -> bhpn", dt, B, x)
        h.ssm_state.copy_(h.ssm_state * rearrange(dA, "b h -> b h 1 1") + dBx)
        y = torch.einsum("bhpn, bn -> bhp", h.ssm_state, C)
        y = y + rearrange(self.D, "h -> h 1") * x
        y = rearrange(y, "b h p -> b (h p)")
        # y = self.norm(y, z)
        y = self.norm(y * F.silu(z))
        y = self.out_proj(y)
        return y.unsqueeze(1), h # (batch, 1, d_model)


def segsum(x, device = None):
    """Stable segment sum calculation.
    `exp(segsum(A))` produces a 1-semiseparable matrix, which is equivalent to a scalar SSM.
    Source: https://github.com/state-spaces/mamba/blob/219f03c840d5a44e7d42e4e728134834fddccf45/mamba_ssm/modules/ssd_minimal.py#L23-L32"""
    T = x.size(-1)
    x = repeat(x, "... d -> ... d e", e=T)
    mask = torch.tril(torch.ones(T, T, dtype=torch.bool, device=device), diagonal=-1)
    x = x.masked_fill(~mask, 0)
    x_segsum = torch.cumsum(x, dim=-2)
    mask = torch.tril(torch.ones(T, T, dtype=torch.bool, device=device), diagonal=0)
    x_segsum = x_segsum.masked_fill(~mask, -torch.inf)
    return x_segsum


def ssd(x, A, B, C, chunk_size, initial_states=None, device = None):
    """Structed State Space Duality (SSD) - the core of Mamba-2
    This is almost the exact same minimal SSD code from the blog post.
    Arguments
        x: (batch, seqlen, n_heads, d_head)
        A: (batch, seqlen, n_heads)
        B: (batch, seqlen, n_heads, d_state)
        C: (batch, seqlen, n_heads, d_state)
    Return y: (batch, seqlen, n_heads, d_head)

    Source
     1. https://tridao.me/blog/2024/mamba2-part3-algorithm/
     2. https://github.com/state-spaces/mamba/blob/219f03c840d5a44e7d42e4e728134834fddccf45/mamba_ssm/modules/ssd_minimal.py#L34-L78
    """
    assert x.shape[1] % chunk_size == 0

    # Rearrange into chunks
    # Step 1, 2 and 4 of SSD can be computed in parallel for each chunk across devices (sequence parallel)
    # This is not implemented and left as an exercise for the reader 😜
    x, A, B, C = [rearrange(m, "b (c l) ... -> b c l ...", l=chunk_size) for m in (x, A, B, C)]

    A = rearrange(A, "b c l h -> b h c l")
    A_cumsum = torch.cumsum(A, dim=-1)

    # 1. Compute the output for each intra-chunk (diagonal blocks)
    L = torch.exp(segsum(A, device=device))
    Y_diag = torch.einsum("bclhn, bcshn, bhcls, bcshp -> bclhp", C, B, L, x)

    # 2. Compute the state for each intra-chunk
    # (right term of low-rank factorization of off-diagonal blocks; B terms)
    decay_states = torch.exp(A_cumsum[:, :, :, -1:] - A_cumsum)
    states = torch.einsum("bclhn, bhcl, bclhp -> bchpn", B, decay_states, x)

    # 3. Compute the inter-chunk SSM recurrence; produces correct SSM states at chunk boundaries
    # (middle term of factorization of off-diag blocks; A terms)
    if initial_states is None:
        initial_states = torch.zeros_like(states[:, :1])
    states = torch.cat([initial_states, states], dim=1)
    decay_chunk = torch.exp(segsum(F.pad(A_cumsum[:, :, :, -1], (1, 0)), device=device))
    new_states = torch.einsum("bhzc, bchpn -> bzhpn", decay_chunk, states)
    states, final_state = new_states[:, :-1], new_states[:, -1]

    # 4. Compute state -> output conversion per chunk
    # (left term of low-rank factorization of off-diagonal blocks; C terms)
    state_decay_out = torch.exp(A_cumsum)
    Y_off = torch.einsum("bclhn, bchpn, bhcl -> bclhp", C, states, state_decay_out)

    # Add output of intra-chunk and inter-chunk terms (diagonal and off-diagonal blocks)
    Y = rearrange(Y_diag + Y_off, "b c l h p -> b (c l) h p")
    return Y, final_state


batch=4
seq_len=16
d_model=256

pred = Mamba2(Mamba2Config)
# pred = Mamba2LMHeadModel(Mamba2Config)

x = torch.randn(batch, seq_len, d_model)
y, h = pred(x)
print(x)
print(y)
print(x.shape)
print(y.shape)

x = torch.randn(batch, 1, d_model)
y, h = pred(x,h)

print(y.shape)
print(y)



In [ ]:
# @title alxndrTL mamba.py
# https://github.com/alxndrTL/mamba.py/blob/main/mambapy/mamba.py
import math
from dataclasses import dataclass
from typing import Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from mambapy.pscan import pscan

""" This file closely follows the mamba_simple.py from the official Mamba implementation, and the mamba-minimal by @johnma2006.
The major differences are :
-the convolution is done with torch.nn.Conv1d
-the selective scan is done in PyTorch

A sequential version of the selective scan is also available for comparison. Also, it is possible to use the official Mamba implementation.

This is the structure of the torch modules :
- A Mamba model is composed of several layers, which are ResidualBlock.
- A ResidualBlock is composed of a MambaBlock, a normalization, and a residual connection : ResidualBlock(x) = mamba(norm(x)) + x
- This leaves us with the MambaBlock : its input x is (B, L, D) and its outputs y is also (B, L, D) (B=batch size, L=seq len, D=model dim).
First, we expand x into (B, L, 2*ED) (where E is usually 2) and split it into x and z, each (B, L, ED).
Then, we apply the short 1d conv to x, followed by an activation function (silu), then the SSM.
We then multiply it by silu(z).
See Figure 3 of the paper (page 8) for a visual representation of a MambaBlock."""

@dataclass
class MambaConfig:
    d_model: int # D
    n_layers: int
    dt_rank: Union[int, str] = 'auto'
    d_state: int = 16 # N in paper/comments
    expand_factor: int = 2 # E in paper/comments
    d_conv: int = 4

    dt_min: float = 0.001
    dt_max: float = 0.1
    dt_init: str = "random" # "random" or "constant"
    dt_scale: float = 1.0
    dt_init_floor = 1e-4

    rms_norm_eps: float = 1e-5
    base_std: float = 0.02

    bias: bool = False
    conv_bias: bool = True
    inner_layernorms: bool = False # apply layernorms to internal activations

    mup: bool = False
    mup_base_width: float = 128 # width=d_model

    pscan: bool = True # use parallel scan mode or sequential mode when training
    use_cuda: bool = False # use official CUDA implementation when training (not compatible with (b)float16)

    def __post_init__(self):
        self.d_inner = self.expand_factor * self.d_model # E*D = ED in comments

        if self.dt_rank == 'auto':
            self.dt_rank = math.ceil(self.d_model / 16)

        # muP
        if self.mup:
            self.mup_width_mult = self.d_model / self.mup_base_width

class Mamba(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList([ResidualBlock(config) for _ in range(config.n_layers)])

    def forward(self, x): # x : (B, L, D)
        for layer in self.layers:
            x = layer(x)
        return x

    def step(self, x, caches):
        # x : (B, L, D)
        # caches : [cache(layer) for all layers], cache : (h, inputs)
        for i, layer in enumerate(self.layers):
            x, caches[i] = layer.step(x, caches[i])
        return x, caches

class ResidualBlock(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()
        self.mixer = MambaBlock(config)
        self.norm = RMSNorm(config.d_model, config.rms_norm_eps, config.mup)

    def forward(self, x): # x : (B, L, D)
        output = self.mixer(self.norm(x)) + x
        return output

    def step(self, x, cache):
        # x : (B, D)
        # cache : (h, inputs)
                # h : (B, ED, N)
                # inputs: (B, ED, d_conv-1)

        # output : (B, D)
        # cache : (h, inputs)
        output, cache = self.mixer.step(self.norm(x), cache)
        output = output + x
        return output, cache

class MambaBlock(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()
        self.config = config
        # projects block input from D to 2*ED (two branches)
        self.in_proj = nn.Linear(config.d_model, 2 * config.d_inner, bias=config.bias)

        self.conv1d = nn.Conv1d(in_channels=config.d_inner, out_channels=config.d_inner,
                              kernel_size=config.d_conv, bias=config.conv_bias,
                              groups=config.d_inner,
                              padding=config.d_conv - 1)

        # projects x to input-dependent delta, B, C
        self.x_proj = nn.Linear(config.d_inner, config.dt_rank + 2 * config.d_state, bias=False)

        # projects delta from dt_rank to d_inner
        self.dt_proj = nn.Linear(config.dt_rank, config.d_inner, bias=True)

        # dt initialization
        # dt weights
        dt_init_std = config.dt_rank**-0.5 * config.dt_scale
        if config.dt_init == "constant":
            nn.init.constant_(self.dt_proj.weight, dt_init_std)
        elif config.dt_init == "random":
            nn.init.uniform_(self.dt_proj.weight, -dt_init_std, dt_init_std)
        else:
            raise NotImplementedError

        # delta bias
        dt = torch.exp(
            torch.rand(config.d_inner) * (math.log(config.dt_max) - math.log(config.dt_min)) + math.log(config.dt_min)
        ).clamp(min=config.dt_init_floor)
        inv_dt = dt + torch.log(-torch.expm1(-dt)) # inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        with torch.no_grad():
            self.dt_proj.bias.copy_(inv_dt)
        #self.dt_proj.bias._no_reinit = True # initialization would set all Linear.bias to zero, need to mark this one as _no_reinit
        # todo : explain why removed

        # S4D real initialization
        A = torch.arange(1, config.d_state + 1, dtype=torch.float32).repeat(config.d_inner, 1)
        self.A_log = nn.Parameter(torch.log(A)) # why store A in log ? to keep A < 0 (cf -torch.exp(...)) ? for gradient stability ?
        self.A_log._no_weight_decay = True

        self.D = nn.Parameter(torch.ones(config.d_inner))
        self.D._no_weight_decay = True

        # projects block output from ED back to D
        self.out_proj = nn.Linear(config.d_inner, config.d_model, bias=config.bias)

        # used in jamba
        if self.config.inner_layernorms:
            self.dt_layernorm = RMSNorm(self.config.dt_rank, config.rms_norm_eps, config.mup)
            self.B_layernorm = RMSNorm(self.config.d_state, config.rms_norm_eps, config.mup)
            self.C_layernorm = RMSNorm(self.config.d_state, config.rms_norm_eps, config.mup)
        else:
            self.dt_layernorm = None
            self.B_layernorm = None
            self.C_layernorm = None

        if self.config.use_cuda:
            try:
                from mamba_ssm.ops.selective_scan_interface import selective_scan_fn
                self.selective_scan_cuda = selective_scan_fn
            except ImportError:
                print("Failed to import mamba_ssm. Falling back to mamba.py.")
                self.config.use_cuda = False

    def _apply_layernorms(self, dt, B, C):
        if self.dt_layernorm is not None:
            dt = self.dt_layernorm(dt)
        if self.B_layernorm is not None:
            B = self.B_layernorm(B)
        if self.C_layernorm is not None:
            C = self.C_layernorm(C)
        return dt, B, C

    def forward(self, x): # x : (B, L, D)
        _, L, _ = x.shape
        xz = self.in_proj(x) # (B, L, 2*ED)
        x, z = xz.chunk(2, dim=-1) # (B, L, ED), (B, L, ED)

        # x branch
        x = x.transpose(1, 2) # (B, ED, L)
        x = self.conv1d(x)[:, :, :L] # depthwise convolution over time, with a short filter
        x = x.transpose(1, 2) # (B, L, ED)

        x = F.silu(x)
        y = self.ssm(x, z)

        if self.config.use_cuda:
            output = self.out_proj(y) # (B, L, D)
            return output # the rest of the operations are done in the ssm function (fused with the CUDA pscan)

        # z branch
        z = F.silu(z)

        output = y * z
        output = self.out_proj(output) # (B, L, D)

        return output

    def ssm(self, x, z): # x : (B, L, ED)
        A = -torch.exp(self.A_log.float()) # (ED, N)
        D = self.D.float()

        deltaBC = self.x_proj(x) # (B, L, dt_rank+2*N)
        delta, B, C = torch.split(deltaBC, [self.config.dt_rank, self.config.d_state, self.config.d_state], dim=-1) # (B, L, dt_rank), (B, L, N), (B, L, N)
        delta, B, C = self._apply_layernorms(delta, B, C)
        delta = self.dt_proj.weight @ delta.transpose(1, 2) # (ED, dt_rank) @ (B, L, dt_rank) -> (B, ED, L)
        # here we just apply the matrix mul operation of delta = softplus(dt_proj(delta))
        # the rest will be applied later (fused if using cuda)

        # choose which selective_scan function to use, according to config
        if self.config.use_cuda:
            # these are unfortunately needed for the selective_scan_cuda function
            x = x.transpose(1, 2)
            B = B.transpose(1, 2)
            C = C.transpose(1, 2)
            z = z.transpose(1, 2)

            # "softplus" + "bias" + "y * silu(z)" operations are fused
            y = self.selective_scan_cuda(x, delta, A, B, C, D, z=z, delta_softplus=True, delta_bias=self.dt_proj.bias.float())
            y = y.transpose(1, 2) # (B, L, ED)
        else:
            delta = delta.transpose(1, 2)
            delta = F.softplus(delta + self.dt_proj.bias)
            if self.config.pscan:
                y = self.selective_scan(x, delta, A, B, C, D)
            else:
                y = self.selective_scan_seq(x, delta, A, B, C, D)
        return y

    def selective_scan(self, x, delta, A, B, C, D):
        # x : (B, L, ED)
        # Δ : (B, L, ED)
        # A : (ED, N)
        # B : (B, L, N)
        # C : (B, L, N)
        # D : (ED)
        # y : (B, L, ED)
        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, L, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(2) # (B, L, ED, N)
        BX = deltaB * (x.unsqueeze(-1)) # (B, L, ED, N)
        hs = pscan(deltaA, BX)
        y = (hs @ C.unsqueeze(-1)).squeeze(3) # (B, L, ED, N) @ (B, L, N, 1) -> (B, L, ED, 1)
        y = y + D * x
        return y

    def selective_scan_seq(self, x, delta, A, B, C, D):
        # x : (B, L, ED)
        # Δ : (B, L, ED)
        # A : (ED, N)
        # B : (B, L, N)
        # C : (B, L, N)
        # D : (ED)
        # y : (B, L, ED)
        _, L, _ = x.shape
        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, L, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(2) # (B, L, ED, N)
        BX = deltaB * (x.unsqueeze(-1)) # (B, L, ED, N)
        h = torch.zeros(x.size(0), self.config.d_inner, self.config.d_state, device=deltaA.device) # (B, ED, N)
        hs = []
        for t in range(0, L):
            h = deltaA[:, t] * h + BX[:, t]
            hs.append(h)
        hs = torch.stack(hs, dim=1) # (B, L, ED, N)
        y = (hs @ C.unsqueeze(-1)).squeeze(3) # (B, L, ED, N) @ (B, L, N, 1) -> (B, L, ED, 1)
        y = y + D * x
        return y

    # -------------------------- inference -------------------------- #
    """
    Concerning auto-regressive inference

    The cool part of using Mamba : inference is constant wrt to sequence length
    We just have to keep in cache, for each layer, two things :
    - the hidden state h (which is (B, ED, N)), as you typically would when doing inference with a RNN
    - the last d_conv-1 inputs of the layer, to be able to compute the 1D conv which is a convolution over the time dimension
      (d_conv is fixed so this doesn't incur a growing cache as we progress on generating the sequence)
      (and d_conv is usually very small, like 4, so we just have to "remember" the last 3 inputs)

    Concretely, these two quantities are put inside a cache tuple, and are named h and inputs respectively.
    h is (B, ED, N), and inputs is (B, ED, d_conv-1)
    The MambaBlock.step() receives this cache, and, along with outputing the output, alos outputs the updated cache for the next call.

    The cache object is initialized as follows : (None, torch.zeros()).
    When h is None, the selective scan function detects it and start with h=0.
    The torch.zeros() isn't a problem (it's same as just feeding the input, because the conv1d is padded)

    As we need one such cache variable per layer, we store a caches object, which is simply a list of cache object. (See mamba_lm.py)
    """

    def step(self, x, cache):
        # x : (B, D)
        # cache : (h, inputs)
                # h : (B, ED, N)
                # inputs : (B, ED, d_conv-1)
        # y : (B, D)
        # cache : (h, inputs)
        h, inputs = cache
        xz = self.in_proj(x) # (B, 2*ED)
        x, z = xz.chunk(2, dim=1) # (B, ED), (B, ED)

        # x branch
        x_cache = x.unsqueeze(2)
        x = self.conv1d(torch.cat([inputs, x_cache], dim=2))[:, :, self.config.d_conv-1] # (B, ED)

        x = F.silu(x)
        y, h = self.ssm_step(x, h)

        # z branch
        z = F.silu(z)

        output = y * z
        output = self.out_proj(output) # (B, D)

        # prepare cache for next call
        inputs = torch.cat([inputs[:, :, 1:], x_cache], dim=2) # (B, ED, d_conv-1)
        cache = (h, inputs)

        return output, cache

    def ssm_step(self, x, h): # x : (B, ED) # h : (B, ED, N)
        A = -torch.exp(self.A_log.float()) # (ED, N) # todo : ne pas le faire tout le temps, puisque c'est indépendant de la timestep
        D = self.D.float()
        deltaBC = self.x_proj(x) # (B, dt_rank+2*N)
        delta, B, C = torch.split(deltaBC, [self.config.dt_rank, self.config.d_state, self.config.d_state], dim=-1) # (B, dt_rank), (B, N), (B, N)
        delta, B, C = self._apply_layernorms(delta, B, C)
        delta = F.softplus(self.dt_proj(delta)) # (B, ED)

        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(1) # (B, ED, N)
        BX = deltaB * (x.unsqueeze(-1)) # (B, ED, N)
        if h is None:
            h = torch.zeros(x.size(0), self.config.d_inner, self.config.d_state, device=deltaA.device) # (B, ED, N)
        h = deltaA * h + BX # (B, ED, N)
        y = (h @ C.unsqueeze(-1)).squeeze(2) # (B, ED, N) @ (B, N, 1) -> (B, ED, 1)
        y = y + D * x
        return y, h

class RMSNorm(nn.Module):
    def __init__(self, d_model: int, eps: float = 1e-5, use_mup: bool = False):
        super().__init__()
        self.use_mup = use_mup
        self.eps = eps
        # https://arxiv.org/abs/2404.05728, RMSNorm gains prevents muTransfer (section 4.2.3)
        if not use_mup:
            self.weight = nn.Parameter(torch.ones(d_model))

    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        if not self.use_mup:
            return output * self.weight
        else:
            return output



In [ ]:
# @title alxndrTL mamba2.py
# https://github.com/alxndrTL/mamba.py/blob/main/mambapy/mamba2.py

"""adapted from https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
It implements a Mamba2 model BUT still relies on the official Triton code for Mamba2.
(Coming soon hopefully is a full torch version, like in mamba.py and pscan.py)

Also, the file implements a caching mecanism, and a config similar to what's being done in mamba.py, as well as supports muP.

When passing an input of length 1 in the forward function, the model automatically routes the call to the step function.
This step function does one "classic" RNN-like step of computation, using the input and the cache provided. It returns the output as well as the new cache.
This is used for inference, to generate tokens one by one.

Also, this model supports prefilling a prompt and decoding token by token : this is just a mix of forwarding (in parallel) the input, and then decoding step by step.
In order to do that, we need the forward/parallel call (also used in training) to output the cache, then used to start the step by step decoding part.
To use this mode, you need to call Mamba2.forward() with your input (of shape (B, L, D)) as well as with a non-None cache (like full zeros, it doesnt matter, just not "None").
The forward call will thus return the output and the cache. From there, you can start your step by step decoding (see just above).

The cache is composed of two objects :
-h_cache: the last hidden state. Just like an RNN : you have to keep track of only the last h.
-conv_state: because Mamba2 uses a convolution over the time sequence, with a filter of length d_conv=4, you have to keep the last d_conv-1=3 inputs of that convolution to be able to run it provided a new input.

h_cache is of shape (B, n_heads, d_head, N) and is initialized at 0 (ie no starting hidden state, which is the default behavior in Mamba).
conv_state is of shape (B, EDN * 2*n_groups*, d_conv) and is initialized at 0

(B=batch_size, L=seq len, E = expand_factor, D=d_model, N=d_state)
(TODO: make it full pytorch (ie translate mamba_chunk_scan_combined in pytorch))
"""

import math
from dataclasses import dataclass
from typing import Union

import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange, repeat

try:
    from causal_conv1d import causal_conv1d_fn, causal_conv1d_update
except ImportError:
    causal_conv1d_fn, causal_conv1d_update = None, None

try:
    from mamba_ssm.ops.triton.layernorm_gated import RMSNorm as RMSNormGated, LayerNorm
    from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined, mamba_split_conv1d_scan_combined
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update

except ImportError:
    RMSNormGated, LayerNorm = None, None
    mamba_chunk_scan_combined, mamba_split_conv1d_scan_combined = None, None
    selective_state_update = None

@dataclass
class Mamba2Config:
    d_model: int # D
    n_layers: int
    d_head: int # todo : plutot n_heads non ?
    d_state: int = 64 # N in paper/comments
    expand_factor: int = 2 # E in paper/comments
    d_conv: int = 4
    n_groups: int = 1# todo : ??

    A_init_range: tuple = (1, 16)
    dt_min: float = 0.001
    dt_max: float = 0.1
    dt_init_floor: float = 1e-4
    dt_limit: tuple = (0.0, float("inf"))
    conv_init = None

    learnable_init_states: bool = False
    activation: str = "swish" # "swish" or "silu"

    rms_norm_eps: float = 1e-5
    base_std: float = 0.02

    bias: bool = False
    conv_bias: bool = True

    mup: bool = False
    mup_base_width: float = 128 # width=d_model

    chunk_size: int = 256
    use_mem_eff_path: bool = True
    dtype=None
    device=None

    def __post_init__(self):
        self.d_inner = self.expand_factor * self.d_model # E*D = ED in comments
        self.n_heads = self.d_inner // self.d_head
        assert self.d_inner % self.d_head == 0
        assert (self.d_inner / self.d_head) % 8 == 0, "requierement of causal_conv1d"

        # # muP
        # if self.mup:
        #     self.mup_width_mult = self.d_model / self.mup_base_width

class Mamba2(nn.Module):
    def __init__(self, config: Mamba2Config):
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList([ResidualBlock(config) for _ in range(config.n_layers)])

    def forward(self, x, caches=None): # (B, L, D)
        if caches is None:
            caches = [None] * self.config.n_layers

        for i, layer in enumerate(self.layers):
            x, caches[i] = layer(x, caches[i])

        if caches[0] == None:
            return x
        else:
            return x, caches

class ResidualBlock(nn.Module):
    def __init__(self, config: Mamba2Config):
        super().__init__()
        self.config = config
        self.mixer = Mamba2Block(self.config)
        self.norm = nn.RMSNorm(self.config.d_model, self.config.rms_norm_eps, self.config.mup)

    def forward(self, x, cache=None): # (B, L, D)
        output, cache = self.mixer(self.norm(x), cache)
        output = output + x # (B, L, D)
        return output, cache

    def get_empty_cache(self, batch_size):
        h_cache = torch.zeros(batch_size, self.config.n_heads, self.config.d_head, self.config.d_state, device=self.mixer.in_proj.weight.device, dtype=self.mixer.in_proj.weight.dtype)
        conv_cache = torch.zeros(batch_size, self.mixer.conv1d.weight.shape[0], self.config.d_conv, device=self.mixer.conv1d.weight.device, dtype=self.mixer.conv1d.weight.dtype)
        return (h_cache, conv_cache)

class Mamba2Block(nn.Module):
    def __init__(self, config: Mamba2Config):
        super().__init__()
        factory_kwargs = {"device": config.device, "dtype": config.dtype}
        self.config = config
        # [z, x, B, C, dt]
        d_in_proj = 2 * self.config.d_inner + 2 * self.config.n_groups * self.config.d_state + self.config.n_heads
        self.in_proj = nn.Linear(self.config.d_model, d_in_proj, bias=self.config.bias, **factory_kwargs)

        conv_dim = self.config.d_inner + 2 * self.config.n_groups * self.config.d_state
        self.conv1d = nn.Conv1d(conv_dim, conv_dim, self.config.d_conv, padding=self.config.d_conv - 1,
            bias=self.config.conv_bias, groups=conv_dim, **factory_kwargs,)

        if self.config.conv_init is not None:
            nn.init.uniform_(self.conv1d.weight, -self.config.conv_init, self.config.conv_init)
        # self.conv1d.weight._no_weight_decay = True

        # todo : mup init + lr
        if self.config.learnable_init_states:
            self.init_states = nn.Parameter(torch.zeros(self.config.n_heads, self.config.d_head, self.config.d_state, **factory_kwargs))
            self.init_states._no_weight_decay = True

        self.act = nn.SiLU()

        # Initialize log dt bias
        dt = torch.exp(
            torch.rand(self.config.n_heads, **factory_kwargs) * (math.log(self.config.dt_max) - math.log(self.config.dt_min))
            + math.log(self.config.dt_min)
        )
        dt = torch.clamp(dt, min=self.config.dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + torch.log(-torch.expm1(-dt))
        self.dt_bias = nn.Parameter(inv_dt)
        # Just to be explicit. Without this we already don't put wd on dt_bias because of the check
        # name.endswith("bias") in param_grouping.py
        self.dt_bias._no_weight_decay = True

        # A parameter
        assert self.config.A_init_range[0] > 0 and self.config.A_init_range[1] >= self.config.A_init_range[0]
        A = torch.empty(self.config.n_heads, dtype=torch.float32, device=self.config.device).uniform_(*self.config.A_init_range)
        A_log = torch.log(A).to(dtype=self.config.dtype)
        self.A_log = nn.Parameter(A_log)
        # self.register_buffer("A_log", torch.zeros(self.nheads, dtype=torch.float32, device=device), persistent=True)
        self.A_log._no_weight_decay = True

        # D "skip" parameter
        self.D = nn.Parameter(torch.ones(self.config.n_heads, device=self.config.device))
        self.D._no_weight_decay = True

        # Extra normalization layer right before output projection
        self.norm = RMSNormGated(self.config.d_inner, eps=1e-5, norm_before_gate=False, **factory_kwargs)

        self.out_proj = nn.Linear(self.config.d_inner, self.config.d_model, bias=self.config.bias, **factory_kwargs)

    def forward(self, u, cache=None, seq_idx=None): # (B, L, D)
        batch, length, _ = u.shape
        return_cache = False
        if cache is not None and length > 1:
            cache = None
            return_cache = True

        if cache is not None:
            out, cache = self.step(u, cache)
            return out, cache

        zxbcdt = self.in_proj(u)  # (B, L, d_in_proj)
        A = -torch.exp(self.A_log)  # (nheads) or (d_inner, d_state)
        initial_states=repeat(self.init_states, "... -> b ...", b=batch) if self.config.learnable_init_states else None
        dt_limit_kwargs = {} if self.config.dt_limit == (0.0, float("inf")) else dict(dt_limit=self.config.dt_limit)

        if self.config.use_mem_eff_path:
            # Fully fused path
            out = mamba_split_conv1d_scan_combined(
                zxbcdt,
                rearrange(self.conv1d.weight, "d 1 w -> d w"),
                self.conv1d.bias,
                self.dt_bias,
                A,
                D=self.D,
                chunk_size=self.config.chunk_size,
                seq_idx=seq_idx,
                activation=self.config.activation,
                rmsnorm_weight=self.norm.weight,
                rmsnorm_eps=self.norm.eps,
                outproj_weight=self.out_proj.weight,
                outproj_bias=self.out_proj.bias,
                headdim=self.config.d_head,
                ngroups=self.config.n_groups,
                norm_before_gate=False,
                initial_states=initial_states,
                return_final_states=return_cache,
                **dt_limit_kwargs,
            )

            if return_cache:
                # get h_cache from output
                out, h_cache = out

                # compute conv_cache with last d_conv entries of xBC
                _, xBC, _ = torch.split(zxbcdt, [self.config.d_inner, self.config.d_inner + 2 * self.config.n_groups * self.config.d_state, self.config.n_heads], dim=-1)
                conv_cache = xBC[:, -self.config.d_conv:].transpose(1, 2) # (error if seqlen<d_conv)

                cache = (h_cache, conv_cache)

        else:
            z, xBC, dt = torch.split(
                zxbcdt, [self.config.d_inner, self.config.d_inner + 2 * self.config.n_groups * self.config.d_state, self.config.n_heads], dim=-1
            )
            dt = F.softplus(dt + self.dt_bias)  # (B, L, nheads)
            assert self.config.activation in ["silu", "swish"]

            # 1D Convolution
            if causal_conv1d_fn is None or self.config.activation not in ["silu", "swish"]:
                xBC = self.act(self.conv1d(xBC.transpose(1, 2)).transpose(1, 2)) # (B, L, self.d_inner + 2 * n_groups * d_state)
            else:
                xBC = causal_conv1d_fn(
                    x=xBC.transpose(1, 2),
                    weight=rearrange(self.conv1d.weight, "d 1 w -> d w"),
                    bias=self.conv1d.bias,
                    activation=self.config.activation,
                ).transpose(1, 2)

            # split into 3 main branches: X, B, C
            # These correspond to V, K, Q respectively in the SSM/attention duality
            x, B, C = torch.split(xBC, [self.config.d_inner, self.config.n_groups * self.config.d_state, self.config.n_groups * self.config.d_state], dim=-1)
            y = mamba_chunk_scan_combined(
                rearrange(x, "b l (h p) -> b l h p", p=self.config.d_head),
                dt,
                A,
                rearrange(B, "b l (g n) -> b l g n", g=self.config.n_groups),
                rearrange(C, "b l (g n) -> b l g n", g=self.config.n_groups),
                chunk_size=self.config.chunk_size,
                D=self.D,
                z=None,
                seq_idx=seq_idx,
                initial_states=initial_states,
                **dt_limit_kwargs,
            )
            y = rearrange(y, "b l h p -> b l (h p)")

            # Multiply "gate" branch and apply extra normalization layer
            y = self.norm(y, z)
            out = self.out_proj(y) # (B, L, D)
        return out, cache

    def step(self, u, cache): # (B, 1, D)
        h_cache, conv_cache = cache

        zxbcdt = self.in_proj(u.squeeze(1))  # (B, 2D)
        d_mlp = (zxbcdt.shape[-1] - 2 * self.config.d_inner - 2 * self.config.n_groups * self.config.d_state - self.config.n_heads) // 2
        z0, x0, z, xBC, dt = torch.split(zxbcdt, [d_mlp, d_mlp, self.config.d_inner, self.config.d_inner + 2 * self.config.n_groups * self.config.d_state, self.config.n_heads], dim=-1)

        # conv step
        if causal_conv1d_update is None:
            conv_cache.copy_(torch.roll(conv_cache, shifts=-1, dims=-1)) # update state (B, D, W)
            conv_cache[:, :, -1] = xBC
            xBC = torch.sum(conv_cache * rearrange(self.conv1d.weight, "d 1 w -> d w"), dim=-1) # (B, D)
            if self.conv1d.bias is not None:
                xBC = xBC + self.conv1d.bias
            xBC = self.act(xBC).to(dtype=x.dtype)
        else:
            xBC = causal_conv1d_update(xBC, conv_cache, rearrange(self.conv1d.weight, "d 1 w -> d w"), self.conv1d.bias, self.config.activation)
        x, B, C = torch.split(xBC, [self.config.d_inner, self.config.n_groups * self.config.d_state, self.config.n_groups * self.config.d_state], dim=-1)
        A = -torch.exp(self.A_log.float()) # (n_heads)

        # SSM step
        if selective_state_update is None:
            assert self.config.n_groups == 1, "Only support ngroups=1 for this inference code path"
            # discretize A
            dt = F.softplus(dt + self.dt_bias.to(dtype=dt.dtype))  # (B, n_heads)
            dA = torch.exp(dt * A)  # (B, n_heads)
            # discretize B
            x = rearrange(x, "b (h p) -> b h p", p=self.config.d_head)
            dBx = torch.einsum("bh,bn,bhp->bhpn", dt, B, x)
            # compute one step
            h_cache.copy_(h_cache * rearrange(dA, "b h -> b h 1 1") + dBx)
            # compute output
            y = torch.einsum("bhpn,bn->bhp", h_cache.to(x.dtype), C)
            y = y + rearrange(self.D.to(x.dtype), "h -> h 1") * x
            y = rearrange(y, "b h p -> b (h p)")

        else:
            A = repeat(A, "h -> h p n", p=self.config.d_head, n=self.config.d_state).to(dtype=torch.float32)
            dt = repeat(dt, "b h -> b h p", p=self.config.d_head)
            dt_bias = repeat(self.dt_bias, "h -> h p", p=self.config.d_head)
            D = repeat(self.D, "h -> h p", p=self.config.d_head)
            B = rearrange(B, "b (g n) -> b g n", g=self.config.n_groups)
            C = rearrange(C, "b (g n) -> b g n", g=self.config.n_groups)
            x_reshaped = rearrange(x, "b (h p) -> b h p", p=self.config.d_head)

            y = selective_state_update(h_cache, x_reshaped, dt, A, B, C, D, z=None, dt_bias=dt_bias, dt_softplus=True)
            y = rearrange(y, "b h p -> b (h p)")

        #if self.rmsnorm:
        y = self.norm(y, z)
        if d_mlp > 0:
            y = torch.cat([F.silu(z0) * x0, y], dim=-1)
        out = self.out_proj(y)
        return out.unsqueeze(1), (h_cache, conv_cache)



## state-spaces mamba-ssm

In [ ]:
# @title pip install mamba-ssm
# !pip install mamba-ssm[causal-conv1d]
# !pip install mamba-ssm
# !wget https://github.com/state-spaces/mamba/releases/download/v2.2.2/mamba_ssm-2.2.2+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
# !pip install mamba_ssm-2.2.2+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl

# !pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118
# !pip install causal_conv1d==1.1.1
# !pip install mamba-ssm==1.2.0.post1


!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.4.0/causal_conv1d-1.4.0+cu118torch2.0cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip install mamba-ssm


# !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
# !pip uninstall mamba-ssm causal-conv1d
# !pip install causal-conv1d && pip install mamba-ssm



# !pip install mamba-ssm[causal-conv1d]==1.0.1

# !pip install mamba-ssm causal-conv1d

# !pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.4.0/causal_conv1d-1.4.0+cu118torch2.0cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
# !pip install mamba-ssm

# !apt-get install -y cuda-11-8

# import os
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '') # Adds the CUDA library path to LD_LIBRARY_PATH


In [ ]:
# @title huggingface mamba2
# https://huggingface.co/docs/transformers/en/model_doc/mamba2
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/mamba2/modeling_mamba2.py#L825
import torch
# import torch.nn as nn
from transformers import Mamba2Config, Mamba2Model

in_dim=132
b,t,d = 4,5,64#256
try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50

# num_heads*head_dim=2*hidden_size ?
# *6

# hidden_size * expand (128) must equal num_heads * head_dim
config = Mamba2Config(hidden_size=d, num_heads=8, head_dim=16, state_size=64, num_hidden_layers=2, output_hidden_states=True, vocab_size=vocab_size)
model = Mamba2Model(config).to(device)
# model.embeddings = nn.Linear(in_dim, d)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3) # og betas=(0.9, 0.999), wd.01
# optim = torch.optim.AdamW(model.parameters(), 1e-3, betas=(0.9, 0.95), weight_decay=1e-1) # og wd.01


# x = torch.randn(batch, seq_len, in_dim)
x = torch.randint(vocab_size, (b,t), device=device)

# out = model(x, use_cache=True)
# # print(out)
# print(out.keys())
# y, cache = out.last_hidden_state, out.cache_params
# # print(y)
# print(y.shape) # [b,t,d]
# # print(cache)
# y = y @ model.embeddings.weight.T
# print(y.shape)
        # logits = model(x).last_hidden_state
        # logits = logits @ model.embeddings.weight.T # for hf mamba2

# h = torch.cat(out.hidden_states, dim=-1) # [batch, seq_len, (num_hidden_layers + 1 ?) * d_model]
# print(h.shape)
# print(model)




In [ ]:
# @title from SimplerMambaSSM.ipynb
# https://colab.research.google.com/drive/1g9qpeVcFa0ca0cnhmqusO4RZtQdh9umY?usp=sharing
# https://www.reddit.com/r/MachineLearning/comments/18d65bz/d_thoughts_on_mamba/?share_id=6IQS8bFpiQNGLsQ-x5xZW
import torch
import torch.nn as nn
from torch.nn import  functional as F
from torch.nn.parameter import Parameter
from tqdm import tqdm
from mamba_ssm import Mamba
# ---------
with open("input.txt", "r") as f:
  text = f.read()

# Unique characters
chars = sorted(list(set(text)))
print(''.join(chars))
vocab_size = len(chars)
print(vocab_size)

#Tokenizers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda xx: [stoi[x] for x in xx]
decode = lambda xx: ''.join([itos[x] for x in xx])
encode("Hello!")
print(decode(encode("Hello!")))


# train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  # generate targets and context
  if split == "train":
    data = train_data
  else:
    data = val_data
  index = torch.randint(0,len(data)-block_size,(batch_size,))
  x = torch.stack([data[ind:ind+block_size] for ind in index])
  y = torch.stack([data[ind+1:ind+block_size+1] for ind in index])
  return x.to(device),y.to(device)


@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'test']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


class FeedForward(nn.Module):
  def __init__(self, n_embed, dropout=.2):
    super().__init__()
    self.ffn = nn.Sequential(
      nn.Linear(n_embed, 4*n_embed), nn.ReLU(),
      nn.Linear(4*n_embed, n_embed),
      nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.ffn(x)

class Block(nn.Module):
  def __init__(self, n_embed, n_heads):
    super().__init__()
    self.head_size = n_embed // n_heads
    # self.sa_head = MultiHeadAttention(n_heads, self.head_size)
    self.sa_head = Mamba(
      # This module uses roughly 3 * expand * d_model^2 parameters
      d_model=n_embed, # Model dimension d_model
      d_state=16,  # SSM state expansion factor
      d_conv=4,    # Local convolution width
      expand=2,    # Block expansion factor
  ).to("cuda")
    self.ffn = FeedForward(n_embed)
    self.ln1 = nn.RMSNorm(n_embed)
    self.ln2 = nn.RMSNorm(n_embed)

  def forward(self, x):
    x = x + self.sa_head(self.ln1(x))
    x = x + self.ffn(self.ln2(x))
    return x

class BigramNeuralNetwork(nn.Module):
  def __init__(self,vocab_size, n_layers):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    # self.sa_head = MultiHeadAttention(4,int(n_embed/4))
    self.lm_head = nn.Linear(n_embed,vocab_size)
    # self.ffn = FeedForward(n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed,n_heads=n_heads) for _ in range(n_layers)])

  def forward(self, idx, targets=None):
    # idx = idx[:,-block_size:]
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B,T,C_e)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C_e)
    x = tok_emb + pos_emb # (B,T,C_e)
    x = self.blocks(x) # (B,T,C_e)
    logits = self.lm_head(x) # (B,T,vocab_size)
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      logits = logits.view(B,T,C)
    return logits, loss
  def generate(self, idx, max_new_tokens):
    # idx is (B,T)
    idx_next = []
    for i in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      last_timestep = logits[:,-1,:]
      probs = F.softmax(last_timestep, dim=1)
      next_index = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, next_index), dim=1)
    for arr in idx:
      print(decode(arr.cpu().detach().numpy()))
    return idx


#hyperparams
epochs = 100
lr = 1e-3
batch_size = 64
block_size = 256
device = "cuda" if torch.cuda.is_available() else "cpu"
max_iters = 10000
print_iters = 100
eval_iters = 10
eval_interval = 300
n_embed=384
n_heads = 6
n_layers = 6


model = BigramNeuralNetwork(vocab_size, n_layers=n_layers)
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)

# checkpoint = torch.load('model.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
checkpoint_path = None#"./differentattention/model_40.pt"
epoch = 0
if checkpoint_path:
  checkpoint = torch.load(checkpoint_path)
  print(checkpoint)
  if checkpoint['model_state_dict']:
    model.load_state_dict(checkpoint['model_state_dict'].to(device))
  if checkpoint['optimizer_state_dict']:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
device = "cuda"
m = model.to(device)
print("Uses device " + device)
MODEL_CHECKPOINT = "./differentattention/model_{iter}.pt"
losses_data = {"train":[], "test":[]}
for iter in tqdm(range(epoch ,max_iters)):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    losses_data['train'].append(losses['train'].cpu().numpy())
    losses_data['test'].append(losses['test'].cpu().numpy())
    print(f"Step {iter}, train loss:{losses['train']:.4f}, test loss:{losses['test']:.4f}")

  if iter % print_iters == 0:
    losses = estimate_loss()
    torch.save({
            'epoch': iter,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': losses,
            }, MODEL_CHECKPOINT.format(iter=iter))
    losses_data['train'].append(losses['train'].cpu().numpy())
    losses_data['test'].append(losses['test'].cpu().numpy())
    model.eval()
    with torch.no_grad():
      #Generate from the model:
      output = m.generate(torch.zeros((1,2), dtype=torch.long).to(device).contiguous(), 1000  )[0].tolist()

    print(f"Step {iter}, train loss:{losses['train']:.4f}, test loss:{losses['test']:.4f}")
    model.train()

  #Get data
  xb,yb = get_batch("train")

  #Evaluate loss
  logits,loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  torch.nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
  optimizer.step()
torch.save(model.state_dict(), "./differentattention/model.pt")
#Generate from the model:
output = m.generate(torch.zeros((1,2), dtype=torch.long).to(device),1000)[0].tolist()


In [ ]:
# @title mamba S6
# https://arxiv.org/pdf/2312.00752
# https://github.com/state-spaces/mamba

from mamba_ssm import Mamba2
model = Mamba2(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=64,  # SSM state expansion factor, typically 64 or 128
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
y = model(x)
assert y.shape == x.shape



# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py

# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py#L118
pred = MixerModel(d_model=256, n_layer=1, d_intermediate=1, vocab_size=0,
        # ssm_cfg=None,
        # attn_layer_idx=None,
        # attn_cfg=None,
        # norm_epsilon: float = 1e-5,
        rms_norm = True,
        # initializer_cfg=None,
        # fused_add_norm=True,
        # residual_in_fp32=False,
        # device=None,
        # dtype=None,
    )


pred.embedding = lambda x: x

batch=4
seq_len=5
d_model=256
x = torch.randn(batch, seq_len, d_model, device="cuda")
y = pred(x)


# torch.rand()



# # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/config_mamba.py
# from dataclasses import dataclass, field

# dataclass
# class MambaConfig:
#     d_model = 256
#     d_intermediate = 0
#     n_layer = 1
#     vocab_size = 1
#     ssm_cfg: dict = field(default_factory=dict)
#     attn_layer_idx: list = field(default_factory=list)
#     attn_cfg: dict = field(default_factory=dict)
#     rms_norm = True
#     residual_in_fp32 = True
#     fused_add_norm = True
#     pad_vocab_size_multiple = 8
#     tie_embeddings = True


# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py#L215
# pred = MambaLMHeadModel(config: MambaConfig,
#         initializer_cfg=None,
#         device=None,
#         dtype=None,
#     )

# pred.backbone.embedding = lambda x: x



In [ ]:
# training


# https://github.com/redotvideo/mamba-chat/blob/main/train_mamba.py
# https://github.com/redotvideo/mamba-chat/blob/main/trainer/mamba_trainer.py



In [ ]:
# @title _mamba_chunk_scan_combined_fwd
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/ssd_combined.py#L331

# x/z: [b,t,h,d]
# B/C: [b,t,g,s]
# dt: [b,t,h]
# A: [h]
# D: [h,d]/[h]
# def _mamba_chunk_scan_combined_fwd(x, dt, A, B, C, chunk_size, D=None, z=None, dt_bias=None, initial_states=None, seq_idx=None, cu_seqlens=None, dt_softplus=False, dt_limit=(0.0, float("inf"))):
def _mamba_chunk_scan_combined_fwd(x, dt, A, B, C, chunk_size, D=None, z=None, dt_bias=None, initial_states=None, seq_idx=None, dt_softplus=False, dt_limit=(0.0, float("inf"))):
    batch, seqlen, nheads, headdim = x.shape
    _, _, ngroups, dstate = B.shape
    assert nheads % ngroups == 0
    if D is not None:
        assert D.shape == (nheads, headdim) or D.shape == (nheads,)
    if seq_idx is not None:
        assert seq_idx.shape == (batch, seqlen)
    if initial_states is not None:
        assert initial_states.shape == (batch, nheads, headdim, dstate)
    # # (batch, nchunks, chunk_size, chunk_size) or (batch, nchunks, nheads, chunk_size, chunk_size)
    dA_cumsum, dt = _chunk_cumsum_fwd(dt, A, chunk_size, dt_bias=dt_bias, dt_softplus=dt_softplus, dt_limit=dt_limit)
    states = _chunk_state_fwd(B, x, dt, dA_cumsum, seq_idx=seq_idx, states_in_fp32=True)
    states, final_states = _state_passing_fwd(rearrange(states, "... p n -> ... (p n)"), dA_cumsum[:, :, :, -1],
                                              initial_states=rearrange(initial_states, "... p n -> ... (p n)") if initial_states is not None else None,
                                              seq_idx=seq_idx, chunk_size=chunk_size, out_dtype=C.dtype)
    states, final_states = [rearrange(t, "... (p n) -> ... p n", n=dstate) for t in [states, final_states]]
    # states_tmp0 = rearrange(_state_passing_fwd(rearrange(states_tmp0, "... p n -> ... (p n)"), dA_cumsum_tmp0[:, :, :, -1], chunk_size=chunk_size), "... (p n) -> ... p n", n=dstate)
    # states_tmp1 = rearrange(_state_passing_fwd(rearrange(states_tmp1, "... p n -> ... (p n)"), dA_cumsum_tmp1[:, :, :, -1], chunk_size=chunk_size), "... (p n) -> ... p n", n=dstate)
    CB = _bmm_chunk_fwd(C, B, chunk_size, seq_idx=seq_idx, output_dtype=torch.float32)
    out, out_x = _chunk_scan_fwd(CB, x, dt, dA_cumsum, C, states, D=D, z=z, seq_idx=seq_idx)
    # if cu_seqlens is None:
    return out, out_x, dt, dA_cumsum, states, final_states
    # else:
    #     assert batch == 1, "passing cu_seqlens to get the varlen states is only supported if batch dimension is 1"
    #     varlen_states = chunk_state_varlen(B.squeeze(0), x.squeeze(0), dt.squeeze(0), dA_cumsum.squeeze(0), cu_seqlens, states.squeeze(0))
    #     return out, out_x, dt, dA_cumsum, states, final_states, varlen_states




In [ ]:
# @title selective_state_update_ref
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/selective_state_update.py

# state: [b,h,d,s]
# x/z: [b,h,d]
# B/C: [b,g,s]
# dt: [b,h,d]
# dt_bias: [h,d]
# A: [h,d,s]
# D: [h,d]

def selective_state_update_ref(state, x, dt, A, B, C, D=None, z=None, dt_bias=None):
    batch, nheads, dim, dstate = state.shape
    ngroups = B.shape[1]
    assert nheads % ngroups == 0, "nheads must be divisible by ngroups"

    dt = F.softplus(dt + dt_bias) # [b,h,d]
    dA = torch.exp(dt.unsqueeze(-1) * A)  # [b,h,d,1]*[h,d,s]->[b,h,d,s]
    # B = repeat(B, "b g n -> b (g h) n", h=nheads // ngroups)  # (batch, nheads, dstate)
    # B = B.repeat(1,nheads//ngroups,1) # [b,g,s]->[b,h,s]
    B = B.repeat_interleave(nheads//ngroups, dim=1) # [b,g,s]->[b,h,s]
    C = C.repeat_interleave(nheads//ngroups, dim=1) # [b,g,s]->[b,h,s]
    dB = dt.unsqueeze(-1) * B.unsqueeze(-2) # [b,h,d,1]*[b,h,1,s]->[b,h,d,s]
    state.copy_(state * dA + dB * x.unsqueeze(-1)) # [b,h,d,s]
    out = torch.einsum("bhdn,bhn->bhd", state.to(C.dtype), C) # [b,h,d]
    out += (x * D).to(out.dtype)
    out = out * F.silu(z).to(x.dtype)
    return out # [b,h,d]


In [ ]:
# @title mamba_ssm/models/mamba2.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, repeat
try: from causal_conv1d import causal_conv1d_fn, causal_conv1d_update
except ImportError: causal_conv1d_fn, causal_conv1d_update = None, None
try: from causal_conv1d.causal_conv1d_varlen import causal_conv1d_varlen_states
except ImportError: causal_conv1d_varlen_states = None
try: from mamba_ssm.ops.triton.selective_state_update import selective_state_update
except ImportError: selective_state_update = None
from mamba_ssm.distributed.tensor_parallel import ColumnParallelLinear, RowParallelLinear
from mamba_ssm.distributed.distributed_utils import all_reduce, reduce_scatter
from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined
from mamba_ssm.ops.triton.ssd_combined import mamba_split_conv1d_scan_combined

class Mamba2(nn.Module):
    def __init__(self, d_model, d_state=128, d_conv=4, expand=2, headdim=64, ngroups=1,
        d_ssm=None,  # If not None, we only apply SSM on this many dimensions, the rest uses gated MLP
        conv_init=None,
        A_init_range=(1, 16),
        D_has_hdim=False,
        norm_before_gate=False,
        dt_min=0.001, dt_max=0.1, dt_init_floor=1e-4, dt_limit=(0.0, float("inf")),
        bias=False,
        conv_bias=True,
        # Fused kernel and sharding options
        layer_idx=None,  # Absorb kwarg for general module
        sequence_parallel=True, device=None, dtype=None):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.conv_init = conv_init
        self.expand = expand
        self.sequence_parallel = sequence_parallel
        self.d_inner = self.expand * self.d_model
        assert self.d_inner == self.expand * self.d_model
        self.headdim = headdim
        self.d_ssm = d_ssm or self.d_inner
        self.ngroups = ngroups
        assert self.d_ssm % self.headdim == 0
        self.nheads = self.d_ssm // self.headdim
        self.D_has_hdim = D_has_hdim
        self.norm_before_gate = norm_before_gate
        self.dt_limit = dt_limit
        self.activation = "silu"
        self.layer_idx = layer_idx

        # Order: [z, x, B, C, dt]
        d_in_proj = 2 * self.d_inner + 2 * self.ngroups * self.d_state + self.nheads
        self.in_proj = nn.Linear(self.d_model, d_in_proj, bias=bias)

        conv_dim = self.d_ssm + 2 * self.ngroups * self.d_state
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, bias=conv_bias, kernel_size=d_conv, groups=conv_dim, padding=d_conv-1)
        if self.conv_init is not None:
            nn.init.uniform_(self.conv1d.weight, -self.conv_init, self.conv_init)

        self.act = nn.SiLU()

        # Initialize log dt bias
        dt = torch.exp(torch.rand(self.nheads) * (math.log(dt_max) - math.log(dt_min)) + math.log(dt_min))
        dt = torch.clamp(dt, min=dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + torch.log(-torch.expm1(-dt))
        self.dt_bias = nn.Parameter(inv_dt)
        # Just to be explicit. Without this we already don't put wd on dt_bias because of the check
        # name.endswith("bias") in param_grouping.py
        self.dt_bias._no_weight_decay = True

        assert A_init_range[0] > 0 and A_init_range[1] >= A_init_range[0]
        A = torch.empty(self.nheads, dtype=torch.float32, device=device).uniform_(*A_init_range)
        A_log = torch.log(A).to(dtype=dtype)
        self.A_log = nn.Parameter(A_log)
        self.A_log._no_weight_decay = True

        # D "skip" parameter
        self.D = nn.Parameter(torch.ones(self.d_ssm if self.D_has_hdim else self.nheads, device=device))
        self.D._no_weight_decay = True

        self.norm = RMSNormGated(self.d_ssm, eps=1e-5, norm_before_gate=self.norm_before_gate, group_size=self.d_ssm//ngroups)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias)

    def forward(self, u, seq_idx=None, cu_seqlens=None, inference_params=None):
        batch, seqlen, dim = u.shape
        conv_state, ssm_state = None, None
        if inference_params is not None:
            inference_batch = cu_seqlens.shape[0] - 1 if cu_seqlens is not None else batch
            conv_state, ssm_state = self._get_states_from_cache(inference_params, inference_batch)
            if inference_params.seqlen_offset > 0:
                # The states are updated inplace
                out, _, _ = self.step(u, conv_state, ssm_state)
                return out

        zxbcdt = self.in_proj(u)  # (B, L, d_in_proj) or (B * L, d_in_proj)
        # If the model is loaded in fp16, without the .float() here, A might be -inf
        A = -torch.exp(self.A_log.float())  # (nheads) or (d_inner, d_state)
        dt_limit_kwargs = {} if self.dt_limit == (0.0, float("inf")) else dict(dt_limit=self.dt_limit)

        if self.use_mem_eff_path and inference_params is None: # T,
            out = mamba_split_conv1d_scan_combined(zxbcdt,
                self.conv1d.weight.unsqueeze(1), self.conv1d.bias, self.dt_bias,
                A,
                D=rearrange(self.D, "(h p) -> h p", p=self.headdim) if self.D_has_hdim else self.D,
                chunk_size=256,
                seq_idx=seq_idx,
                activation=self.activation,
                rmsnorm_weight=self.norm.weight
                rmsnorm_eps=self.norm.eps
                outproj_weight=self.out_proj.weight,
                outproj_bias=self.out_proj.bias,
                headdim=None if self.D_has_hdim else self.headdim,
                ngroups=self.ngroups,
                norm_before_gate=self.norm_before_gate,
                **dt_limit_kwargs,
            )
        else:
            d_mlp = (zxbcdt.shape[-1] - 2* self.d_ssm - 2* self.ngroups*self.d_state - self.nheads) // 2
            z0, x0, z, xBC, dt = torch.split(zxbcdt, [d_mlp, d_mlp, self.d_ssm, self.d_ssm + 2 * self.ngroups * self.d_state, self.nheads], dim=-1)
            if conv_state is not None:
                if cu_seqlens is None:
                    # If we just take xBC[:, :, -self.d_conv :], it will error if seqlen < self.d_conv
                    # Instead F.pad will pad with zeros if seqlen < self.d_conv, and truncate otherwise.
                    xBC_t = rearrange(xBC, "b l d -> b d l")
                    conv_state.copy_(F.pad(xBC_t, (self.d_conv - xBC_t.shape[-1], 0)))  # Update state (B D W)
                else:
                    assert causal_conv1d_varlen_states is not None, "varlen inference requires causal_conv1d package"
                    assert batch == 1, "varlen inference only supports batch dimension 1"
                    conv_varlen_states = causal_conv1d_varlen_states(xBC.squeeze(0), cu_seqlens, state_len=conv_state.shape[-1])
                    conv_state.copy_(conv_varlen_states)
            assert self.activation in ["silu", "swish"]
            if causal_conv1d_fn is None or self.activation not in ["silu", "swish"]:
                assert seq_idx is None, "varlen conv1d requires the causal_conv1d package"
                xBC = self.act(self.conv1d(xBC.transpose(1,2)).transpose(1, 2)[:, :-(self.d_conv-1)])  # (B, L, self.d_ssm + 2 * ngroups * d_state)
            else:
                xBC = causal_conv1d_fn(xBC.transpose(1, 2), rearrange(self.conv1d.weight, "d 1 w -> d w"), bias=self.conv1d.bias, activation=self.activation, seq_idx=seq_idx).transpose(1, 2)

            x, B, C = torch.split(xBC, [self.d_ssm, self.ngroups*self.d_state, self.ngroups*self.d_state], dim=-1)

            y = mamba_chunk_scan_combined(
                x.unflatten(-1, (-1,self.headdim)), dt, A,
                B.unflatten(-1, (self.ngroups,-1)), C.unflatten(-1, (self.ngroups,-1)),
                chunk_size=256,
                D=rearrange(self.D, "(h p) -> h p", p=self.headdim) if self.D_has_hdim else self.D,
                z=rearrange(z, "b l (h p) -> b l h p", p=self.headdim) if not self.rmsnorm else None,
                dt_bias=self.dt_bias,
                dt_softplus=True,
                seq_idx=seq_idx,
                cu_seqlens=cu_seqlens,
                **dt_limit_kwargs,
                return_final_states=ssm_state is not None,
                return_varlen_states=cu_seqlens is not None and inference_params is not None,
            )

            if ssm_state is not None:
                y, last_state, *rest = y
                if cu_seqlens is None:
                    ssm_state.copy_(last_state)
                else:
                    varlen_states = rest[0]
                    ssm_state.copy_(varlen_states)

            y = y.flatten(-2) # [b,t,d]
            y = self.norm(y, z)
            if d_mlp > 0:
                y = torch.cat([F.silu(z0) * x0, y], dim=-1)
            # if seqlen_og is not None:
            #     y = rearrange(y, "b l d -> (b l) d")
            out = self.out_proj(y)
        return out

    def step(self, hidden_states, conv_state, ssm_state):
        dtype = hidden_states.dtype
        assert hidden_states.shape[1] == 1, "Only support decoding with 1 token at a time for now"
        zxbcdt = self.in_proj(hidden_states.squeeze(1))  # (B 2D)
        d_mlp = (zxbcdt.shape[-1] - 2 * self.d_ssm - 2 * self.ngroups * self.d_state - self.nheads) // 2
        z0, x0, z, xBC, dt = torch.split(zxbcdt, [d_mlp, d_mlp, self.d_ssm, self.d_ssm + 2* self.ngroups*self.d_state, self.nheads], dim=-1)

        # Conv step
        if causal_conv1d_update is None:
            conv_state.copy_(torch.roll(conv_state, shifts=-1, dims=-1))  # Update state (B D W)
            conv_state[:, :, -1] = xBC
            xBC = torch.sum(conv_state * rearrange(self.conv1d.weight, "d 1 w -> d w"), dim=-1)  # (B D)
            if self.conv1d.bias is not None:
                xBC = xBC + self.conv1d.bias
            xBC = self.act(xBC).to(dtype=dtype)
        else:
            xBC = causal_conv1d_update(xBC, conv_state, rearrange(self.conv1d.weight, "d 1 w -> d w"), self.conv1d.bias, self.activation)

        x, B, C = torch.split(xBC, [self.d_ssm, self.ngroups * self.d_state, self.ngroups * self.d_state], dim=-1)
        A = -torch.exp(self.A_log.float())  # (nheads,)

        # SSM step
        if selective_state_update is None:
            assert self.ngroups == 1, "Only support ngroups=1 for this inference code path"
            # Discretize A and B
            dt = F.softplus(dt + self.dt_bias.to(dtype=dt.dtype))  # (batch, nheads)
            dA = torch.exp(dt * A)  # (batch, nheads)
            x = rearrange(x, "b (h p) -> b h p", p=self.headdim)
            dBx = torch.einsum("bh,bn,bhp->bhpn", dt, B, x)
            ssm_state.copy_(ssm_state * rearrange(dA, "b h -> b h 1 1") + dBx)
            y = torch.einsum("bhpn,bn->bhp", ssm_state.to(dtype), C)
            y = y + rearrange(self.D.to(dtype), "h -> h 1") * x
            y = rearrange(y, "b h p -> b (h p)")
            # if not self.rmsnorm:
            #     y = y * self.act(z)  # (B D)
        else:
            A = repeat(A, "h -> h p n", p=self.headdim, n=self.d_state).to(dtype=torch.float32)
            dt = repeat(dt, "b h -> b h p", p=self.headdim)
            dt_bias = repeat(self.dt_bias, "h -> h p", p=self.headdim)
            D = repeat(self.D, "h -> h p", p=self.headdim)
            B = rearrange(B, "b (g n) -> b g n", g=self.ngroups)
            C = rearrange(C, "b (g n) -> b g n", g=self.ngroups)
            x_reshaped = rearrange(x, "b (h p) -> b h p", p=self.headdim)
            # if not self.rmsnorm:
            #     z = rearrange(z, "b (h p) -> b h p", p=self.headdim)
            y = selective_state_update(
                ssm_state, x_reshaped, dt, A, B, C, D, z=z if not self.rmsnorm else None,
                dt_bias=dt_bias, dt_softplus=True
            )
            y = rearrange(y, "b h p -> b (h p)")

        y = self.norm(y, z)
        if d_mlp > 0:
            y = torch.cat([F.silu(z0) * x0, y], dim=-1)
        out = self.out_proj(y)
        return out.unsqueeze(1), conv_state, ssm_state

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        device = self.out_proj.weight.device
        conv_dtype = self.conv1d.weight.dtype if dtype is None else dtype
        conv_state = torch.zeros(batch_size, self.d_conv, self.conv1d.weight.shape[0], device=device, dtype=conv_dtype).transpose(1, 2)
        ssm_dtype = self.in_proj.weight.dtype if dtype is None else dtype
        ssm_state = torch.zeros(batch_size, self.nheads, self.headdim, self.d_state, device=device, dtype=ssm_dtype)
        return conv_state, ssm_state

    def _get_states_from_cache(self, inference_params, batch_size, initialize_states=False):
        assert self.layer_idx is not None
        if self.layer_idx not in inference_params.key_value_memory_dict:
            batch_shape = (batch_size,)
            conv_state = torch.zeros(batch_size, self.d_conv, self.conv1d.weight.shape[0],
                device=self.conv1d.weight.device, dtype=self.conv1d.weight.dtype).transpose(1, 2)
            ssm_state = torch.zeros(batch_size, self.nheads, self.headdim, self.d_state,
                device=self.in_proj.weight.device, dtype=self.in_proj.weight.dtype)
            inference_params.key_value_memory_dict[self.layer_idx] = (conv_state, ssm_state)
        else:
            conv_state, ssm_state = inference_params.key_value_memory_dict[self.layer_idx]
            # TODO: What if batch size changes between generation, and we reuse the same states?
            if initialize_states:
                conv_state.zero_()
                ssm_state.zero_()
        return conv_state, ssm_state


In [ ]:
# @title mamba_ssm/models/block.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/block.py
from typing import Optional
import torch
from torch import nn, Tensor
from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn


class Block(nn.Module):
    def __init__(
        self, dim, mixer_cls, mlp_cls, norm_cls=nn.LayerNorm, fused_add_norm=False, residual_in_fp32=False
    ):
        """
        Simple block wrapping a mixer class with LayerNorm/RMSNorm and residual connection"

        This Block has a slightly different structure compared to a regular
        prenorm Transformer block.
        The standard block is: LN -> MHA/MLP -> Add.
        [Ref: https://arxiv.org/abs/2002.04745]
        Here we have: Add -> LN -> Mixer, returning both
        the hidden_states (output of the mixer) and the residual.
        This is purely for performance reasons, as we can fuse add and LayerNorm.
        The residual needs to be provided (except for the very first block).
        """
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm
        self.norm = norm_cls(dim)
        self.mixer = mixer_cls(dim)
        if mlp_cls is not nn.Identity:
            self.norm2 = norm_cls(dim)
            self.mlp = mlp_cls(dim)
        else:
            self.mlp = None
        if self.fused_add_norm:
            assert RMSNorm is not None, "RMSNorm import fails"
            assert isinstance(
                self.norm, (nn.LayerNorm, RMSNorm)
            ), "Only LayerNorm and RMSNorm are supported for fused_add_norm"

    def forward(self, hidden_states: Tensor, residual: Optional[Tensor] = None, inference_params=None, **mixer_kwargs):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """
        if not self.fused_add_norm:
            residual = (hidden_states + residual) if residual is not None else hidden_states
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            hidden_states, residual = layer_norm_fn(
                hidden_states,
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
                is_rms_norm=isinstance(self.norm, RMSNorm)
            )
        hidden_states = self.mixer(hidden_states, inference_params=inference_params, **mixer_kwargs)

        if self.mlp is not None:
            if not self.fused_add_norm:
                residual = hidden_states + residual
                hidden_states = self.norm2(residual.to(dtype=self.norm2.weight.dtype))
                if self.residual_in_fp32:
                    residual = residual.to(torch.float32)
            else:
                hidden_states, residual = layer_norm_fn(
                    hidden_states,
                    self.norm2.weight,
                    self.norm2.bias,
                    residual=residual,
                    prenorm=True,
                    residual_in_fp32=self.residual_in_fp32,
                    eps=self.norm2.eps,
                    is_rms_norm=isinstance(self.norm2, RMSNorm)
                )
            hidden_states = self.mlp(hidden_states)

        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.mixer.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)



In [ ]:
# @title mamba_ssm/models/mamba2.py base
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, repeat
try: from causal_conv1d import causal_conv1d_fn, causal_conv1d_update
except ImportError: causal_conv1d_fn, causal_conv1d_update = None, None
try: from causal_conv1d.causal_conv1d_varlen import causal_conv1d_varlen_states
except ImportError: causal_conv1d_varlen_states = None
try: from mamba_ssm.ops.triton.selective_state_update import selective_state_update
except ImportError: selective_state_update = None
from mamba_ssm.ops.triton.layernorm_gated import RMSNorm as RMSNormGated
from mamba_ssm.distributed.tensor_parallel import ColumnParallelLinear, RowParallelLinear
from mamba_ssm.distributed.distributed_utils import all_reduce, reduce_scatter
from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined
from mamba_ssm.ops.triton.ssd_combined import mamba_split_conv1d_scan_combined
from huggingface_hub import PyTorchModelHubMixin

class Mamba2(nn.Module, PyTorchModelHubMixin):
    def __init__(self, d_model, d_state=128, d_conv=4, expand=2, headdim=64, ngroups=1,
        d_ssm=None,  # If not None, we only apply SSM on this many dimensions, the rest uses gated MLP
        conv_init=None,
        A_init_range=(1, 16),
        D_has_hdim=False,
        rmsnorm=True,
        norm_before_gate=False,
        dt_min=0.001, dt_max=0.1, dt_init_floor=1e-4, dt_limit=(0.0, float("inf")),
        bias=False,
        conv_bias=True,
        # Fused kernel and sharding options
        chunk_size=256,
        use_mem_eff_path=True,
        layer_idx=None,  # Absorb kwarg for general module
        process_group=None, sequence_parallel=True, device=None, dtype=None):
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.conv_init = conv_init
        self.expand = expand
        self.process_group = process_group
        self.sequence_parallel = sequence_parallel
        self.world_size = 1 if process_group is None else process_group.size()
        self.local_rank = 0 if process_group is None else process_group.rank()
        self.d_inner = (self.expand * self.d_model) // self.world_size
        assert self.d_inner * self.world_size == self.expand * self.d_model
        self.headdim = headdim
        self.d_ssm = self.d_inner if d_ssm is None else d_ssm // self.world_size
        assert ngroups % self.world_size == 0
        self.ngroups = ngroups // self.world_size
        assert self.d_ssm % self.headdim == 0
        self.nheads = self.d_ssm // self.headdim
        self.D_has_hdim = D_has_hdim
        self.rmsnorm = rmsnorm
        self.norm_before_gate = norm_before_gate
        self.dt_limit = dt_limit
        self.activation = "silu"
        self.chunk_size = chunk_size
        self.use_mem_eff_path = use_mem_eff_path
        self.layer_idx = layer_idx

        # Order: [z, x, B, C, dt]
        d_in_proj = 2 * self.d_inner + 2 * self.ngroups * self.d_state + self.nheads
        if self.process_group is None:
            self.in_proj = nn.Linear(self.d_model, d_in_proj, bias=bias, **factory_kwargs)
        else:
            self.in_proj = ColumnParallelLinear(self.d_model, d_in_proj * self.world_size, bias=bias, process_group=self.process_group, sequence_parallel=self.sequence_parallel, **factory_kwargs)

        conv_dim = self.d_ssm + 2 * self.ngroups * self.d_state
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim, bias=conv_bias, kernel_size=d_conv, groups=conv_dim, padding=d_conv-1, **factory_kwargs)
        if self.conv_init is not None:
            nn.init.uniform_(self.conv1d.weight, -self.conv_init, self.conv_init)

        self.act = nn.SiLU()

        # Initialize log dt bias
        dt = torch.exp(torch.rand(self.nheads, **factory_kwargs) * (math.log(dt_max) - math.log(dt_min)) + math.log(dt_min))
        dt = torch.clamp(dt, min=dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + torch.log(-torch.expm1(-dt))
        self.dt_bias = nn.Parameter(inv_dt)
        # Just to be explicit. Without this we already don't put wd on dt_bias because of the check
        # name.endswith("bias") in param_grouping.py
        self.dt_bias._no_weight_decay = True

        assert A_init_range[0] > 0 and A_init_range[1] >= A_init_range[0]
        A = torch.empty(self.nheads, dtype=torch.float32, device=device).uniform_(*A_init_range)
        A_log = torch.log(A).to(dtype=dtype)
        self.A_log = nn.Parameter(A_log)
        self.A_log._no_weight_decay = True

        # D "skip" parameter
        self.D = nn.Parameter(torch.ones(self.d_ssm if self.D_has_hdim else self.nheads, device=device))
        self.D._no_weight_decay = True

        if self.rmsnorm:
            assert RMSNormGated is not None
            self.norm = RMSNormGated(self.d_ssm, eps=1e-5, norm_before_gate=self.norm_before_gate, group_size=self.d_ssm // ngroups, **factory_kwargs)

        if self.process_group is None:
            self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias, **factory_kwargs)
        else:
            self.out_proj = RowParallelLinear(self.d_inner * self.world_size, self.d_model, bias=bias, process_group=self.process_group, sequence_parallel=self.sequence_parallel, **factory_kwargs)

    def forward(self, u, seqlen=None, seq_idx=None, cu_seqlens=None, inference_params=None):
        """
        u: (batch, seqlen, hidden_dim) if seqlen=None.
            If seqlen is not None, u is (batch * seqlen, hidden_dim). This is so that when we
            split u during sequence parallel, we split the batch * seqlen dimension
            (in case batch is small).
        Returns: same shape as u
        """
        seqlen_og = seqlen
        if seqlen is None:
            batch, seqlen, dim = u.shape
        else:
            batch_seqlen, dim = u.shape
            batch = batch_seqlen // seqlen

        conv_state, ssm_state = None, None
        if inference_params is not None:
            inference_batch = cu_seqlens.shape[0] - 1 if cu_seqlens is not None else batch
            conv_state, ssm_state = self._get_states_from_cache(inference_params, inference_batch)
            if inference_params.seqlen_offset > 0:
                # The states are updated inplace
                out, _, _ = self.step(u, conv_state, ssm_state)
                return out

        zxbcdt = self.in_proj(u)  # (B, L, d_in_proj) or (B * L, d_in_proj)
        if seqlen_og is not None:
            zxbcdt = rearrange(zxbcdt, "(b l) d -> b l d", l=seqlen)
        # If the model is loaded in fp16, without the .float() here, A might be -inf
        A = -torch.exp(self.A_log.float())  # (nheads) or (d_inner, d_state)
        dt_limit_kwargs = {} if self.dt_limit == (0.0, float("inf")) else dict(dt_limit=self.dt_limit)
        if self.use_mem_eff_path and inference_params is None:
            out = mamba_split_conv1d_scan_combined(
                zxbcdt,
                rearrange(self.conv1d.weight, "d 1 w -> d w"),
                self.conv1d.bias,
                self.dt_bias,
                A,
                D=rearrange(self.D, "(h p) -> h p", p=self.headdim) if self.D_has_hdim else self.D,
                chunk_size=self.chunk_size,
                seq_idx=seq_idx,
                activation=self.activation,
                rmsnorm_weight=self.norm.weight if self.rmsnorm else None,
                rmsnorm_eps=self.norm.eps if self.rmsnorm else 1e-6,
                outproj_weight=self.out_proj.weight,
                outproj_bias=self.out_proj.bias,
                headdim=None if self.D_has_hdim else self.headdim,
                ngroups=self.ngroups,
                norm_before_gate=self.norm_before_gate,
                **dt_limit_kwargs,
            )
            if seqlen_og is not None:
                out = rearrange(out, "b l d -> (b l) d")
            if self.process_group is not None:
                reduce_fn = reduce_scatter if self.sequence_parallel else all_reduce
                out = reduce_fn(out, self.process_group)
        else:
            d_mlp = (zxbcdt.shape[-1] - 2 * self.d_ssm - 2 * self.ngroups * self.d_state - self.nheads) // 2
            z0, x0, z, xBC, dt = torch.split(
                zxbcdt,
                [d_mlp, d_mlp, self.d_ssm, self.d_ssm + 2 * self.ngroups * self.d_state, self.nheads],
                dim=-1
            )
            if conv_state is not None:
                if cu_seqlens is None:
                    # If we just take xBC[:, :, -self.d_conv :], it will error if seqlen < self.d_conv
                    # Instead F.pad will pad with zeros if seqlen < self.d_conv, and truncate otherwise.
                    xBC_t = rearrange(xBC, "b l d -> b d l")
                    conv_state.copy_(F.pad(xBC_t, (self.d_conv - xBC_t.shape[-1], 0)))  # Update state (B D W)
                else:
                    assert causal_conv1d_varlen_states is not None, "varlen inference requires causal_conv1d package"
                    assert batch == 1, "varlen inference only supports batch dimension 1"
                    conv_varlen_states = causal_conv1d_varlen_states(
                        xBC.squeeze(0), cu_seqlens, state_len=conv_state.shape[-1]
                    )
                    conv_state.copy_(conv_varlen_states)
            assert self.activation in ["silu", "swish"]
            if causal_conv1d_fn is None or self.activation not in ["silu", "swish"]:
                assert seq_idx is None, "varlen conv1d requires the causal_conv1d package"
                xBC = self.act(
                    self.conv1d(xBC.transpose(1, 2)).transpose(1, 2)[:, :-(self.d_conv - 1)]
                )  # (B, L, self.d_ssm + 2 * ngroups * d_state)
            else:
                xBC = causal_conv1d_fn(
                    xBC.transpose(1, 2),
                    rearrange(self.conv1d.weight, "d 1 w -> d w"),
                    bias=self.conv1d.bias,
                    activation=self.activation,
                    seq_idx=seq_idx,
                ).transpose(1, 2)
            x, B, C = torch.split(xBC, [self.d_ssm, self.ngroups * self.d_state, self.ngroups * self.d_state], dim=-1)
            y = mamba_chunk_scan_combined(
                rearrange(x, "b l (h p) -> b l h p", p=self.headdim),
                dt,
                A,
                rearrange(B, "b l (g n) -> b l g n", g=self.ngroups),
                rearrange(C, "b l (g n) -> b l g n", g=self.ngroups),
                chunk_size=self.chunk_size,
                D=rearrange(self.D, "(h p) -> h p", p=self.headdim) if self.D_has_hdim else self.D,
                z=rearrange(z, "b l (h p) -> b l h p", p=self.headdim) if not self.rmsnorm else None,
                dt_bias=self.dt_bias,
                dt_softplus=True,
                seq_idx=seq_idx,
                cu_seqlens=cu_seqlens,
                **dt_limit_kwargs,
                return_final_states=ssm_state is not None,
                return_varlen_states=cu_seqlens is not None and inference_params is not None,
            )
            if ssm_state is not None:
                y, last_state, *rest = y
                if cu_seqlens is None:
                    ssm_state.copy_(last_state)
                else:
                    varlen_states = rest[0]
                    ssm_state.copy_(varlen_states)
            y = rearrange(y, "b l h p -> b l (h p)")
            if self.rmsnorm:
                y = self.norm(y, z)
            if d_mlp > 0:
                y = torch.cat([F.silu(z0) * x0, y], dim=-1)
            if seqlen_og is not None:
                y = rearrange(y, "b l d -> (b l) d")
            out = self.out_proj(y)
        return out

    def step(self, hidden_states, conv_state, ssm_state):
        dtype = hidden_states.dtype
        assert hidden_states.shape[1] == 1, "Only support decoding with 1 token at a time for now"
        zxbcdt = self.in_proj(hidden_states.squeeze(1))  # (B 2D)
        d_mlp = (zxbcdt.shape[-1] - 2 * self.d_ssm - 2 * self.ngroups * self.d_state - self.nheads) // 2
        z0, x0, z, xBC, dt = torch.split(zxbcdt, [d_mlp, d_mlp, self.d_ssm, self.d_ssm + 2* self.ngroups*self.d_state, self.nheads], dim=-1)

        # Conv step
        if causal_conv1d_update is None:
            conv_state.copy_(torch.roll(conv_state, shifts=-1, dims=-1))  # Update state (B D W)
            conv_state[:, :, -1] = xBC
            xBC = torch.sum(conv_state * rearrange(self.conv1d.weight, "d 1 w -> d w"), dim=-1)  # (B D)
            if self.conv1d.bias is not None:
                xBC = xBC + self.conv1d.bias
            xBC = self.act(xBC).to(dtype=dtype)
        else:
            xBC = causal_conv1d_update(xBC, conv_state, rearrange(self.conv1d.weight, "d 1 w -> d w"), self.conv1d.bias, self.activation)

        x, B, C = torch.split(xBC, [self.d_ssm, self.ngroups * self.d_state, self.ngroups * self.d_state], dim=-1)
        A = -torch.exp(self.A_log.float())  # (nheads,)

        # SSM step
        if selective_state_update is None:
            assert self.ngroups == 1, "Only support ngroups=1 for this inference code path"
            # Discretize A and B
            dt = F.softplus(dt + self.dt_bias.to(dtype=dt.dtype))  # (batch, nheads)
            dA = torch.exp(dt * A)  # (batch, nheads)
            x = rearrange(x, "b (h p) -> b h p", p=self.headdim)
            dBx = torch.einsum("bh,bn,bhp->bhpn", dt, B, x)
            ssm_state.copy_(ssm_state * rearrange(dA, "b h -> b h 1 1") + dBx)
            y = torch.einsum("bhpn,bn->bhp", ssm_state.to(dtype), C)
            y = y + rearrange(self.D.to(dtype), "h -> h 1") * x
            y = rearrange(y, "b h p -> b (h p)")
            if not self.rmsnorm:
                y = y * self.act(z)  # (B D)
        else:
            A = repeat(A, "h -> h p n", p=self.headdim, n=self.d_state).to(dtype=torch.float32)
            dt = repeat(dt, "b h -> b h p", p=self.headdim)
            dt_bias = repeat(self.dt_bias, "h -> h p", p=self.headdim)
            D = repeat(self.D, "h -> h p", p=self.headdim)
            B = rearrange(B, "b (g n) -> b g n", g=self.ngroups)
            C = rearrange(C, "b (g n) -> b g n", g=self.ngroups)
            x_reshaped = rearrange(x, "b (h p) -> b h p", p=self.headdim)
            if not self.rmsnorm:
                z = rearrange(z, "b (h p) -> b h p", p=self.headdim)
            y = selective_state_update(
                ssm_state, x_reshaped, dt, A, B, C, D, z=z if not self.rmsnorm else None,
                dt_bias=dt_bias, dt_softplus=True
            )
            y = rearrange(y, "b h p -> b (h p)")
        if self.rmsnorm:
            y = self.norm(y, z)
        if d_mlp > 0:
            y = torch.cat([F.silu(z0) * x0, y], dim=-1)
        out = self.out_proj(y)
        return out.unsqueeze(1), conv_state, ssm_state

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        device = self.out_proj.weight.device
        conv_dtype = self.conv1d.weight.dtype if dtype is None else dtype
        conv_state = torch.zeros(batch_size, self.d_conv, self.conv1d.weight.shape[0], device=device, dtype=conv_dtype).transpose(1, 2)
        ssm_dtype = self.in_proj.weight.dtype if dtype is None else dtype
        ssm_state = torch.zeros(batch_size, self.nheads, self.headdim, self.d_state, device=device, dtype=ssm_dtype)
        return conv_state, ssm_state

    def _get_states_from_cache(self, inference_params, batch_size, initialize_states=False):
        assert self.layer_idx is not None
        if self.layer_idx not in inference_params.key_value_memory_dict:
            batch_shape = (batch_size,)
            conv_state = torch.zeros(batch_size, self.d_conv, self.conv1d.weight.shape[0],
                device=self.conv1d.weight.device, dtype=self.conv1d.weight.dtype).transpose(1, 2)
            ssm_state = torch.zeros(batch_size, self.nheads, self.headdim, self.d_state,
                device=self.in_proj.weight.device, dtype=self.in_proj.weight.dtype)
            inference_params.key_value_memory_dict[self.layer_idx] = (conv_state, ssm_state)
        else:
            conv_state, ssm_state = inference_params.key_value_memory_dict[self.layer_idx]
            # TODO: What if batch size changes between generation, and we reuse the same states?
            if initialize_states:
                conv_state.zero_()
                ssm_state.zero_()
        return conv_state, ssm_state


In [ ]:
# @title mamba_ssm/modules/mamba2_simple.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba2_simple.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, repeat

try: from causal_conv1d import causal_conv1d_fn
except ImportError: causal_conv1d_fn = None
# from mamba_ssm.ops.triton.layernorm_gated import RMSNorm as RMSNormGated, LayerNorm
# from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined
# from mamba_ssm.ops.triton.ssd_combined import mamba_split_conv1d_scan_combined

class Mamba2Simple(nn.Module):
    def __init__(self, d_model, d_state=64, d_conv=4,
        conv_init=None,
        expand=2, headdim=128, ngroups=1,
        A_init_range=(1, 16),
        dt_min=0.001, dt_max=0.1, dt_init_floor=1e-4, dt_limit=(0.0, float("inf")),
        learnable_init_states=False,
        activation="swish",
        bias=False, conv_bias=True,
        # Fused kernel and sharding options
        chunk_size=256, use_mem_eff_path=True,
        layer_idx=None,  # Absorb kwarg for general module
        device=None, dtype=None,):

        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_conv = d_conv
        self.conv_init = conv_init
        self.expand = expand
        self.d_inner = self.expand * self.d_model
        self.headdim = headdim
        self.ngroups = ngroups
        assert self.d_inner % self.headdim == 0
        self.nheads = self.d_inner // self.headdim
        self.dt_limit = dt_limit
        self.learnable_init_states = learnable_init_states
        self.activation = activation
        self.chunk_size = chunk_size
        self.use_mem_eff_path = use_mem_eff_path
        self.layer_idx = layer_idx

        # Order: [z, x, B, C, dt]
        d_in_proj = 2 * self.d_inner + 2 * self.ngroups * self.d_state + self.nheads
        self.in_proj = nn.Linear(self.d_model, d_in_proj, bias=bias, **factory_kwargs)

        conv_dim = self.d_inner + 2 * self.ngroups * self.d_state
        self.conv1d = nn.Conv1d(in_channels=conv_dim, out_channels=conv_dim,
            bias=conv_bias,
            kernel_size=d_conv,
            groups=conv_dim, padding=d_conv - 1, **factory_kwargs,)
        if self.conv_init is not None:
            nn.init.uniform_(self.conv1d.weight, -self.conv_init, self.conv_init)
        # self.conv1d.weight._no_weight_decay = True

        if self.learnable_init_states:
            self.init_states = nn.Parameter(torch.zeros(self.nheads, self.headdim, self.d_state, **factory_kwargs))
            self.init_states._no_weight_decay = True

        self.act = nn.SiLU()

        # Initialize log dt bias
        dt = torch.exp(torch.rand(self.nheads, **factory_kwargs) * (math.log(dt_max) - math.log(dt_min)) + math.log(dt_min))
        dt = torch.clamp(dt, min=dt_init_floor)
        # Inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        inv_dt = dt + torch.log(-torch.expm1(-dt))
        self.dt_bias = nn.Parameter(inv_dt)
        # Just to be explicit. Without this we already don't put wd on dt_bias because of the check
        # name.endswith("bias") in param_grouping.py
        self.dt_bias._no_weight_decay = True

        # A parameter
        assert A_init_range[0] > 0 and A_init_range[1] >= A_init_range[0]
        A = torch.empty(self.nheads, dtype=torch.float32, device=device).uniform_(*A_init_range)
        A_log = torch.log(A).to(dtype=dtype)
        self.A_log = nn.Parameter(A_log)
        # self.register_buffer("A_log", torch.zeros(self.nheads, dtype=torch.float32, device=device), persistent=True)
        self.A_log._no_weight_decay = True

        # D "skip" parameter
        self.D = nn.Parameter(torch.ones(self.nheads, device=device))
        self.D._no_weight_decay = True

        # Extra normalization layer right before output projection
        self.norm = RMSNormGated(self.d_inner, eps=1e-5, norm_before_gate=False, **factory_kwargs)
        self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias, **factory_kwargs)

    def forward(self, u, seq_idx=None):
        """u: (B, L, D)
        Returns: same shape as u"""
        batch, seqlen, dim = u.shape

        zxbcdt = self.in_proj(u)  # (B, L, d_in_proj)
        A = -torch.exp(self.A_log)  # (nheads) or (d_inner, d_state)
        initial_states=repeat(self.init_states, "... -> b ...", b=batch) if self.learnable_init_states else None
        dt_limit_kwargs = {} if self.dt_limit == (0.0, float("inf")) else dict(dt_limit=self.dt_limit)

        if self.use_mem_eff_path:
            # Fully fused path
            out = mamba_split_conv1d_scan_combined(
                zxbcdt,
                rearrange(self.conv1d.weight, "d 1 w -> d w"),
                self.conv1d.bias,
                self.dt_bias,
                A,
                D=self.D,
                chunk_size=self.chunk_size,
                seq_idx=seq_idx,
                activation=self.activation,
                rmsnorm_weight=self.norm.weight,
                rmsnorm_eps=self.norm.eps,
                outproj_weight=self.out_proj.weight,
                outproj_bias=self.out_proj.bias,
                headdim=self.headdim,
                ngroups=self.ngroups,
                norm_before_gate=False,
                initial_states=initial_states,
                **dt_limit_kwargs,
            )
        else:
            z, xBC, dt = torch.split(zxbcdt, [self.d_inner, self.d_inner + 2 * self.ngroups * self.d_state, self.nheads], dim=-1)
            dt = F.softplus(dt + self.dt_bias)  # (B, L, nheads)
            assert self.activation in ["silu", "swish"]

            # 1D Convolution
            if causal_conv1d_fn is None or self.activation not in ["silu", "swish"]:
                xBC = self.act(self.conv1d(xBC.transpose(1, 2)).transpose(1, 2))  # (B, L, self.d_inner + 2 * ngroups * d_state)
                xBC = xBC[:, :seqlen, :]
            else:
                xBC = causal_conv1d_fn(
                    x=xBC.transpose(1, 2),
                    weight=rearrange(self.conv1d.weight, "d 1 w -> d w"),
                    bias=self.conv1d.bias,
                    activation=self.activation,
                ).transpose(1, 2)

            # Split into 3 main branches: X, B, C
            # These correspond to V, K, Q respectively in the SSM/attention duality
            x, B, C = torch.split(xBC, [self.d_inner, self.ngroups * self.d_state, self.ngroups * self.d_state], dim=-1)
            y = mamba_chunk_scan_combined(
                rearrange(x, "b l (h p) -> b l h p", p=self.headdim),
                dt,
                A,
                rearrange(B, "b l (g n) -> b l g n", g=self.ngroups),
                rearrange(C, "b l (g n) -> b l g n", g=self.ngroups),
                chunk_size=self.chunk_size,
                D=self.D,
                z=None,
                seq_idx=seq_idx,
                initial_states=initial_states,
                **dt_limit_kwargs,
            )
            y = rearrange(y, "b l h p -> b l (h p)")

            # Multiply "gate" branch and apply extra normalization layer
            y = self.norm(y, z)
            out = self.out_proj(y)
        return out



In [ ]:
# @title mamba_ssm/modules/mha.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mha.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

try: from flash_attn import flash_attn_with_kvcache
except ImportError: flash_attn_with_kvcache = None
try: from flash_attn.layers.rotary import RotaryEmbedding
except ImportError: RotaryEmbedding = None
try: from causal_conv1d import causal_conv1d_fn, causal_conv1d_update
except ImportError: causal_conv1d_fn, causal_conv1d_update = None, None


def _update_kv_cache(kv, inference_params, layer_idx):
    """kv: (batch_size, seqlen, 2, nheads, head_dim) or (batch_size, 1, 2, nheads, head_dim)"""
    # Pre-allocate memory for key-values for inference.
    num_heads, head_dim = kv.shape[-2:]
    assert layer_idx in inference_params.key_value_memory_dict
    kv_cache, _ = inference_params.key_value_memory_dict[layer_idx]
    # Adjust key and value for inference
    batch_start = inference_params.batch_size_offset
    batch_end = batch_start + kv.shape[0]
    sequence_start = inference_params.seqlen_offset
    sequence_end = sequence_start + kv.shape[1]
    assert batch_end <= kv_cache.shape[0]
    assert sequence_end <= kv_cache.shape[1]
    assert kv_cache is not None
    kv_cache[batch_start:batch_end, sequence_start:sequence_end, ...] = kv
    return kv_cache[batch_start:batch_end, :sequence_end, ...]


class MHA(nn.Module):
    """Multi-head self-attention and cross-attention"""

    def __init__(
        self,
        embed_dim,
        num_heads,
        num_heads_kv=None,
        head_dim=None,  # If None, use embed_dim // num_heads
        mlp_dim=0,
        qkv_proj_bias=True,
        out_proj_bias=True,
        softmax_scale=None,
        causal=False,
        layer_idx=None,
        d_conv=0,
        rotary_emb_dim=0,
        rotary_emb_base=10000.0,
        rotary_emb_interleaved=False,
        device=None,
        dtype=None,
    ) -> None:
        """
        num_heads_kv: can be used to toggle MQA / GQA. If None, use num_heads.
        return_residual: whether to return the input x along with the output. This is for
            performance reason: for post-norm architecture, returning the input allows us
            to fuse the backward of nn.Linear with the residual connection.
        """
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.embed_dim = embed_dim
        self.layer_idx = layer_idx
        self.d_conv = d_conv
        self.rotary_emb_dim = rotary_emb_dim
        self.softmax_scale = softmax_scale
        self.causal = causal

        self.num_heads = num_heads
        self.num_heads_kv = num_heads_kv if num_heads_kv is not None else num_heads
        assert (
            self.num_heads % self.num_heads_kv == 0
        ), "num_heads must be divisible by num_heads_kv"
        if head_dim is None:
            assert self.embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.head_dim = head_dim if head_dim is not None else self.embed_dim // num_heads
        self.mlp_dim = math.ceil(mlp_dim / 256) * 256
        qkv_dim = self.head_dim * (self.num_heads + 2 * self.num_heads_kv)
        out_dim = self.head_dim * self.num_heads

        if self.rotary_emb_dim > 0:
            assert RotaryEmbedding is not None, "rotary requires flash_attn to be installed"
            self.rotary_emb = RotaryEmbedding(
                self.rotary_emb_dim,
                base=rotary_emb_base,
                interleaved=rotary_emb_interleaved,
                device=device,
            )

        self.in_proj = nn.Linear(embed_dim, qkv_dim + self.mlp_dim, bias=qkv_proj_bias, **factory_kwargs)
        if self.d_conv > 0:
            self.conv1d = nn.Conv1d(
                qkv_dim, qkv_dim, kernel_size=self.d_conv, padding=self.d_conv - 1, groups=qkv_dim,
                **factory_kwargs
            )
        self.out_proj = nn.Linear(out_dim + self.mlp_dim // 2, embed_dim, bias=out_proj_bias, **factory_kwargs)

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None):
        dtype = self.out_proj.weight.dtype if dtype is None else dtype
        device = self.out_proj.weight.device
        if self.d_conv > 0:
            conv_state = torch.zeros(
                batch_size, self.conv1d.weight.shape[0], self.d_conv, device=device, dtype=dtype
            )
        else:
            conv_state = None
        kv_cache = torch.empty(
            batch_size, max_seqlen, 2, self.num_heads_kv, self.head_dim, dtype=dtype, device=device,
        )
        return kv_cache, conv_state

    def _update_kv_cache(self, kv, inference_params):
        """kv: (batch_size, seqlen, 2, nheads, head_dim) or (batch_size, 1, 2, nheads, head_dim)"""
        assert self.layer_idx is not None, "Generation requires layer_idx in the constructor"
        return _update_kv_cache(kv, inference_params, self.layer_idx)

    def _apply_rotary_update_kvcache_attention(self, q, kv, inference_params):
        """
        Fast path that combine 3 steps: apply rotary to Q and K, update kv cache, and apply attention.
        q: (batch_size, seqlen_q, nheads, head_dim)
        kv: (batch_size, seqlen_k, 2, nheads_kv, head_dim)
        """
        assert inference_params is not None and inference_params.seqlen_offset > 0
        if self.rotary_emb_dim > 0:
            self.rotary_emb._update_cos_sin_cache(
                inference_params.max_seqlen, device=q.device, dtype=q.dtype
            )
            rotary_cos, rotary_sin = self.rotary_emb._cos_cached, self.rotary_emb._sin_cached
        else:
            rotary_cos, rotary_sin = None, None
        batch = q.shape[0]
        kv_cache, _ = inference_params.key_value_memory_dict[self.layer_idx]
        kv_cache = kv_cache[:batch]
        cache_seqlens = (
            inference_params.lengths_per_sample[:batch]
            if inference_params.lengths_per_sample is not None
            else inference_params.seqlen_offset
        )
        assert flash_attn_with_kvcache is not None, "flash_attn must be installed"
        context = flash_attn_with_kvcache(
            q,
            kv_cache[:, :, 0],
            kv_cache[:, :, 1],
            kv[:, :, 0],
            kv[:, :, 1],
            rotary_cos=rotary_cos,
            rotary_sin=rotary_sin,
            cache_seqlens=cache_seqlens,
            softmax_scale=self.softmax_scale,
            causal=self.causal,
            rotary_interleaved=self.rotary_emb.interleaved if self.rotary_emb_dim > 0 else False,
        )
        return context

    def _update_kvcache_attention(self, q, kv, inference_params):
        """Write kv to inference_params, then do attention"""
        if (
            inference_params.seqlen_offset == 0
            or flash_attn_with_kvcache is None
        ):
            # TODO: this only uses seqlen_offset and not lengths_per_sample.
            kv = self._update_kv_cache(kv, inference_params)
            k, v = kv.unbind(dim=-3)
            k = torch.repeat_interleave(k, dim=2, repeats=self.num_heads // self.num_heads_kv)
            v = torch.repeat_interleave(v, dim=2, repeats=self.num_heads // self.num_heads_kv)
            return F.scaled_dot_product_attention(
                q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2), is_causal=self.causal, scale=self.softmax_scale
            ).transpose(1, 2)
        else:
            batch = q.shape[0]
            kv_cache, _ = inference_params.key_value_memory_dict[self.layer_idx]
            kv_cache = kv_cache[:batch]
            cache_seqlens = (
                inference_params.lengths_per_sample[:batch]
                if inference_params.lengths_per_sample is not None
                else inference_params.seqlen_offset
            )
            return flash_attn_with_kvcache(
                q,
                kv_cache[:, :, 0],
                kv_cache[:, :, 1],
                kv[:, :, 0],
                kv[:, :, 1],
                cache_seqlens=cache_seqlens,
                softmax_scale=self.softmax_scale,
                causal=self.causal,
            )

    def forward(self, x, inference_params=None):
        """
        Arguments:
            x: (batch, seqlen, hidden_dim) (where hidden_dim = num heads * head dim) if
                cu_seqlens is None and max_seqlen is None, else (total, hidden_dim) where total
                is the is the sum of the sequence lengths in the batch.
            inference_params: for generation. Adapted from Megatron-LM (and Apex)
            https://github.com/NVIDIA/apex/blob/3ff1a10f72ec07067c4e44759442329804ac5162/apex/transformer/testing/standalone_transformer_lm.py#L470
        """
        if inference_params is not None and self.layer_idx not in inference_params.key_value_memory_dict:
            inference_params.key_value_memory_dict[self.layer_idx] = self.allocate_inference_cache(
                x.shape[0], inference_params.max_seqlen, dtype=x.dtype
            )
        seqlen_offset = (
            0
            if inference_params is None
            else (
                inference_params.lengths_per_sample
                if inference_params.lengths_per_sample is not None
                else inference_params.seqlen_offset
            )
        )
        rotary_max_seqlen = inference_params.max_seqlen if inference_params is not None else None
        qkv = self.in_proj(x)
        if self.mlp_dim > 0:
            qkv, x_mlp = qkv.split([qkv.shape[-1] - self.mlp_dim, self.mlp_dim], dim=-1)
            x_mlp_up, x_mlp_gate = x_mlp.chunk(2, dim=-1)
            x_mlp = x_mlp_up * F.silu(x_mlp_gate)
        if self.d_conv > 0:
            # The inference code for conv1d is pretty messy, should clean it up
            if (inference_params is None or inference_params.seqlen_offset == 0):
                if causal_conv1d_fn is None:
                    qkv = rearrange(
                        self.conv1d(rearrange(qkv, "b s d -> b d s"))[..., :-(self.d_conv - 1)], "b d s -> b s d"
                    ).contiguous()
                else:
                    qkv = causal_conv1d_fn(
                        qkv.transpose(1, 2),
                        rearrange(self.conv1d.weight, "d 1 w -> d w"),
                        self.conv1d.bias
                    ).transpose(1, 2)
                if inference_params is not None:
                    _, conv_state = inference_params.key_value_memory_dict[self.layer_idx]
                    # If we just take qkv[:, :, -self.d_conv :], it will error if seqlen < self.d_conv
                    # Instead F.pad will pad with zeros if seqlen < self.d_conv, and truncate otherwise.
                    qkv_t = rearrange(qkv, "b l d -> b d l")
                    conv_state.copy_(F.pad(qkv_t, (self.d_conv - qkv_t.shape[-1], 0)))  # Update state (B D W)
            else:
                _, conv_state = inference_params.key_value_memory_dict[self.layer_idx]
                assert qkv.shape[1] == 1, "Only support decoding with 1 token at a time for now"
                qkv = qkv.squeeze(1)
                # Conv step
                if causal_conv1d_update is None:
                    conv_state.copy_(torch.roll(conv_state, shifts=-1, dims=-1))  # Update state (B D W)
                    conv_state[:, :, -1] = qkv
                    qkv = torch.sum(conv_state * rearrange(self.conv1d.weight, "d 1 w -> d w"), dim=-1)  # (B D)
                    if self.conv1d.bias is not None:
                        qkv = qkv + self.conv1d.bias
                else:
                    qkv = causal_conv1d_update(
                        qkv,
                        conv_state,
                        rearrange(self.conv1d.weight, "d 1 w -> d w"),
                        self.conv1d.bias
                    )
                qkv = qkv.unsqueeze(1)
        q, kv = qkv.split([self.num_heads * self.head_dim, self.num_heads_kv * 2 * self.head_dim], dim=-1)
        q = rearrange(q, "... (h d) -> ... h d", d=self.head_dim)
        kv = rearrange(kv, "... (two hkv d) -> ... two hkv d", two=2, d=self.head_dim)
        if (
            inference_params is None
            or inference_params.seqlen_offset == 0
            or (self.rotary_emb_dim == 0 or self.rotary_emb_dim % 16 != 0)
        ):
            if self.rotary_emb_dim > 0:
                q, kv = self.rotary_emb(
                    q, kv, seqlen_offset=seqlen_offset, max_seqlen=rotary_max_seqlen
                )
            if inference_params is None:
                k, v = kv.unbind(dim=-3)
                k = torch.repeat_interleave(k, dim=2, repeats=self.num_heads // self.num_heads_kv)
                v = torch.repeat_interleave(v, dim=2, repeats=self.num_heads // self.num_heads_kv)
                context = F.scaled_dot_product_attention(
                    q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2), is_causal=self.causal, scale=self.softmax_scale
                ).transpose(1, 2)
            else:
                context = self._update_kvcache_attention(q, kv, inference_params)
        else:
            context = self._apply_rotary_update_kvcache_attention(q, kv, inference_params)
        context = rearrange(context, "... h d -> ... (h d)")
        if self.mlp_dim > 0:
            context = torch.cat([context, x_mlp], dim=-1)
        out = self.out_proj(context)
        return out


In [ ]:
# @title mamba_ssm/modules/mlp.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mlp.py
from torch import nn
from torch.nn import functional as F

class GatedMLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None,
        activation=F.silu, bias=False,
        multiple_of=128, device=None, dtype=None,):
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        out_features = out_features if out_features is not None else in_features
        hidden_features = (hidden_features if hidden_features is not None else int(8 * in_features / 3))
        hidden_features = (hidden_features + multiple_of - 1) // multiple_of * multiple_of
        self.fc1 = nn.Linear(in_features, 2 * hidden_features, bias=bias, **factory_kwargs)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_features, out_features, bias=bias, **factory_kwargs)

    def forward(self, x):
        y = self.fc1(x)
        y, gate = y.chunk(2, dim=-1)
        y = y * self.activation(gate)
        y = self.fc2(y)
        return y



In [ ]:
# @title mamba_ssm/models/mixer_seq_simple.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py
import math
from functools import partial
import json
import os
import copy
from collections import namedtuple
import torch
import torch.nn as nn
from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.modules.mamba_simple import Mamba
from mamba_ssm.modules.mamba2 import Mamba2
from mamba_ssm.modules.mha import MHA
from mamba_ssm.modules.mlp import GatedMLP
from mamba_ssm.modules.block import Block
from mamba_ssm.utils.generation import GenerationMixin
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

try:
    from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None


def create_block(
    d_model,
    d_intermediate,
    ssm_cfg=None,
    attn_layer_idx=None,
    attn_cfg=None,
    norm_epsilon=1e-5,
    rms_norm=False,
    residual_in_fp32=False,
    fused_add_norm=False,
    layer_idx=None,
    device=None,
    dtype=None,
):
    if ssm_cfg is None:
        ssm_cfg = {}
    if attn_layer_idx is None:
        attn_layer_idx = []
    if attn_cfg is None:
        attn_cfg = {}
    factory_kwargs = {"device": device, "dtype": dtype}
    if layer_idx not in attn_layer_idx:
        # Create a copy of the config to modify
        ssm_cfg = copy.deepcopy(ssm_cfg) if ssm_cfg is not None else {}
        ssm_layer = ssm_cfg.pop("layer", "Mamba1")
        if ssm_layer not in ["Mamba1", "Mamba2"]:
            raise ValueError(f"Invalid ssm_layer: {ssm_layer}, only support Mamba1 and Mamba2")
        mixer_cls = partial(
            Mamba2 if ssm_layer == "Mamba2" else Mamba,
            layer_idx=layer_idx,
            **ssm_cfg,
            **factory_kwargs
        )
    else:
        mixer_cls = partial(MHA, layer_idx=layer_idx, **attn_cfg, **factory_kwargs)
    norm_cls = partial(nn.LayerNorm if not rms_norm else RMSNorm, eps=norm_epsilon, **factory_kwargs)
    if d_intermediate == 0:
        mlp_cls = nn.Identity
    else:
        mlp_cls = partial(GatedMLP, hidden_features=d_intermediate, out_features=d_model, **factory_kwargs)
    block = Block(
        d_model,
        mixer_cls,
        mlp_cls,
        norm_cls=norm_cls,
        fused_add_norm=fused_add_norm,
        residual_in_fp32=residual_in_fp32,
    )
    block.layer_idx = layer_idx
    return block


# https://github.com/huggingface/transformers/blob/c28d04e9e252a1a099944e325685f14d242ecdcd/src/transformers/models/gpt2/modeling_gpt2.py#L454
def _init_weights(
    module,
    n_layer,
    initializer_range=0.02,  # Now only used for embedding layer.
    rescale_prenorm_residual=True,
    n_residuals_per_layer=1,  # Change to 2 if we have MLP
):
    if isinstance(module, nn.Linear):
        if module.bias is not None:
            if not getattr(module.bias, "_no_reinit", False):
                nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, std=initializer_range)

    if rescale_prenorm_residual:
        # Reinitialize selected weights subject to the OpenAI GPT-2 Paper Scheme:
        #   > A modified initialization which accounts for the accumulation on the residual path with model depth. Scale
        #   > the weights of residual layers at initialization by a factor of 1/√N where N is the # of residual layers.
        #   >   -- GPT-2 :: https://openai.com/blog/better-language-models/
        #
        # Reference (Megatron-LM): https://github.com/NVIDIA/Megatron-LM/blob/main/megatron/model/gpt_model.py
        for name, p in module.named_parameters():
            if name in ["out_proj.weight", "fc2.weight"]:
                # Special Scaled Initialization --> There are 2 Layer Norms per Transformer Block
                # Following Pytorch init, except scale by 1/sqrt(2 * n_layer)
                # We need to reinit p since this code could be called multiple times
                # Having just p *= scale would repeatedly scale it down
                nn.init.kaiming_uniform_(p, a=math.sqrt(5))
                with torch.no_grad():
                    p /= math.sqrt(n_residuals_per_layer * n_layer)


class MixerModel(nn.Module):
    def __init__(
        self,
        d_model,
        n_layer,
        d_intermediate,
        vocab_size,
        ssm_cfg=None,
        attn_layer_idx=None,
        attn_cfg=None,
        norm_epsilon: float = 1e-5,
        rms_norm: bool = False,
        initializer_cfg=None,
        fused_add_norm=False,
        residual_in_fp32=False,
        device=None,
        dtype=None,
    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32

        self.embedding = nn.Embedding(vocab_size, d_model, **factory_kwargs)

        # We change the order of residual and layer norm:
        # Instead of LN -> Attn / MLP -> Add, we do:
        # Add -> LN -> Attn / MLP / Mixer, returning both the residual branch (output of Add) and
        # the main branch (output of MLP / Mixer). The model definition is unchanged.
        # This is for performance reason: we can fuse add + layer_norm.
        self.fused_add_norm = fused_add_norm
        if self.fused_add_norm:
            if layer_norm_fn is None or rms_norm_fn is None:
                raise ImportError("Failed to import Triton LayerNorm / RMSNorm kernels")

        self.layers = nn.ModuleList([
                create_block(
                    d_model,
                    d_intermediate=d_intermediate,
                    ssm_cfg=ssm_cfg,
                    attn_layer_idx=attn_layer_idx,
                    attn_cfg=attn_cfg,
                    norm_epsilon=norm_epsilon,
                    rms_norm=rms_norm,
                    residual_in_fp32=residual_in_fp32,
                    fused_add_norm=fused_add_norm,
                    layer_idx=i,
                    **factory_kwargs,
                )
                for i in range(n_layer)])

        self.norm_f = (nn.LayerNorm if not rms_norm else RMSNorm)(
            d_model, eps=norm_epsilon, **factory_kwargs
        )

        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
                n_residuals_per_layer=1 if d_intermediate == 0 else 2,  # 2 if we have MLP
            )
        )

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return {
            i: layer.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)
            for i, layer in enumerate(self.layers)
        }

    def forward(self, input_ids, inference_params=None, **mixer_kwargs):
        hidden_states = self.embedding(input_ids)
        residual = None
        for layer in self.layers:
            hidden_states, residual = layer(hidden_states, residual, inference_params=inference_params, **mixer_kwargs)
        if not self.fused_add_norm:
            residual = (hidden_states + residual) if residual is not None else hidden_states
            hidden_states = self.norm_f(residual.to(dtype=self.norm_f.weight.dtype))
        else:
            # Set prenorm=False here since we don't need the residual
            hidden_states = layer_norm_fn(
                hidden_states,
                self.norm_f.weight,
                self.norm_f.bias,
                eps=self.norm_f.eps,
                residual=residual,
                prenorm=False,
                residual_in_fp32=self.residual_in_fp32,
                is_rms_norm=isinstance(self.norm_f, RMSNorm)
            )
        return hidden_states


class MambaLMHeadModel(nn.Module, GenerationMixin):
    def __init__(self, config: MambaConfig, initializer_cfg=None, device=None, dtype=None,) -> None:
        self.config = config
        d_model = config.d_model
        n_layer = config.n_layer
        d_intermediate = config.d_intermediate
        vocab_size = config.vocab_size
        ssm_cfg = config.ssm_cfg
        attn_layer_idx = config.attn_layer_idx
        attn_cfg = config.attn_cfg
        rms_norm = config.rms_norm
        residual_in_fp32 = config.residual_in_fp32
        fused_add_norm = config.fused_add_norm
        pad_vocab_size_multiple = config.pad_vocab_size_multiple
        factory_kwargs = {"device": device, "dtype": dtype}

        super().__init__()
        if vocab_size % pad_vocab_size_multiple != 0:
            vocab_size += pad_vocab_size_multiple - (vocab_size % pad_vocab_size_multiple)
        self.backbone = MixerModel(
            d_model=d_model,
            n_layer=n_layer,
            d_intermediate=d_intermediate,
            vocab_size=vocab_size,
            ssm_cfg=ssm_cfg,
            attn_layer_idx=attn_layer_idx,
            attn_cfg=attn_cfg,
            rms_norm=rms_norm,
            initializer_cfg=initializer_cfg,
            fused_add_norm=fused_add_norm,
            residual_in_fp32=residual_in_fp32,
            **factory_kwargs,
        )
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False, **factory_kwargs)

        # Initialize weights and apply final processing
        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            )
        )
        self.tie_weights()

    def tie_weights(self):
        if self.config.tie_embeddings:
            self.lm_head.weight = self.backbone.embedding.weight

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.backbone.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)

    def forward(self, input_ids, position_ids=None, inference_params=None, num_last_tokens=0, **mixer_kwargs):
        """
        "position_ids" is just to be compatible with Transformer generation. We don't use it.
        num_last_tokens: if > 0, only return the logits for the last n tokens
        """
        hidden_states = self.backbone(input_ids, inference_params=inference_params, **mixer_kwargs)
        if num_last_tokens > 0:
            hidden_states = hidden_states[:, -num_last_tokens:]
        lm_logits = self.lm_head(hidden_states)
        CausalLMOutput = namedtuple("CausalLMOutput", ["logits"])
        return CausalLMOutput(logits=lm_logits)

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)
        model = cls(config, device=device, dtype=dtype, **kwargs)
        model.load_state_dict(load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype))
        return model

    def save_pretrained(self, save_directory):
        """
        Minimal implementation of save_pretrained for MambaLMHeadModel.
        Save the model and its configuration file to a directory.
        """
        # Ensure save_directory exists
        os.makedirs(save_directory, exist_ok=True)

        # Save the model's state_dict
        model_path = os.path.join(save_directory, 'pytorch_model.bin')
        torch.save(self.state_dict(), model_path)

        # Save the configuration of the model
        config_path = os.path.join(save_directory, 'config.json')
        with open(config_path, 'w') as f:
            json.dump(self.config.__dict__, f, indent=4)


In [ ]:
# @title mamba_ssm/utils/generation.py#L247
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/utils/generation.py#L247

class GenerationMixin:
    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        raise NotImplementedError

    def generate(
        self,
        input_ids,
        max_length,
        top_k=1,
        top_p=0.0,
        min_p=0.0,
        temperature=1.0,
        return_dict_in_generate=False,
        output_scores=False,
        **kwargs,
    ):
        output = decode(
            input_ids, self, max_length, top_k=top_k, top_p=top_p, min_p = min_p, temperature=temperature, output_scores=output_scores, **kwargs
        )
        if not output_scores:
            output.scores = None
        return output if return_dict_in_generate else output.sequences



In [ ]:
# @title mamba_ssm/modules/ssd_minimal.py
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/ssd_minimal.py
import torch
import torch.nn.functional as F
from einops import rearrange, repeat
from mamba_ssm.ops.triton.ssd_combined import mamba_chunk_scan_combined


def segsum_unstable(x):
    """Naive segment sum calculation."""
    T = x.size(-1)
    x_cumsum = torch.cumsum(x, dim=-1)
    x_segsum = x_cumsum[..., :, None] - x_cumsum[..., None, :]
    mask = torch.tril(torch.ones(T, T, device=x.device, dtype=bool), diagonal=0)
    x_segsum = x_segsum.masked_fill(~mask, -torch.inf)
    return x_segsum

def segsum(x):
    """More stable segment sum calculation."""
    T = x.size(-1)
    x = repeat(x, "... d -> ... d e", e=T)
    mask = torch.tril(torch.ones(T, T, device=x.device, dtype=bool), diagonal=-1)
    x = x.masked_fill(~mask, 0)
    x_segsum = torch.cumsum(x, dim=-2)
    mask = torch.tril(torch.ones(T, T, device=x.device, dtype=bool), diagonal=0)
    x_segsum = x_segsum.masked_fill(~mask, -torch.inf)
    return x_segsum

def ssd_minimal_discrete(X, A, B, C, block_len, initial_states=None):
    """
    Arguments:
        X: (batch, length, n_heads, d_head)
        A: (batch, length, n_heads)
        B: (batch, length, n_heads, d_state)
        C: (batch, length, n_heads, d_state)
    Return:
        Y: (batch, length, n_heads, d_head)
    """
    assert X.dtype == A.dtype == B.dtype == C.dtype
    assert X.shape[1] % block_len == 0

    # Rearrange into blocks/chunks
    X, A, B, C = [rearrange(x, "b (c l) ... -> b c l ...", l=block_len) for x in (X, A, B, C)]

    A = rearrange(A, "b c l h -> b h c l")
    A_cumsum = torch.cumsum(A, dim=-1)

    # 1. Compute the output for each intra-chunk (diagonal blocks)
    L = torch.exp(segsum(A))
    Y_diag  = torch.einsum("bclhn,bcshn,bhcls,bcshp->bclhp", C, B, L, X)

    # 2. Compute the state for each intra-chunk
    # (right term of low-rank factorization of off-diagonal blocks; B terms)
    decay_states = torch.exp((A_cumsum[:, :, :, -1:] - A_cumsum))
    states = torch.einsum("bclhn,bhcl,bclhp->bchpn", B, decay_states, X)

    # 3. Compute the inter-chunk SSM recurrence; produces correct SSM states at chunk boundaries
    # (middle term of factorization of off-diag blocks; A terms)
    if initial_states is None:
        initial_states = torch.zeros_like(states[:, :1])
    states = torch.cat([initial_states, states], dim=1)
    decay_chunk = torch.exp(segsum(F.pad(A_cumsum[:, :, :, -1], (1, 0))))
    new_states = torch.einsum("bhzc,bchpn->bzhpn", decay_chunk, states)
    states, final_state = new_states[:, :-1], new_states[:, -1]

    # 4. Compute state -> output conversion per chunk
    # (left term of low-rank factorization of off-diagonal blocks; C terms)
    state_decay_out = torch.exp(A_cumsum)
    Y_off = torch.einsum('bclhn,bchpn,bhcl->bclhp', C, states, state_decay_out)

    # Add output of intra-chunk and inter-chunk terms (diagonal and off-diagonal blocks)
    Y = rearrange(Y_diag+Y_off, "b c l h p -> b (c l) h p")
    return Y, final_state


# Simple test
def test_correctness():
    torch.manual_seed(42)

    ## Dimensions
    # Denoted (B, T, Q, D, P) in the paper
    batch, seqlen, chunk_size, dim, headdim = 1, 2048, 64, 2048, 64
    nheads = dim // headdim  # (H) in the paper
    ngroups = 1 # (G) in the paper
    dstate = 64  # (N) in the paper
    dtype = torch.float32
    device = "cuda"

    x = torch.randn(batch, seqlen, nheads, headdim, dtype=dtype, device=device)
    dt = F.softplus(torch.randn(batch, seqlen, nheads, dtype=torch.float32, device=device) - 4).requires_grad_()
    A = (-torch.exp(torch.rand(nheads, dtype=torch.float32, device=device))).requires_grad_()
    B = torch.randn(batch, seqlen, ngroups, dstate, dtype=dtype, device=device)
    C = torch.randn(batch, seqlen, ngroups, dstate, dtype=dtype, device=device)
    D = torch.randn(nheads, dtype=dtype, device=device)

    # Comparing fused version and minimal version
    y = mamba_chunk_scan_combined(x, dt, A, B, C, chunk_size, D=None)
    y_min, _ = ssd_minimal_discrete(x*dt.unsqueeze(-1), A*dt, B, C, chunk_size)


In [ ]:
# @title mixer_seq_simple
# https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py
import math
from functools import partial
import torch
import torch.nn as nn
from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.modules.mamba_simple import Mamba
from mamba_ssm.modules.mamba2 import Mamba2
# from mamba_ssm.modules.mha import MHA
# from mamba_ssm.modules.mlp import GatedMLP
# from mamba_ssm.modules.block import Block
from mamba_ssm.utils.generation import GenerationMixin
# from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

# try: from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
# except ImportError: RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None

import copy
def create_block(
    d_model,
    d_intermediate,
    ssm_cfg=None,
    attn_layer_idx=None,
    attn_cfg=None,
    norm_epsilon=1e-5,
    rms_norm=False,
    residual_in_fp32=False,
    fused_add_norm=False,
    layer_idx=None,
    device=None,
    dtype=None,
):
    if ssm_cfg is None:
        ssm_cfg = {}
    if attn_layer_idx is None:
        attn_layer_idx = []
    if attn_cfg is None:
        attn_cfg = {}
    factory_kwargs = {"device": device, "dtype": dtype}
    if layer_idx not in attn_layer_idx:
        # Create a copy of the config to modify
        ssm_cfg = copy.deepcopy(ssm_cfg) if ssm_cfg is not None else {}
        ssm_layer = ssm_cfg.pop("layer", "Mamba1")
        if ssm_layer not in ["Mamba1", "Mamba2"]:
            raise ValueError(f"Invalid ssm_layer: {ssm_layer}, only support Mamba1 and Mamba2")
        mixer_cls = partial(
            Mamba2 if ssm_layer == "Mamba2" else Mamba,
            layer_idx=layer_idx,
            **ssm_cfg,
            **factory_kwargs
        )
    else:
        mixer_cls = partial(MHA, layer_idx=layer_idx, **attn_cfg, **factory_kwargs)
    norm_cls = partial(
        nn.LayerNorm if not rms_norm else RMSNorm, eps=norm_epsilon, **factory_kwargs
    )
    if d_intermediate == 0:
        mlp_cls = nn.Identity
    else:
        mlp_cls = partial(
            GatedMLP, hidden_features=d_intermediate, out_features=d_model, **factory_kwargs
        )
    block = Block(
        d_model,
        mixer_cls,
        mlp_cls,
        norm_cls=norm_cls,
        fused_add_norm=fused_add_norm,
        residual_in_fp32=residual_in_fp32,
    )
    block.layer_idx = layer_idx
    return block


# https://github.com/huggingface/transformers/blob/c28d04e9e252a1a099944e325685f14d242ecdcd/src/transformers/models/gpt2/modeling_gpt2.py#L454
def _init_weights(
    module,
    n_layer,
    initializer_range=0.02,  # Now only used for embedding layer.
    rescale_prenorm_residual=True,
    n_residuals_per_layer=1,  # Change to 2 if we have MLP
):
    if isinstance(module, nn.Linear):
        if module.bias is not None:
            if not getattr(module.bias, "_no_reinit", False):
                nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, std=initializer_range)

    if rescale_prenorm_residual:
        # Reinitialize selected weights subject to the OpenAI GPT-2 Paper Scheme:
        #   > A modified initialization which accounts for the accumulation on the residual path with model depth. Scale
        #   > the weights of residual layers at initialization by a factor of 1/√N where N is the # of residual layers.
        #   >   -- GPT-2 :: https://openai.com/blog/better-language-models/
        #
        # Reference (Megatron-LM): https://github.com/NVIDIA/Megatron-LM/blob/main/megatron/model/gpt_model.py
        for name, p in module.named_parameters():
            if name in ["out_proj.weight", "fc2.weight"]:
                # Special Scaled Initialization --> There are 2 Layer Norms per Transformer Block
                # Following Pytorch init, except scale by 1/sqrt(2 * n_layer)
                # We need to reinit p since this code could be called multiple times
                # Having just p *= scale would repeatedly scale it down
                nn.init.kaiming_uniform_(p, a=math.sqrt(5))
                with torch.no_grad():
                    p /= math.sqrt(n_residuals_per_layer * n_layer)


class MixerModel(nn.Module):
    def __init__(
        self,
        d_model,
        n_layer,
        d_intermediate,
        vocab_size,
        ssm_cfg=None,
        attn_layer_idx=None,
        attn_cfg=None,
        norm_epsilon: float = 1e-5,
        rms_norm: bool = False,
        initializer_cfg=None,
        fused_add_norm=False,
        residual_in_fp32=False,
        device=None,
        dtype=None,
    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32

        self.embedding = nn.Embedding(vocab_size, d_model, **factory_kwargs)

        # We change the order of residual and layer norm:
        # Instead of LN -> Attn / MLP -> Add, we do:
        # Add -> LN -> Attn / MLP / Mixer, returning both the residual branch (output of Add) and
        # the main branch (output of MLP / Mixer). The model definition is unchanged.
        # This is for performance reason: we can fuse add + layer_norm.
        self.fused_add_norm = fused_add_norm
        if self.fused_add_norm:
            if layer_norm_fn is None or rms_norm_fn is None:
                raise ImportError("Failed to import Triton LayerNorm / RMSNorm kernels")

        self.layers = nn.ModuleList([
                create_block(
                    d_model,
                    d_intermediate=d_intermediate,
                    ssm_cfg=ssm_cfg,
                    attn_layer_idx=attn_layer_idx,
                    attn_cfg=attn_cfg,
                    norm_epsilon=norm_epsilon,
                    rms_norm=rms_norm,
                    residual_in_fp32=residual_in_fp32,
                    fused_add_norm=fused_add_norm,
                    layer_idx=i,
                    **factory_kwargs,
                )
                for i in range(n_layer)])

        self.norm_f = (nn.LayerNorm if not rms_norm else RMSNorm)(
            d_model, eps=norm_epsilon, **factory_kwargs
        )

        self.apply(partial(_init_weights, n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
                n_residuals_per_layer=1 if d_intermediate == 0 else 2,  # 2 if we have MLP
            ))

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return {
            i: layer.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)
            for i, layer in enumerate(self.layers)
        }

    def forward(self, input_ids, inference_params=None, **mixer_kwargs):
        hidden_states = self.embedding(input_ids)
        residual = None
        for layer in self.layers:
            hidden_states, residual = layer(hidden_states, residual, inference_params=inference_params, **mixer_kwargs)
        if not self.fused_add_norm:
            residual = (hidden_states + residual) if residual is not None else hidden_states
            hidden_states = self.norm_f(residual.to(dtype=self.norm_f.weight.dtype))
        else:
            # Set prenorm=False here since we don't need the residual
            hidden_states = layer_norm_fn(
                hidden_states,
                self.norm_f.weight,
                self.norm_f.bias,
                eps=self.norm_f.eps,
                residual=residual,
                prenorm=False,
                residual_in_fp32=self.residual_in_fp32,
                is_rms_norm=isinstance(self.norm_f, RMSNorm)
            )
        return hidden_states


from collections import namedtuple
class MambaLMHeadModel(nn.Module, GenerationMixin):
    def __init__(self, config: MambaConfig, initializer_cfg=None, device=None, dtype=None,) -> None:
        self.config = config
        d_model = config.d_model
        n_layer = config.n_layer
        d_intermediate = config.d_intermediate
        vocab_size = config.vocab_size
        ssm_cfg = config.ssm_cfg
        attn_layer_idx = config.attn_layer_idx
        attn_cfg = config.attn_cfg
        rms_norm = config.rms_norm
        residual_in_fp32 = config.residual_in_fp32
        fused_add_norm = config.fused_add_norm
        pad_vocab_size_multiple = config.pad_vocab_size_multiple
        factory_kwargs = {"device": device, "dtype": dtype}

        super().__init__()
        if vocab_size % pad_vocab_size_multiple != 0:
            vocab_size += pad_vocab_size_multiple - (vocab_size % pad_vocab_size_multiple)
        self.backbone = MixerModel(
            d_model=d_model,
            n_layer=n_layer,
            d_intermediate=d_intermediate,
            vocab_size=vocab_size,
            ssm_cfg=ssm_cfg,
            attn_layer_idx=attn_layer_idx,
            attn_cfg=attn_cfg,
            rms_norm=rms_norm,
            initializer_cfg=initializer_cfg,
            fused_add_norm=fused_add_norm,
            residual_in_fp32=residual_in_fp32,
            **factory_kwargs,
        )
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False, **factory_kwargs)

        # Initialize weights and apply final processing
        self.apply(partial(_init_weights, n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            ))
        self.tie_weights()

    def tie_weights(self):
        if self.config.tie_embeddings:
            self.lm_head.weight = self.backbone.embedding.weight

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.backbone.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)

    def forward(self, input_ids, position_ids=None, inference_params=None, num_last_tokens=0, **mixer_kwargs):
        """
        "position_ids" is just to be compatible with Transformer generation. We don't use it.
        num_last_tokens: if > 0, only return the logits for the last n tokens
        """
        hidden_states = self.backbone(input_ids, inference_params=inference_params, **mixer_kwargs)
        if num_last_tokens > 0:
            hidden_states = hidden_states[:, -num_last_tokens:]
        lm_logits = self.lm_head(hidden_states)
        CausalLMOutput = namedtuple("CausalLMOutput", ["logits"])
        return CausalLMOutput(logits=lm_logits)




# model_path = os.path.join(save_directory, 'pytorch_model.bin')
# torch.save(self.state_dict(), model_path)
# model.load_state_dict(load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype))



## basement

In [ ]:
# @title seq conv lcse
# ht = at*ht-1 + bt

def seq(at, bt, h0=None): # [t,b,d], [t,b,d], [b,d]
    h = 0 if h0==None else h0
    ht = []
    # for t in range(at.size(0)):
    for t in range(at.size(1)):
        # h = at[t] * h + bt[t]
        h = at[:,t] * h + bt[:,t]
        ht.append(h)
    # ht = torch.stack(ht, dim=0)
    ht = torch.stack(ht, dim=1)
    return ht

# conv
# h1=a1*h0+b1 =       a1*h0 +       b1
# h2=a2*h1+b2 =    a2*a1*h0 +    a2*b1 +    b2
# h3=a3*h2+b3 = a3*a2*a1*h0 + a3*a2*b1 + a3*b2 + b3

# h1/a1   = h0 + b1/a1
# h2/a12  = h0 + b1/a1 + b2/a12
# h3/a123 = h0 + b1/a1 + b2/a12 + b3/a123

def conv(at, bt, h0=None): # [t,b,d], [t,b,d], [b,d]
    # a123 = torch.cumprod(at, dim=0) # [t,b,d]
    # ht = torch.cumsum(bt/a123, dim=0) # cusum b1/a1, b2/a12, b3/a123, ...
    a123 = torch.cumprod(at, dim=1) # [t,b,d]
    ht = torch.cumsum(bt/a123, dim=1) # cusum b1/a1, b2/a12, b3/a123, ...
    ht = ((h0 if h0!=None else 0)+ht)*a123
    return ht

# ht = A* ht-1 + B*ut ; yt = C *ht + D*ut
# let kernel K: Km = CA^mB

# K = C @ A**torch.arange(1,t+1) @ B
# u = F.pad(u, (t-1, 0))
# y = F.conv1d(u, K) + D*u

# fft
# L_fft = 2 * T
# uf = torch.fft.rfft(u, n=L_fft)            # (B, Din, L_fft//2+1)
# kf = torch.fft.rfft(K, n=L_fft)            # (Dout, Din, L_fft//2+1)
# # Multiply + sum over input channels
# # out[b, dout, f] = sum_{din} uf[b,din,f] * kf[dout,din,f]
# yf = torch.einsum("bdf, odf -> bof", uf, kf)
# y = torch.fft.irfft(yf, n=L_fft)[..., :T]   # causal output


# lcse
# computing the sequence ht = at*ht-1 + bt
# xt = e^(a*_t +tail(LCSE(cat(log x0, log b_t -a*_t))))
# b*_t = Sum t] e^( log bt - a*_t)
# a*_t = Sum t] log a_t
def lcse(at, bt, h0=None): # [t,b,d], [t,b,d], [b,d]
    if h0!=None:
        # x_in = torch.cat([h0.log().unsqueeze(0), bt.log()], dim=0) # [1+t,b,d]
        # at = torch.cat([torch.zeros(1,*at.shape[1:]), torch.cumsum(at.log(), dim=0)], dim=0) # a*_t # [1+t,b,d]
        # ht = torch.exp(at[1:] + torch.logcumsumexp(x_in - at, dim=0)[1:])
        x_in = torch.cat([h0.log().unsqueeze(1), bt.log()], dim=1) # [b,1+t,d]
        at = torch.cat([torch.zeros(at.shape[0],1,*at.shape[2:]), torch.cumsum(at.log(), dim=1)], dim=1) # a*_t # [1+t,b,d]
        ht = torch.exp(at[:,1:] + torch.logcumsumexp(x_in - at, dim=1)[:,1:])
    else:
        # at = torch.cumsum(at.log(), dim=0) # a*_t # [t,b,d]
        # ht = torch.exp(at + torch.logcumsumexp(bt.log() - at, dim=0))
        at = torch.cumsum(at.log(), dim=1) # a*_t # [t,b,d]
        ht = torch.exp(at + torch.logcumsumexp(bt.log() - at, dim=1))
    return ht


t, b, d = 30, 4, 5
# dA = torch.exp(dt*A)[...,None,None] # [b,t,h,1,1]
# dBx = torch.einsum("bth,btgs,bthd->bthds", dt, B, x)
# at = torch.randn(b,t,d)
# bt = torch.randn(b,t,d)
at = torch.randn(b,t,d,1,1).exp()
bt = torch.randn(b,t,d,d,d)

h0 = torch.randn(b,d, dtype=torch.complex64)
h0 = None
ht = seq(at, bt, h0)
# ht = seq(at+0j, bt+0j, h0)
# ht1 = conv(at, bt, h0)
# ht1 = lcse(at, bt, h0)
ht1 = lcse(at+0j, bt+0j, h0).real
print(abs(ht-ht1).sum())
# print(ht[:3,0,:5])
# print(ht1[:3,0,:5])

# cpu; t, b, d = 200, 64, 64
# %timeit seq(at, bt, h0) # 6.16 ms 6.09 6.25
# %timeit conv(at, bt, h0) # 25.6 ms 24.2
# %timeit lcse(at, bt, h0) # 344 ms 348 ms



In [ ]:
# @title test lcse
def lcse(at, bt, h0=None): # [t,b,d], [t,b,d], [b,d]
    if h0!=None:
        # x_in = torch.cat([h0.log().unsqueeze(1), bt.log()], dim=1) # [b,1+t,d]
        # at = torch.cat([torch.zeros(at.shape[0],1,*at.shape[2:]), torch.cumsum(at.log(), dim=1)], dim=1) # a*_t # [1+t,b,d]
        # ht = torch.exp(at[:,1:] + torch.logcumsumexp(x_in - at, dim=1)[:,1:])
        at = torch.cumsum(at.log(), dim=1) # a*_t # [b,t,d]
        ht = torch.exp(at + torch.logcumsumexp(torch.cat([h0.log(), bt.log() - at], dim=1))[:,1:])
    else:
        at = torch.cumsum(at.log(), dim=1) # a*_t # [b,t,d]
        # print(at[:3,0,:5])
        ht = torch.exp(at + torch.logcumsumexp(bt.log() - at, dim=1))
        # ht = torch.exp(at + torch.logcumsumexp(bt.log() - at))
    return ht


t, b, d = 30, 4, 5
# t, b, d = 200, 64, 64
at = torch.randn(b,t,d)*10
bt = torch.randn(b,t,d)*10

h0 = torch.randn(b,d, dtype=torch.complex64)
h0 = None
ht = seq(at, bt, h0)
# ht = seq(at+0j, bt+0j, h0)
# ht1 = lcse(at, bt, h0)
ht1 = lcse(at+0j, bt+0j, h0).real
print(abs(ht-ht1).sum())
print(ht[:3,0,:5])
print(ht1[:3,0,:5])


In [ ]:
# @title mamba seq lcse
# x:[b,t,h,d], B/C:[b,t,g,s]
# A:[h], dt:[b,t,h]

b,t,h,d = 3,256,4,4
g,s = 4,4
x = torch.randn(b,t,h,d)
B = torch.randn(b,t,g,s)
C = torch.randn(b,t,g,s)
A = torch.randn(h)
dt = torch.randn(b,t,h)


# y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64, h0) # 256
# y, h_ssm = ssd(x*dt.unsqueeze(-1), A*dt, B, C, 64) # 256
# print(y.shape, h_ssm.shape)

# dA = torch.exp(dt*A) # [b,n_heads] # https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/triton/selective_state_update.py#L104
# # print('dt, B, x', dt.shape, B.shape, x.shape)
# dBx = torch.einsum("bh,bhs,bhd->bhds", dt, B, x)
# if h==None: h_ssm = dBx
# else: h_ssm = h[1] * dA[...,None,None] + dBx # [b,h,d,s]*[b,h,1,1]+[b,h,d,s]
# # print('h.ssm_state, C', h.ssm_state.shape, C.shape)

# h = at[t] * h + bt[t]


dA = torch.exp(dt*A)[...,None,None] # [b,t,h,1,1]
dBx = torch.einsum("bth,btgs,bthd->bthds", dt, B, x)
# print(dA.shape, dBx.shape)
print(dA[0,:15].squeeze())
# dA = torch.randn(b,t,h,1,1)
# dBx = torch.randn(b,t,h,d,s)

# h: [b,h,d,s]
# h = at[t] * h + bt[t]
# print(dA[:,:5])
# print(dBx[:,:5])


h_ssm = seq(dA, dBx, h0=None) # [b,t,d], [b,t,d], [b,d]
# h_ssm = seq(dA+0j, dBx+0j, h0=None) # [b,t,d], [b,t,d], [b,d]
print(h_ssm.shape)
# h_ssm1 = lcse(dA+0j, dBx+0j, h0=None) # [b,t,d], [b,t,d], [b,d]
h_ssm1 = lcse(dA+0j, dBx+0j, h0=None).real # [b,t,d], [b,t,d], [b,d]
# print(h_ssm.shape)
# print(h_ssm[0,:3])
# print(h_ssm1[0,:3])
print(abs(h_ssm-h_ssm1).sum())
# print((h_ssm-h_ssm1)[0,:3])
# print((h_ssm-h_ssm1)[0])

# # y = torch.einsum("bhds,bgs->bhd", h_ssm, C) # [b,h,d,s]*[b,h,s]
y1 = torch.einsum("bthds,btgs->bthd", h_ssm, C) # [b,h,d,s]*[b,h,s]
# print(y1.shape)
# print(y[0,:3])
# print(y1[0,:3])
# print(abs(y-y1).sum())


In [ ]:
# @title huggingface mamba2
# https://huggingface.co/docs/transformers/en/model_doc/mamba2
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/mamba2/modeling_mamba2.py#L825
import torch
import torch.nn as nn
from transformers import Mamba2Config, Mamba2Model

# import torch.nn as nn
# class Id(nn.Module):
#     def __init__(self): super().__init__()
#     def forward(self, x): return x
# pred.embeddings = Id()

batch=4
seq_len=5
in_dim=132
d_model=128#256
# num_heads*head_dim=2*hidden_size ?

config = Mamba2Config(hidden_size=d_model, num_heads=8, head_dim=32, state_size=128, num_hidden_layers=1, output_hidden_states=True, vocab_size=0)
pred = Mamba2Model(config)
pred.embeddings = nn.Linear(in_dim, d_model)

x = torch.randn(batch, seq_len, in_dim)

# out = pred(inputs_embeds=x, use_cache=True)
out = pred(x, use_cache=True)
y, cache = out.last_hidden_state, out.cache_params
# print(x)
# print(y)
print(x.shape) # [batch, seq_len, d_model]
print(y.shape) # [batch, seq_len, d_model]
# print(len(h)) # num_hidden_layers + 1 ?
# h = torch.cat(out.hidden_states, dim=-1) # [batch, seq_len, (num_hidden_layers + 1 ?) * d_model]
# print(h.shape)
print(cache)

#  if key/value cache contains 10 tokens (no matter how many of it is a pad token), the cache position for the next token should be torch.tensor([10]) https://huggingface.co/docs/transformers/main/en/kv_cache
# out = pred(inputs_embeds=x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
out = pred(x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
y, cache = out.last_hidden_state, out.cache_params

print(y.shape)
print(len(h)) # num_hidden_layers + 1 ?
# print(h.shape)






x = torch.randn(batch, seq_len, in_dim)
out = pred(x, use_cache=True)
y, cache = out.last_hidden_state, out.cache_params
h = torch.cat(out.hidden_states, dim=-1) # [batch, seq_len, (num_hidden_layers + 1 ?) * d_model]
out = pred(x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
y, cache = out.last_hidden_state, out.cache_params


# Mamba2Output
# Mamba2Cache
pred = Mamba2Model(Mamba2Config(hidden_size=d_model, num_heads=8, head_dim=32, state_size=128, num_hidden_layers=1, output_hidden_states=True, vocab_size=0))

self.ssm_states = {
    i: torch.zeros(batch_size, config.num_heads, config.head_dim, config.state_size, device=device, dtype=dtype)
    for i in range(config.num_hidden_layers)
}


# x = torch.randn(4*batch, seq_len, in_dim)
# out = pred(x, cache_params=cache, use_cache=True, cache_position=x.shape[1]) # torch.tensor([10])
# y, cache = out.last_hidden_state, out.cache_params

# print(cache)
# print(cache.ssm_states)
# print(len(cache.ssm_states))

# config = Mamba2Config(hidden_size=d_model, num_heads=8, head_dim=32, state_size=128, num_hidden_layers=1, output_hidden_states=True, vocab_size=0)
# print(config.num_heads, config.head_dim, config.state_size)
# print(config.num_hidden_layers)
x = x.detach().repeat(4,1,1,1) # [batch, num_heads, head_dim, state_size]
for i,v in cache.ssm_states.items():
    print(i,v.shape)





In [ ]:
# @title johnma2006/mamba-minimal
# https://github.com/johnma2006/mamba-minimal/blob/master/model.py
"""Simple, minimal implementation of Mamba in one file of PyTorch.
    [1] Mamba: Linear-Time Sequence Modeling with Selective State Spaces (Albert Gu and Tri Dao) https://arxiv.org/abs/2312.00752
    [2] The Annotated S4 (Sasha Rush and Sidd Karamcheti) https://srush.github.io/annotated-s4

Glossary:
    b: batch size                       (`B` in Mamba paper [1] Algorithm 2)
    l: sequence length                  (`L` in [1] Algorithm 2)
    d or d_model: hidden dim
    n or d_state: latent state dim      (`N` in [1] Algorithm 2)
    expand: expansion factor            (`E` in [1] Section 3.4)
    d_in or d_inner: d * expand         (`D` in [1] Algorithm 2)
    A, B, C, D: state space parameters  (See any state space representation formula)
                                        (B, C are input-dependent (aka selective, a key innovation in Mamba); A, D are not)
    Δ or delta: input-dependent step size
    dt_rank: rank of Δ                  (See [1] Section 3.6 "Parameterization of ∆")
"""
import torch
import torch.nn as nn
import torch.nn.functional as F

class Mamba(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        # self.embedding = nn.Embedding(args.vocab_size, args.d_model)
        self.layers = nn.ModuleList([ResidualBlock(args) for _ in range(args.n_layer)])
        self.norm_f = nn.RMSNorm(args.d_model)
        # self.lm_head = nn.Linear(args.d_model, args.vocab_size, bias=False)
        # self.lm_head.weight = self.embedding.weight  # Tie output projection to embedding weights. # See "Weight Tying" paper

    def forward(self, input_ids):
        """ Args: input_ids (long tensor): shape (b, l)    (See Glossary at top for definitions of b, l, d_in, n...)
        Returns: logits: shape (b, l, vocab_size)
        Official Implementation: class MambaLMHeadModel, https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py#L173"""
        # x = self.embedding(input_ids)
        x = input_ids

        for layer in self.layers:
            x = layer(x)
        x = self.norm_f(x)
        # logits = self.lm_head(x)
        logits = x
        return logits

class ResidualBlock(nn.Module):
    def __init__(self, args):
        """Simple block wrapping Mamba block with normalization and residual connection."""
        super().__init__()
        # self.args = args
        self.mixer = MambaBlock(args)
        self.norm = nn.RMSNorm(args.d_model)

    def forward(self, x): # (b, l, d)
        """Official Implementation: Block.forward(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L297
            Note: the official repo chains residual blocks that look like
                [Add -> Norm -> Mamba] -> [Add -> Norm -> Mamba] -> [Add -> Norm -> Mamba] -> ...
            where the first Add is a no-op. This is purely for performance reasons as this
            allows them to fuse the Add->Norm.
            We instead implement our blocks as the more familiar, simpler, and numerically equivalent
                [Norm -> Mamba -> Add] -> [Norm -> Mamba -> Add] -> [Norm -> Mamba -> Add] -> ...."""
        output = x + self.mixer(self.norm(x))
        return output # (b, l, d)


from einops import rearrange, repeat, einsum
class MambaBlock(nn.Module):
    def __init__(self, args):
        """A single Mamba block, as described in Figure 3 in Section 3.4 in the Mamba paper [1]."""
        super().__init__()
        self.args = args
        self.in_proj = nn.Linear(args.d_model, args.d_inner * 2, bias=args.bias)
        self.conv1d = nn.Conv1d(in_channels=args.d_inner, out_channels=args.d_inner, bias=args.conv_bias,
            kernel_size=args.d_conv, groups=args.d_inner, padding=args.d_conv - 1,)

        self.x_proj = nn.Linear(args.d_inner, args.dt_rank + args.d_state * 2, bias=False) # takes in `x` and outputs the input-specific Δ, B, C
        self.dt_proj = nn.Linear(args.dt_rank, args.d_inner, bias=True) # project Δ from dt_rank to d_in

        A = repeat(torch.arange(1, args.d_state + 1), 'n -> d n', d=args.d_inner)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(args.d_inner))
        self.out_proj = nn.Linear(args.d_inner, args.d_model, bias=args.bias)


    def forward(self, x): # (b, l, d)
        """Mamba block forward. This looks the same as Figure 3 in Section 3.4 in the Mamba paper [1].
        Official Implementation: class Mamba, https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L119
            mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311"""
        b, l, d = x.shape
        x_res = self.in_proj(x)  # shape (b, l, 2 * d_in)
        x, res = x_res.split([self.args.d_inner, self.args.d_inner], dim=-1)
        x = self.conv1d(x.permute(0, 2, 1))[:, :, :l].permute(0, 2, 1) # (b, l, d_in)
        y = self.ssm(F.silu(x)) * F.silu(res)
        output = self.out_proj(y)
        return output # (b, l, d)


    def ssm(self, x): # (b, l, d_in)
        """- Algorithm 2 in Section 3.2 in the Mamba paper [1]
            - run_SSM(A, B, C, u) in The Annotated S4 [2]
        Official Implementation: mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311"""
        d_in, n = self.A_log.shape
        # Compute ∆ A B C D, the state space parameters.
        #     A, D are input independent (see Mamba paper [1] Section 3.5.2 "Interpretation of A" for why A isn't selective)
        #     ∆, B, C are input-dependent (this is a key difference between Mamba and the linear time invariant S4, and is why Mamba is called **selective** state spaces)
        A = -torch.exp(self.A_log.float())  # shape (d_in, n)
        D = self.D.float()

        # 2: B : (B, L, N) ← sB (x)
        # 3: C : (B, L, N) ← sC (x)
        # 4: Δ : (B, L, D) ← 𝜏Δ(Parameter+𝑠Δ(x))
        # 5: A, B : (B, L, D, N) ← discretize(Δ, A, B)
        # 6: y ← SSM(A, B, C) (x)

        x_dbl = self.x_proj(x)  # (b, l, dt_rank + 2*n)
        delta, B, C = x_dbl.split([self.args.dt_rank, n, n], dim=-1)  # delta: (b, l, dt_rank). B, C: (b, l, n)
        delta = F.softplus(self.dt_proj(delta)) # (b, l, d_in)
        y = self.selective_scan(x, delta, A, B, C, D)  # This is similar to run_SSM(A, B, C, u) in The Annotated S4 [2]
        return y # (b, l, d_in)


    def selective_scan(self, u, delta, A, B, C, D): # u: (b, l, d_in); delta: (b, l, d_in); A: (d_in, n); B: (b, l, n); C: (b, l, n); D: (d_in,)
        """- Section 2 State Space Models in the Mamba paper [1]
            - Algorithm 2 in Section 3.2 in the Mamba paper [1]
            - run_SSM(A, B, C, u) in The Annotated S4 [2]

            x(t+1) = Ax(t) + Bu(t)
            y(t)   = Cx(t) + Du(t)
        except B and C (and the step size delta, which is used for discretization) are dependent on the input x(t).

        Official Implementation: selective_scan_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L86
            Note: I refactored some parts out of `selective_scan_ref` out, so the functionality doesn't match exactly."""
        b, l, d_in = u.shape
        n = A.shape[1]

        # Discretize continuous parameters (A, B)
        deltaA = torch.exp(einsum(delta, A, 'b l d_in, d_in n -> b l d_in n')) # - A is discretized using zero-order hold (ZOH) discretization (see Section 2 Equation 4 in the Mamba paper [1])
        deltaB_u = einsum(delta, B, u, 'b l d_in, b l n, b l d_in -> b l d_in n') # - B is discretized using a simplified Euler discretization instead of ZOH. From a discussion with authors: "A is the more important term and the performance doesn't change much with the simplification on B"

        # Perform selective scan (see scan_SSM() in The Annotated S4 [2])
        # Note that the below is sequential, while the official implementation does a much faster parallel scan that is additionally hardware-aware (like FlashAttention).
        x = torch.zeros((b, d_in, n), device=deltaA.device)
        ys = []
        for i in range(l):
            x = deltaA[:, i] * x + deltaB_u[:, i]
            y = einsum(x, C[:, i, :], 'b d_in n, b n -> b d_in')
            ys.append(y)
        y = torch.stack(ys, dim=1)  # shape (b, l, d_in)
        y = y + u * D
        return y # (b, l, d_in)


import math
from dataclasses import dataclass
@dataclass
class ModelArgs:
    d_model = 256
    n_layer = 1
    # vocab_size = 0
    d_state = 16
    expand = 2
    d_conv = 4
    # pad_vocab_size_multiple = 8
    conv_bias = True
    bias = False
    d_inner = int(expand * d_model)
    dt_rank = math.ceil(d_model / 16)


pred = Mamba(ModelArgs)

batch=4
seq_len=500
d_model=256
x = torch.randn(batch, seq_len, d_model)
y = pred(x)
# print(x)
# print(y)




In [ ]:
# print(h)
# for j in h:
#     print(j.shape)

print(cache)


In [ ]:
# @title test einops
# from einops import repeat, einsum
# b,l,d_in,n = 4, 500, 256, 256
# # einsum(A,B,'b l d, b l d -> b l d')

# delta = torch.randn(b,l,d_in)
# B = torch.randn(b,l,n)
# u = torch.randn(b,l,d_in)
# x0=einsum(delta, B, u, 'b l d_in, b l n, b l d_in -> b l d_in n')
# x1=torch.einsum('bld,bln,bld->bldn', delta, B, u)
# print((x0-x1).sum())

# A=torch.randn(b,d_in,n)
# C=torch.randn(b,d_in)
# y0 = einsum(A, C, 'b d_in n, b n -> b d_in')
# y1 = torch.einsum('bdn,bn->bd', A, C)
# print((y0-y1).sum())


# A = repeat(torch.arange(1, n + 1), 'n -> d n', d=d_in)
# print(A)
# A1 = torch.arange(1, n + 1).repeat(d_in, 1)
# print((A-A1).sum())

# %timeit x0=einsum(delta, B, u, 'b l d_in, b l n, b l d_in -> b l d_in n') # 304 ms, 300
# %timeit x1=torch.einsum('bld,bln,bld->bldn', delta, B, u) # 303 ms, 301, 301
# %timeit y0 = einsum(A, C, 'b d_in n, b n -> b d_in') # 159 µs, 160
# %timeit y1 = torch.einsum('bdn,bn->bd', A, C) # 155, 146
# %timeit pred(x) # ein 862ms, 866, 977, 984 ; torch 849, 959, 867, 924
